In [ ]:
import os

# List all uploaded files
os.listdir()

['.config',
 'horror_imdb_enriched_uncompressed.csv',
 'drive',
 'horror_imdb_enriched.csv.gz',
 'the_numbers_horror_raw.csv',
 'enhanced_box_office_data(2000-2024)u (1).csv',
 'horror_movies_tmdb_enriched - horror_movies_tmdb_enriched.csv.csv',
 'sample_data']

In [ ]:
import pandas as pd

# Load the dataset (compressed CSV)
horror = pd.read_csv("horror_imdb_enriched.csv.gz", compression="gzip")

# Quick checks
print("Rows:", len(horror))
print("Columns:", horror.columns.tolist())

# See a preview
horror.head()

Rows: 44741
Columns: ['tconst', 'primaryTitle', 'originalTitle', 'startYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'language_best', 'region_best', 'DirectorsNames', 'WritersNames', 'TopCast']


,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,language_best,region_best,DirectorsNames,WritersNames,TopCast
0,tt0003419,The Student of Prague,Der Student von Prag,1913.0,83.0,"Drama,Fantasy,Horror",6.4,2616.0,en,US,"Hanns Heinz Ewers, Stellan Rye","Hanns Heinz Ewers, Alfred de Musset","Paul Wegener, Grete Berger"
1,tt0003643,The Avenging Conscience: or 'Thou Shalt Not Kill',The Avenging Conscience: or 'Thou Shalt Not Kill',1914.0,78.0,"Crime,Drama,Horror",6.4,1549.0,ja,US,D.W. Griffith,"Edgar Allan Poe, D.W. Griffith","Henry B. Walthall, Spottiswoode Aitken"
2,tt0004013,The Ghost Breaker,The Ghost Breaker,1914.0,60.0,"Adventure,Horror",4.7,52.0,NaN,NaN,"Oscar Apfel, Cecil B. DeMille","Oscar Apfel, Cecil B. DeMille","H.B. Warner, Rita Stanwood"
3,tt0004026,The Golem,Der Golem,1914.0,60.0,Horror,6.7,1305.0,ja,US,"Paul Wegener, Henrik Galeen","Paul Wegener, Henrik Galeen","Paul Wegener, Henrik Galeen"
4,tt0004121,The Hound of the Baskervilles (1914),Der Hund von Baskerville,1914.0,65.0,"Crime,Horror,Mystery",5.6,183.0,en,AT,Rudolf Meinert,"Arthur Conan Doyle, Richard Oswald","Alwin Neuß, Friedrich Kühne"


In [ ]:
# Step 6: Clean up columns and check missing values
keep_cols = [
    "tconst","primaryTitle","originalTitle","startYear","runtimeMinutes","genres",
    "averageRating","numVotes","language_best","region_best",
    "DirectorsNames","WritersNames","TopCast"
]

horror_clean = horror[keep_cols].copy()

# Check what percent of each column is missing
missing = horror_clean.isna().mean().sort_values(ascending=False)
print("Missing % by column:\n", (missing*100).round(1))

# Quick look
print("\nSample:")
print(horror_clean.head(10))
print("\nFinal rows:", len(horror_clean))

Missing % by column:
 language_best     60.8
region_best       60.8
numVotes          42.7
averageRating     42.7
runtimeMinutes    36.9
startYear         27.8
TopCast           18.6
DirectorsNames    11.2
WritersNames       9.6
originalTitle      0.0
primaryTitle       0.0
tconst             0.0
genres             0.0
dtype: float64

Sample:
      tconst                                       primaryTitle  \
0  tt0003419                              The Student of Prague   
1  tt0003643  The Avenging Conscience: or 'Thou Shalt Not Kill'   
2  tt0004013                                  The Ghost Breaker   
3  tt0004026                                          The Golem   
4  tt0004121               The Hound of the Baskervilles (1914)   
5  tt0005231                      The Hound of the Baskervilles   
6  tt0005513  Der Hund von Baskerville, 3. Teil - Das unheim...   
7  tt0005514                  Der Hund von Baskerville, 4. Teil   
8  tt0005529                                      Bl

In [ ]:
horror_clean.to_csv("horror_imdb_enriched.csv.gz", index=False, compression="gzip")
print("✅ Final version saved")

✅ Final version saved


In [ ]:
horror.to_csv("horror_imdb_enriched_uncompressed.csv", index=False)


## Merging imdb with tmdb

In [ ]:
# --- STEP 1: Imports ---
import pandas as pd
import numpy as np

# --- STEP 2: Load both datasets ---
imdb = pd.read_csv("/content/horror_imdb_enriched_uncompressed.csv", low_memory=False)
tmdb = pd.read_csv("/content/horror_movies_tmdb_enriched - horror_movies_tmdb_enriched.csv.csv", low_memory=False)

# --- STEP 3: Helper functions ---
def std_title(s):
    s = s.fillna("").astype(str).str.lower()
    s = s.str.replace(r"[^0-9a-z\s]", " ", regex=True)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

def extract_year_from_date(col):
    dt = pd.to_datetime(col, errors="coerce")
    return dt.dt.year

# --- STEP 4: Standardize titles and years ---
# IMDb
imdb["year"] = pd.to_numeric(imdb["startYear"], errors="coerce").astype("Int64")
imdb["title_std_primary"]  = std_title(imdb["primaryTitle"])
imdb["title_std_original"] = std_title(imdb["originalTitle"])

# TMDb
tmdb["title_std"] = std_title(tmdb["title"])
tmdb["year"] = extract_year_from_date(tmdb["release_date"]).astype("Int64")

# --- STEP 5: Keep only relevant TMDb columns ---
tmdb_slim = tmdb[["id", "title_std", "year", "release_date", "overview", "original_language"]].drop_duplicates(subset=["title_std", "year"])

# --- STEP 6: Merge by primaryTitle first ---
merged = imdb.merge(
    tmdb_slim,
    left_on=["title_std_primary", "year"],
    right_on=["title_std", "year"],
    how="left",
    suffixes=("", "_tmdb")
)

# --- STEP 7: For IMDb rows still missing release_date, try matching on originalTitle ---
missing_mask = merged["release_date"].isna()

second_merge = imdb.loc[missing_mask, ["tconst", "title_std_original", "year"]].merge(
    tmdb_slim,
    left_on=["title_std_original", "year"],
    right_on=["title_std", "year"],
    how="left"
)

# Fill missing values from the second merge
for col in ["release_date", "overview", "original_language"]:
    merged.loc[missing_mask, col] = second_merge[col].values

# --- STEP 8: Rename and supplement IMDb language ---
merged.rename(columns={
    "release_date": "release_date_tmdb",
    "overview": "overview_tmdb",
    "original_language": "original_language_tmdb"
}, inplace=True)

merged["language_best_filled"] = merged["language_best"]
missing_lang = merged["language_best_filled"].isna() | (merged["language_best_filled"].str.strip() == "")
merged.loc[missing_lang, "language_best_filled"] = merged.loc[missing_lang, "original_language_tmdb"]




In [ ]:
# --- STEP 9: Verify results ---
total = len(merged)
has_release = merged["release_date_tmdb"].notna().sum()
has_overview = merged["overview_tmdb"].notna().sum()
has_language = merged["language_best_filled"].notna().sum()

print("✅ Merge Complete")
print(f"Total IMDb movies: {total:,}")
print(f"Got release_date_tmdb for {has_release:,} ({has_release/total:.1%})")
print(f"Got overview_tmdb for {has_overview:,} ({has_overview/total:.1%})")
print(f"Filled language_best for {has_language:,} ({has_language/total:.1%})")

print("\n🧩 Sample:")
print(merged[["primaryTitle","year","release_date_tmdb","language_best","language_best_filled","overview_tmdb"]].head(10))


✅ Merge Complete
Total IMDb movies: 44,741
Got release_date_tmdb for 18,182 (40.6%)
Got overview_tmdb for 17,923 (40.1%)
Filled language_best for 23,778 (53.1%)

🧩 Sample:
                                        primaryTitle  year release_date_tmdb  \
0                              The Student of Prague  1913               NaN   
1  The Avenging Conscience: or 'Thou Shalt Not Kill'  1914               NaN   
2                                  The Ghost Breaker  1914        1914-12-07   
3                                          The Golem  1914               NaN   
4               The Hound of the Baskervilles (1914)  1914               NaN   
5                      The Hound of the Baskervilles  1915               NaN   
6  Der Hund von Baskerville, 3. Teil - Das unheim...  1916               NaN   
7                  Der Hund von Baskerville, 4. Teil  1916               NaN   
8                                      Blind Justice  1916        1916-09-25   
9                           

In [ ]:
# --- STEP 10: Save verified dataset ---
merged.to_csv("horror_imdb_tmdb_enriched.csv.gz", index=False, compression="gzip")
print("\n💾 Saved as horror_imdb_tmdb_enriched.csv.gz in your current Colab directory")


💾 Saved as horror_imdb_tmdb_enriched.csv.gz in your current Colab directory


In [ ]:
import os
os.listdir("/content")

['.config',
 'horror_imdb_enriched_uncompressed.csv',
 'drive',
 'horror_imdb_enriched.csv.gz',
 'horror_imdb_tmdb_enriched.csv.gz',
 'the_numbers_horror_raw.csv',
 'enhanced_box_office_data(2000-2024)u (1).csv',
 'horror_movies_tmdb_enriched - horror_movies_tmdb_enriched.csv.csv',
 'sample_data']

# Getting more dates

Seeing if it works:

In [ ]:
import requests

api_key = "da7f82afbb332b194570f410bec399f2"  # put your key in quotes

test = requests.get(
    f"https://api.themoviedb.org/3/movie/550?api_key={api_key}"  # 550 = Fight Club
).json()

print(test["release_date"])  # should print "1999-10-15"

1999-10-15


### Getting dates

In [ ]:
# ============================
# FAST TMDb date fetcher (async, resume-safe, backoff + recovery)
# ============================

import os, time, asyncio
import pandas as pd
from datetime import datetime

try:
    import aiohttp, nest_asyncio
except ImportError:
    !pip -q install aiohttp nest_asyncio
    import aiohttp, nest_asyncio

nest_asyncio.apply()  # makes asyncio happy inside notebooks

# ---- CONFIG ----
API_KEY = os.getenv("TMDB_KEY") or "da7f82afbb332b194570f410bec399f2"  # set env or paste your key
INPUT_FILE  = "horror_imdb_tmdb_enriched.csv.gz"             # spine with 'tconst' + 'release_date_tmdb'
OUTPUT_FILE = "horror_imdb_tmdb_with_api_dates.csv.gz"       # final enriched output
CACHE_FILE  = "tmdb_date_cache.csv"                          # imdb_id -> release_date cache

LOG_EVERY   = 250       # print progress every N fetched
SAVE_EVERY  = 1000      # save cache every N fetched

# Speed controls
CONCURRENCY     = 15     # parallel workers (~req/sec if BASE_DELAY_SEC=1.0)
BASE_DELAY_SEC  = 1.0    # per-worker base delay (faster -> lower)
MIN_DELAY_SEC   = 0.5    # never go below this per-worker delay
MAX_DELAY_SEC   = 4.0    # cap when backing off after 429
BACKOFF_FACTOR  = 1.7    # multiply delay on 429
COOLDOWN_SEC    = 5.0    # extra sleep after 429 before retry

if API_KEY == "PUT_YOUR_TMDB_KEY_HERE":
    raise ValueError("Set your TMDB key: os.environ['TMDB_KEY']='...' or edit API_KEY above.")

# ---- Load input ----
df = pd.read_csv(INPUT_FILE, compression="gzip", low_memory=False)
if "tconst" not in df.columns or "release_date_tmdb" not in df.columns:
    raise ValueError("Expected 'tconst' and 'release_date_tmdb' in input. Run the TMDb merge step first.")

needs_df = df[df["release_date_tmdb"].isna()][["tconst"]].drop_duplicates()
print(f"🎬 Missing release_date_tmdb: {len(needs_df):,}")

# ---- Load cache (resume) ----
if os.path.exists(CACHE_FILE):
    cache = pd.read_csv(CACHE_FILE, dtype={"tconst": str, "release_date_api": str})
    cache_map = dict(zip(cache["tconst"], cache["release_date_api"]))
    print(f"📦 Loaded cache: {len(cache_map):,} entries.")
else:
    cache_map = {}
    print("📦 No cache file yet; starting fresh.")

todo_ids = [tid for tid in needs_df["tconst"].tolist() if tid not in cache_map]
print(f"📝 Remaining to fetch: {len(todo_ids):,}")

# ---- Shared state for async workers ----
delay_sec = BASE_DELAY_SEC           # shared mutable delay across workers
delay_lock = asyncio.Lock()          # guard delay updates
save_counter = 0
save_lock = asyncio.Lock()

async def save_cache_now():
    """Write cache_map to disk (async-safe)."""
    global save_counter
    async with save_lock:
        if save_counter == 0:
            return
        out = pd.DataFrame({"tconst": list(cache_map.keys()), "release_date_api": list(cache_map.values())})
        out.to_csv(CACHE_FILE, index=False)
        save_counter = 0
        print(f"💾 Cache saved ({len(out):,} total cached).")

async def fetch_one(session: aiohttp.ClientSession, imdb_id: str):
    """Fetch release date for one IMDb id with backoff and recovery."""
    global delay_sec, save_counter
    url = f"https://api.themoviedb.org/3/find/{imdb_id}"
    params = {"api_key": API_KEY, "external_source": "imdb_id"}

    attempts = 0
    while True:
        attempts += 1
        try:
            async with session.get(url, params=params, timeout=15) as resp:
                if resp.status == 200:
                    data = await resp.json()
                    movies = data.get("movie_results", [])
                    date_val = (movies[0] or {}).get("release_date") if movies else None
                    cache_map[imdb_id] = date_val
                    save_counter += 1

                    # 🔽 RECOVERY: speed up gradually after successful requests
                    async with delay_lock:
                        delay_sec = max(delay_sec * 0.95, BASE_DELAY_SEC)  # ease toward base
                    break

                elif resp.status == 429:
                    # 🔼 BACKOFF: slow down globally
                    async with delay_lock:
                        delay_sec = min(delay_sec * BACKOFF_FACTOR, MAX_DELAY_SEC)
                        local_delay = delay_sec
                    print(f"⚠️ 429 for {imdb_id}. Backing off. New per-worker delay ~{local_delay:.2f}s")
                    await asyncio.sleep(COOLDOWN_SEC)
                else:
                    # Record None on non-200 and move on
                    cache_map[imdb_id] = None
                    save_counter += 1
                    break

        except (asyncio.TimeoutError, aiohttp.ClientError):
            if attempts >= 3:
                cache_map[imdb_id] = None
                save_counter += 1
                break
            await asyncio.sleep(1.0)

    # Polite pacing per worker (adapts via delay_sec)
    async with delay_lock:
        local_sleep = max(delay_sec, MIN_DELAY_SEC)
    await asyncio.sleep(local_sleep)

async def worker(name, session, queue, total_count, progress):
    while True:
        imdb_id = await queue.get()
        if imdb_id is None:
            queue.task_done()
            break

        await fetch_one(session, imdb_id)

        progress["done"] += 1
        done = progress["done"]

        if done % LOG_EVERY == 0:
            elapsed = time.time() - progress["start"]
            rate = done / elapsed if elapsed > 0 else 0.0
            remain = total_count - done
            eta = int(remain / max(rate, 1e-6))
            print(f"✓ Queried {done:,}/{total_count:,} | rate ~{rate:.1f}/s | ETA ~{eta//60}m {eta%60}s")

        if done % SAVE_EVERY == 0:
            await save_cache_now()

        queue.task_done()

async def main_async(ids):
    total = len(ids)
    q = asyncio.Queue()
    progress = {"done": 0, "start": time.time()}

    # preload tasks
    for x in ids: q.put_nowait(x)
    for _ in range(CONCURRENCY): q.put_nowait(None)  # sentinel per worker

    connector = aiohttp.TCPConnector(limit=CONCURRENCY)
    async with aiohttp.ClientSession(connector=connector) as session:
        workers = [asyncio.create_task(worker(f"W{i+1}", session, q, total, progress))
                   for i in range(CONCURRENCY)]
        await q.join()
        for w in workers:
            await w

# ---- Run ----
if len(todo_ids) == 0:
    print("Nothing to fetch — everything is cached already.")
else:
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(main_async(todo_ids))
    finally:
        loop.run_until_complete(save_cache_now())

# ---- Apply cache and save final dataset ----
df["release_date_api"]  = df["tconst"].map(cache_map)
df["release_date_final"] = df["release_date_tmdb"].combine_first(df["release_date_api"])

added   = pd.Series(df["release_date_api"]).notna().sum()
final   = pd.Series(df["release_date_final"]).notna().sum()
missing = len(df) - final

print("\n✅ TMDb API enrichment done (or paused safely)")
print(f"New dates in cache (this run): {added:,}")
print(f"Total with any date (TMDb merge + API): {final:,}")
print(f"Still missing: {missing:,} ({missing/len(df):.1%})")




🎬 Missing release_date_tmdb: 26,559
📦 Loaded cache: 340 entries.
📝 Remaining to fetch: 26,219
✓ Queried 250/26,219 | rate ~12.3/s | ETA ~35m 4s
✓ Queried 500/26,219 | rate ~12.1/s | ETA ~35m 28s
✓ Queried 750/26,219 | rate ~12.1/s | ETA ~34m 56s
✓ Queried 1,000/26,219 | rate ~12.1/s | ETA ~34m 43s
💾 Cache saved (1,352 total cached).
✓ Queried 1,250/26,219 | rate ~12.1/s | ETA ~34m 19s
✓ Queried 1,500/26,219 | rate ~12.1/s | ETA ~33m 59s
✓ Queried 1,750/26,219 | rate ~12.1/s | ETA ~33m 39s
✓ Queried 2,000/26,219 | rate ~12.1/s | ETA ~33m 18s
💾 Cache saved (2,350 total cached).
✓ Queried 2,250/26,219 | rate ~12.1/s | ETA ~32m 59s
✓ Queried 2,500/26,219 | rate ~12.1/s | ETA ~32m 37s
✓ Queried 2,750/26,219 | rate ~12.1/s | ETA ~32m 17s
✓ Queried 3,000/26,219 | rate ~12.1/s | ETA ~31m 56s
💾 Cache saved (3,352 total cached).
✓ Queried 3,250/26,219 | rate ~12.1/s | ETA ~31m 36s
✓ Queried 3,500/26,219 | rate ~12.1/s | ETA ~31m 17s
✓ Queried 3,750/26,219 | rate ~12.1/s | ETA ~30m 55s
✓ Queried 

In [ ]:

ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_path = f"horror_imdb_tmdb_with_api_dates_{ts}.csv.gz"
df.to_csv(out_path, index=False, compression="gzip")
print(f"💾 Saved: {out_path}")


💾 Saved: horror_imdb_tmdb_with_api_dates_20251016_221937.csv.gz


Seeing whats missing

In [ ]:
import glob, pandas as pd

# Auto-pick latest timestamped file you saved earlier
paths = sorted(glob.glob("horror_imdb_tmdb_with_api_dates_*.csv.gz"))
df_path = paths[-1] if paths else "horror_imdb_tmdb_enriched.csv.gz"
print("Using:", df_path)

df = pd.read_csv(df_path, compression="gzip", low_memory=False)

# Have a final date column? If not, create from TMDb + prior API fills
if "release_date_final" not in df.columns:
    if "release_date_api" not in df.columns:
        df["release_date_api"] = pd.NA
    df["release_date_final"] = df["release_date_tmdb"].combine_first(df["release_date_api"])

missing_dates = df["release_date_final"].isna().sum()
print(f"Still missing release date: {missing_dates:,} / {len(df):,}")


Using: horror_imdb_tmdb_with_api_dates_20251016_221937.csv.gz
Still missing release date: 19,341 / 44,741


Verifying the api key works:

In [ ]:
import requests

OMDB_KEY = "694598b8"
r = requests.get(f"http://www.omdbapi.com/?i=tt0081505&apikey={OMDB_KEY}")  # The Shining
print(r.status_code)
print(r.json())


200
{'Title': 'The Shining', 'Year': '1980', 'Rated': 'R', 'Released': '13 Jun 1980', 'Runtime': '146 min', 'Genre': 'Drama, Horror', 'Director': 'Stanley Kubrick', 'Writer': 'Stephen King, Stanley Kubrick, Diane Johnson', 'Actors': 'Jack Nicholson, Shelley Duvall, Danny Lloyd', 'Plot': 'A family heads to an isolated hotel for the winter, where a sinister presence influences the father into violence. At the same time, his psychic son sees horrifying forebodings from both the past and the future.', 'Language': 'English', 'Country': 'United Kingdom, United States', 'Awards': '6 wins & 9 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNmM5ZThhY2ItOGRjOS00NzZiLWEwYTItNDgyMjFkOTgxMmRiXkEyXkFqcGc@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '84%'}, {'Source': 'Metacritic', 'Value': '68/100'}], 'Metascore': '68', 'imdbRating': '8.4', 'imdbVotes': '1,183,648', 'imdbID': 'tt0081505', 'Type': 

Getting data using the OMDB Api

In [ ]:
# ============================
# Horror Data Master — OMDb Enrichment (with progress bar + versioned save)
# ============================

import os, time, glob, requests
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm.notebook import tqdm

# ---------- CONFIG ----------
OMDB_KEY = os.getenv("OMDB_KEY") or "694598b8"   # Paste your OMDb key
CACHE_FILE = "omdb_cache.csv"
DELAY = 0.02            # seconds between requests (~12–13 req/sec safe)
BATCH_SAVE = 1000       # save cache every N requests
VERSION = 1             # bump manually for new versions
OUTPUT_FILE = f"horror_data_master_v{VERSION}.csv.gz"

# ---------- FIND SPINE ----------
paths = sorted(glob.glob("horror_imdb_tmdb_with_api_dates_*.csv.gz"))
if not paths:
    raise FileNotFoundError("No 'with_api_dates_' file found. Make sure it’s in the Colab files panel.")
SPINE = paths[-1]
print(f"📂 Using spine: {SPINE}")

df = pd.read_csv(SPINE, compression="gzip", low_memory=False)
print(f"Loaded {len(df):,} movies from spine")

# ---------- LOAD / INIT CACHE ----------
if os.path.exists(CACHE_FILE):
    cache = pd.read_csv(CACHE_FILE, dtype=str)
    cache_map = {row["tconst"]: row.to_dict() for _, row in cache.iterrows()}  # all dicts
    print(f"🔁 Loaded OMDb cache with {len(cache_map):,} entries")
else:
    cache_map = {}
    print("🆕 Starting fresh cache")


todo = [t for t in df["tconst"].dropna().unique() if t not in cache_map]
print(f"🎬 Movies to fetch: {len(todo):,}")

# ---------- FETCH FUNCTION ----------
def fetch_omdb(imdb_id: str) -> dict:
    url = "http://www.omdbapi.com/"
    params = {"i": imdb_id, "apikey": OMDB_KEY, "type": "movie"}
    try:
        r = requests.get(url, params=params, timeout=10)
        data = r.json() if r.status_code == 200 else {}
        if not data or data.get("Response") != "True":
            return {"tconst": imdb_id}
        return {
            "tconst": imdb_id,
            "Title": data.get("Title"),
            "Year": data.get("Year"),
            "Rated": data.get("Rated"),
            "Released": data.get("Released"),
            "Runtime": data.get("Runtime"),
            "Genre": data.get("Genre"),
            "Director": data.get("Director"),
            "Writer": data.get("Writer"),
            "Actors": data.get("Actors"),
            "Plot": data.get("Plot"),
            "Language": data.get("Language"),
            "Country": data.get("Country"),
            "Awards": data.get("Awards"),
            "imdbRating": data.get("imdbRating"),
            "imdbVotes": data.get("imdbVotes"),
            "Metascore": data.get("Metascore"),
            "BoxOffice": data.get("BoxOffice"),
        }
    except Exception:
        return {"tconst": imdb_id}

# ---------- FETCH LOOP WITH PROGRESS BAR ----------
if todo:
    for imdb_id in tqdm(todo, desc="Fetching OMDb data", unit="movie"):
        cache_map[imdb_id] = fetch_omdb(imdb_id)
        if len(cache_map) % BATCH_SAVE == 0:
            save_cache_safe(cache_map, CACHE_FILE)
        time.sleep(DELAY)

# Final save of cache
save_cache_safe(cache_map, CACHE_FILE)

print(f"💾 Cache saved: {len(cache_map):,} entries")

# ---------- BUILD OMDb DF ----------
omdb = pd.DataFrame.from_records(list(cache_map.values()))
print(f"📊 OMDb data: {len(omdb):,} rows, {len(omdb.columns)} cols")

# ---------- CLEAN / RENAME ----------
def to_num(series):
    return (pd.Series(series, dtype="object")
              .astype(str)
              .str.replace(r"[^\d.]", "", regex=True)
              .replace({"": np.nan, "nan": np.nan})
              .astype(float))

def parse_date(series):
    return pd.to_datetime(series, errors="coerce")

rename_map = {
    "Released": "release_date_omdb",
    "Language": "language_omdb_raw",
    "Country": "country_omdb_raw",
    "imdbRating": "imdb_rating_omdb",
    "imdbVotes": "imdb_votes_omdb",
    "Metascore": "metascore_omdb",
    "BoxOffice": "revenue_domestic_omdb",
    "Rated": "rated_omdb",
    "Runtime": "runtime_omdb",
    "Genre": "genre_omdb",
    "Director": "director_omdb",
    "Writer": "writer_omdb",
    "Actors": "actors_omdb",
    "Plot": "plot_omdb",
    "Awards": "awards_omdb",
}
omdb.rename(columns=rename_map, inplace=True)
omdb["release_date_omdb"] = parse_date(omdb["release_date_omdb"])
omdb["imdb_rating_omdb"] = to_num(omdb["imdb_rating_omdb"])
omdb["imdb_votes_omdb"] = to_num(omdb["imdb_votes_omdb"])
omdb["metascore_omdb"] = to_num(omdb["metascore_omdb"])
omdb["revenue_domestic_omdb"] = to_num(omdb["revenue_domestic_omdb"])

# ---------- MERGE ----------
df2 = df.merge(omdb, on="tconst", how="left")

# ---------- MAP LANGUAGE / COUNTRY ----------
lang_map = {
    "english":"en","spanish":"es","french":"fr","german":"de","italian":"it","japanese":"ja",
    "chinese":"zh","mandarin":"zh","cantonese":"zh","korean":"ko","russian":"ru","hindi":"hi",
    "portuguese":"pt","polish":"pl","turkish":"tr","thai":"th","danish":"da","swedish":"sv",
    "norwegian":"no","finnish":"fi","dutch":"nl","greek":"el","czech":"cs","hungarian":"hu",
    "romanian":"ro","bulgarian":"bg","serbian":"sr","croatian":"hr","slovak":"sk","ukrainian":"uk",
    "arabic":"ar","persian":"fa","hebrew":"he","indonesian":"id","malay":"ms","vietnamese":"vi"
}
country_map = {
    "united states":"US","usa":"US","united kingdom":"GB","uk":"GB","canada":"CA","australia":"AU",
    "germany":"DE","france":"FR","italy":"IT","spain":"ES","mexico":"MX","japan":"JP","china":"CN",
    "hong kong":"HK","south korea":"KR","india":"IN","brazil":"BR","argentina":"AR","russia":"RU",
    "poland":"PL","netherlands":"NL","sweden":"SE","norway":"NO","denmark":"DK","finland":"FI",
    "ireland":"IE","new zealand":"NZ","thailand":"TH","turkey":"TR","romania":"RO","bulgaria":"BG",
    "hungary":"HU","greece":"GR","czech republic":"CZ","portugal":"PT","belgium":"BE","switzerland":"CH",
    "austria":"AT","south africa":"ZA"
}

df2["language_omdb_code"] = df2["language_omdb_raw"].apply(
    lambda x: lang_map.get(str(x).split(",")[0].strip().lower(), np.nan) if pd.notna(x) else np.nan
)
df2["country_omdb_code"] = df2["country_omdb_raw"].apply(
    lambda x: country_map.get(str(x).split(",")[0].strip().lower(), np.nan) if pd.notna(x) else np.nan
)

# ---------- COALESCE INTO FINAL COLUMNS ----------
df2["release_date_final"] = pd.to_datetime(df2.get("release_date_final"), errors="coerce").combine_first(df2["release_date_omdb"])
df2["averageRating_final"] = df2.get("averageRating").combine_first(df2["imdb_rating_omdb"])
df2["numVotes_final"] = df2.get("numVotes").combine_first(df2["imdb_votes_omdb"])
df2["rated"] = df2.get("rated").combine_first(df2["rated_omdb"])
df2["revenue_domestic"] = df2.get("revenue_domestic").combine_first(df2["revenue_domestic_omdb"])

# Provenance tracking
df2["release_date_source"] = np.where(df2.get("release_date_tmdb").notna(), "tmdb", np.where(df2["release_date_omdb"].notna(), "omdb", pd.NA))
df2["language_source"] = np.where(df2["language_omdb_code"].notna(), "omdb", pd.NA)
df2["region_source"] = np.where(df2["country_omdb_code"].notna(), "omdb", pd.NA)
df2["rated_source"] = np.where(df2["rated_omdb"].notna(), "omdb", pd.NA)
df2["revenue_domestic_source"] = np.where(df2["revenue_domestic_omdb"].notna(), "omdb", pd.NA)



📂 Using spine: horror_imdb_tmdb_with_api_dates_20251016_221937.csv.gz
Loaded 44,741 movies from spine
🔁 Loaded OMDb cache with 8,000 entries
🎬 Movies to fetch: 36,741


Fetching OMDb data:   0%|          | 0/36741 [00:00<?, ?movie/s]

AttributeError: 'dict' object has no attribute 'dtype'

In [ ]:
# --- HOTFIX: normalize cache and resume safely ---

import pandas as pd
import numpy as np

# 1) Rebuild cache_map so all values are dicts (not Series)
if 'cache_map' in globals():
    norm = {}
    for k, v in cache_map.items():
        if isinstance(v, dict):
            d = dict(v)
        elif hasattr(v, 'to_dict'):
            d = v.to_dict()
        else:
            d = {}
        d.setdefault('tconst', k)
        norm[k] = d
    cache_map = norm
    print(f"✅ Normalized cache_map to dicts: {len(cache_map):,} entries")

# 2) Replace the batch save with a safe version
def save_cache_safe(cm, path="omdb_cache.csv"):
    rows = []
    for k, v in cm.items():
        if isinstance(v, dict):
            d = v
        elif hasattr(v, 'to_dict'):
            d = v.to_dict()
        else:
            d = {}
        d.setdefault('tconst', k)
        rows.append(d)
    pd.DataFrame.from_records(rows).to_csv(path, index=False)
    print(f"💾 Cache saved safely: {len(rows):,} rows")

print("You can now re-run the fetch loop cell. It will use save_cache_safe() below.")

# 3) Monkey-patch your fetch loop’s save call (for convenience)
# If your loop uses: pd.DataFrame.from_records(list(cache_map.values())).to_csv(...)
# replace that line with: save_cache_safe(cache_map)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/horror_master
!cp -n omdb_cache.csv /content/drive/MyDrive/horror_master/
!cp -n horror_data_master_v1.csv.gz /content/drive/MyDrive/horror_master/
print("✅ Backed up cache + master to Drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat 'horror_data_master_v1.csv.gz': No such file or directory
✅ Backed up cache + master to Drive.


In [ ]:
# ---------- SAVE ----------
df2.to_csv(OUTPUT_FILE, index=False, compression="gzip")
print(f"\n✅ Saved: {OUTPUT_FILE}")

New cells

In [ ]:
# ============================
# Cell 1: Setup, load spine, load/normalize cache, helpers
# ============================

import os, time, glob, requests
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm.notebook import tqdm

# ---------- CONFIG ----------
OMDB_KEY = os.getenv("OMDB_KEY") or "694598b8"  # paste your key or set env var
CACHE_FILE = "omdb_cache.csv"
DELAY = 0.02          # seconds between requests (~50 req/sec; bump to 0.03 if you see throttling)
BATCH_SAVE = 1000     # save cache every N requests
VERSION = 1           # bump each time you produce a new master
OUTPUT_FILE = f"horror_data_master_v{VERSION}.csv.gz"

# ---------- FIND & LOAD SPINE ----------
paths = sorted(glob.glob("horror_imdb_tmdb_with_api_dates_*.csv.gz"))
if not paths:
    raise FileNotFoundError("No 'with_api_dates_' file found. Make sure it’s in the Colab files panel.")
SPINE = paths[-1]
print(f"📂 Using spine: {SPINE}")

df = pd.read_csv(SPINE, compression="gzip", low_memory=False)
print(f"Loaded {len(df):,} movies from spine")

# Ensure key baseline columns exist (won't overwrite if present)
for need in ["tconst", "release_date_tmdb", "language_best", "region_best", "averageRating", "numVotes"]:
    if need not in df.columns:
        df[need] = pd.NA

# If a unified final date doesn't exist yet, initialize it from prior TMDb/API passes
if "release_date_final" not in df.columns:
    if "release_date_api" not in df.columns:
        df["release_date_api"] = pd.NA
    df["release_date_final"] = pd.to_datetime(
        pd.Series(df["release_date_tmdb"]).combine_first(df["release_date_api"]),
        errors="coerce"
    )

# ---------- LOAD / INIT CACHE (as dicts) ----------
if os.path.exists(CACHE_FILE):
    cache = pd.read_csv(CACHE_FILE, dtype=str)
    # Load rows as dicts to avoid mixed types later
    cache_map = {row["tconst"]: row.to_dict() for _, row in cache.iterrows()}
    print(f"🔁 Loaded OMDb cache with {len(cache_map):,} entries")
else:
    cache_map = {}
    print("🆕 Starting fresh cache")

# ---------- NORMALIZE CACHE (safety) ----------
# Ensure all cache_map values are dicts and include tconst
norm = {}
for k, v in cache_map.items():
    if isinstance(v, dict):
        d = dict(v)
    elif hasattr(v, "to_dict"):
        d = v.to_dict()
    else:
        d = {}
    d.setdefault("tconst", k)
    norm[k] = d
cache_map = norm
print(f"✅ Normalized cache_map to dicts: {len(cache_map):,} entries")

# ---------- HELPERS ----------
def save_cache_safe(cm, path=CACHE_FILE):
    rows = []
    for k, v in cm.items():
        if isinstance(v, dict):
            d = v
        elif hasattr(v, "to_dict"):
            d = v.to_dict()
        else:
            d = {}
        d.setdefault("tconst", k)
        rows.append(d)
    pd.DataFrame.from_records(rows).to_csv(path, index=False)
    print(f"💾 Cache saved safely: {len(rows):,} rows")

def fetch_omdb(imdb_id: str) -> dict:
    """Fetch one title from OMDb. Returns a dict of fields (or minimal dict on failure)."""
    url = "http://www.omdbapi.com/"
    params = {"i": imdb_id, "apikey": OMDB_KEY, "type": "movie"}
    try:
        r = requests.get(url, params=params, timeout=10)
        data = r.json() if r.status_code == 200 else {}
        if not data or data.get("Response") != "True":
            return {"tconst": imdb_id}
        return {
            "tconst": imdb_id,
            "Title": data.get("Title"),
            "Year": data.get("Year"),
            "Rated": data.get("Rated"),
            "Released": data.get("Released"),
            "Runtime": data.get("Runtime"),
            "Genre": data.get("Genre"),
            "Director": data.get("Director"),
            "Writer": data.get("Writer"),
            "Actors": data.get("Actors"),
            "Plot": data.get("Plot"),
            "Language": data.get("Language"),
            "Country": data.get("Country"),
            "Awards": data.get("Awards"),
            "imdbRating": data.get("imdbRating"),
            "imdbVotes": data.get("imdbVotes"),
            "Metascore": data.get("Metascore"),
            "BoxOffice": data.get("BoxOffice"),
        }
    except Exception:
        return {"tconst": imdb_id}

print("✅ Cell 1 ready")

📂 Using spine: horror_imdb_tmdb_with_api_dates_20251016_221937.csv.gz
Loaded 44,741 movies from spine
🔁 Loaded OMDb cache with 44,741 entries
✅ Normalized cache_map to dicts: 44,741 entries
✅ Cell 1 ready


In [ ]:
# ============================
# Cell 2: Build TODO and fetch OMDb with progress bar
# ============================

if OMDB_KEY == "PUT_YOUR_OMDB_KEY_HERE":
    raise ValueError("Set OMDB_KEY (env var or inline) before running the fetch.")

todo = [t for t in df["tconst"].dropna().unique() if t not in cache_map]
print(f"🎬 Movies to fetch: {len(todo):,}")

if todo:
    for imdb_id in tqdm(todo, desc="Fetching OMDb data", unit="movie"):
        cache_map[imdb_id] = fetch_omdb(imdb_id)
        # periodic cache save
        if len(cache_map) % BATCH_SAVE == 0:
            save_cache_safe(cache_map, CACHE_FILE)
        time.sleep(DELAY)

# Final cache save
save_cache_safe(cache_map, CACHE_FILE)
print("✅ Fetch complete (or nothing to do)")

🎬 Movies to fetch: 36,741


Fetching OMDb data:   0%|          | 0/36741 [00:00<?, ?movie/s]

💾 Cache saved safely: 9,000 rows
💾 Cache saved safely: 10,000 rows
💾 Cache saved safely: 11,000 rows
💾 Cache saved safely: 12,000 rows
💾 Cache saved safely: 13,000 rows
💾 Cache saved safely: 14,000 rows
💾 Cache saved safely: 15,000 rows
💾 Cache saved safely: 16,000 rows
💾 Cache saved safely: 17,000 rows
💾 Cache saved safely: 18,000 rows
💾 Cache saved safely: 19,000 rows
💾 Cache saved safely: 20,000 rows
💾 Cache saved safely: 21,000 rows
💾 Cache saved safely: 22,000 rows
💾 Cache saved safely: 23,000 rows
💾 Cache saved safely: 24,000 rows
💾 Cache saved safely: 25,000 rows
💾 Cache saved safely: 26,000 rows
💾 Cache saved safely: 27,000 rows
💾 Cache saved safely: 28,000 rows
💾 Cache saved safely: 29,000 rows
💾 Cache saved safely: 30,000 rows
💾 Cache saved safely: 31,000 rows
💾 Cache saved safely: 32,000 rows
💾 Cache saved safely: 33,000 rows
💾 Cache saved safely: 34,000 rows
💾 Cache saved safely: 35,000 rows
💾 Cache saved safely: 36,000 rows
💾 Cache saved safely: 37,000 rows
💾 Cache saved s

In [ ]:
len(cache_map)

44741

In [ ]:
# ============================
# Cell 3: Build and clean OMDb DataFrame
# ============================

# Build DataFrame from cache_map
omdb = pd.DataFrame.from_records(list(cache_map.values()))
print(f"📊 OMDb data: {len(omdb):,} rows, {len(omdb.columns)} cols")

# Cleaners
def to_num(series):
    return (pd.Series(series, dtype="object")
              .astype(str)
              .str.replace(r"[^\d.]", "", regex=True)
              .replace({"": np.nan, "nan": np.nan})
              .astype(float))

def parse_date(series):
    return pd.to_datetime(series, errors="coerce")

# Rename to _omdb fields
rename_map = {
    "Released": "release_date_omdb",
    "Language": "language_omdb_raw",
    "Country": "country_omdb_raw",
    "imdbRating": "imdb_rating_omdb",
    "imdbVotes": "imdb_votes_omdb",
    "Metascore": "metascore_omdb",
    "BoxOffice": "revenue_domestic_omdb",
    "Rated": "rated_omdb",
    "Runtime": "runtime_omdb",
    "Genre": "genre_omdb",
    "Director": "director_omdb",
    "Writer": "writer_omdb",
    "Actors": "actors_omdb",
    "Plot": "plot_omdb",
    "Awards": "awards_omdb",
}
omdb = omdb.rename(columns=rename_map)

# Parse / convert
omdb["release_date_omdb"] = parse_date(omdb["release_date_omdb"])
omdb["imdb_rating_omdb"]  = to_num(omdb["imdb_rating_omdb"])
omdb["imdb_votes_omdb"]   = to_num(omdb["imdb_votes_omdb"])
omdb["metascore_omdb"]    = to_num(omdb["metascore_omdb"])
omdb["revenue_domestic_omdb"] = to_num(omdb["revenue_domestic_omdb"])

print("✅ OMDb frame cleaned")
omdb.head(3)

📊 OMDb data: 44,741 rows, 18 cols
✅ OMDb frame cleaned


,tconst,Title,Year,rated_omdb,release_date_omdb,runtime_omdb,genre_omdb,director_omdb,writer_omdb,actors_omdb,plot_omdb,language_omdb_raw,country_omdb_raw,awards_omdb,imdb_rating_omdb,imdb_votes_omdb,metascore_omdb,revenue_domestic_omdb
0,tt0003419,The Student of Prague,1913,Approved,1913-09-01,83 min,"Drama, Fantasy, Horror","Hanns Heinz Ewers, Stellan Rye","Hanns Heinz Ewers, Alfred de Musset","Paul Wegener, Grete Berger, Lyda Salmonova",The poor student Balduin sells his mirror imag...,"German, English",Germany,NaN,6.4,2612.0,NaN,NaN
1,tt0003643,The Avenging Conscience: or 'Thou Shalt Not Kill',1914,Not Rated,1914-08-24,78 min,"Crime, Drama, Horror",D.W. Griffith,"Edgar Allan Poe, D.W. Griffith","Henry B. Walthall, Spottiswoode Aitken, Blanch...",Prevented from dating his sweetheart by his un...,English,United States,NaN,6.4,1502.0,NaN,NaN
2,tt0004013,The Ghost Breaker,1914,NaN,1914-12-07,60 min,"Adventure, Horror","Oscar Apfel, Cecil B. DeMille","Oscar Apfel, Cecil B. DeMille, Paul Dickey","H.B. Warner, Rita Stanwood, Theodore Roberts",The treasure of the Aragon family has never be...,"None, English",United States,NaN,5.2,39.0,NaN,NaN


In [ ]:
# ============================
# Cell 4: Merge, map codes, coalesce, provenance
# ============================

# Merge OMDb columns onto spine
df2 = df.merge(omdb, on="tconst", how="left")

# Maps for language / country -> codes
lang_map = {
    "english":"en","spanish":"es","french":"fr","german":"de","italian":"it","japanese":"ja",
    "chinese":"zh","mandarin":"zh","cantonese":"zh","korean":"ko","russian":"ru","hindi":"hi",
    "portuguese":"pt","polish":"pl","turkish":"tr","thai":"th","danish":"da","swedish":"sv",
    "norwegian":"no","finnish":"fi","dutch":"nl","greek":"el","czech":"cs","hungarian":"hu",
    "romanian":"ro","bulgarian":"bg","serbian":"sr","croatian":"hr","slovak":"sk","ukrainian":"uk",
    "arabic":"ar","persian":"fa","hebrew":"he","indonesian":"id","malay":"ms","vietnamese":"vi"
}
country_map = {
    "united states":"US","usa":"US","united kingdom":"GB","uk":"GB","canada":"CA","australia":"AU",
    "germany":"DE","france":"FR","italy":"IT","spain":"ES","mexico":"MX","japan":"JP","china":"CN",
    "hong kong":"HK","south korea":"KR","india":"IN","brazil":"BR","argentina":"AR","russia":"RU",
    "poland":"PL","netherlands":"NL","sweden":"SE","norway":"NO","denmark":"DK","finland":"FI",
    "ireland":"IE","new zealand":"NZ","thailand":"TH","turkey":"TR","romania":"RO","bulgaria":"BG",
    "hungary":"HU","greece":"GR","czech republic":"CZ","portugal":"PT","belgium":"BE","switzerland":"CH",
    "austria":"AT","south africa":"ZA"
}

df2["language_omdb_code"] = df2["language_omdb_raw"].apply(
    lambda x: lang_map.get(str(x).split(",")[0].strip().lower(), np.nan) if pd.notna(x) else np.nan
)
df2["country_omdb_code"] = df2["country_omdb_raw"].apply(
    lambda x: country_map.get(str(x).split(",")[0].strip().lower(), np.nan) if pd.notna(x) else np.nan
)

# Coalesce into your canonical columns (supplement only when missing)
df2["release_date_final"] = pd.to_datetime(df2.get("release_date_final"), errors="coerce").combine_first(df2["release_date_omdb"])
df2["averageRating_final"] = df2.get("averageRating").combine_first(df2["imdb_rating_omdb"])
df2["numVotes_final"]      = df2.get("numVotes").combine_first(df2["imdb_votes_omdb"])
df2["rated"] = df2.get("rated", pd.Series([pd.NA]*len(df2))).combine_first(df2["rated_omdb"])
df2["revenue_domestic"] = df2.get("revenue_domestic", pd.Series([pd.NA]*len(df2))).combine_first(df2["revenue_domestic_omdb"])
# Provenance
df2["release_date_source"] = np.where(df2.get("release_date_tmdb").notna(), "tmdb",
                               np.where(df2["release_date_omdb"].notna(), "omdb", pd.NA))
df2["language_source"]     = np.where(df2["language_omdb_code"].notna(), "omdb", pd.NA)
df2["region_source"]       = np.where(df2["country_omdb_code"].notna(), "omdb", pd.NA)
df2["rated_source"]        = np.where(df2["rated_omdb"].notna(), "omdb", pd.NA)
df2["revenue_domestic_source"] = np.where(df2["revenue_domestic_omdb"].notna(), "omdb", pd.NA)

print("✅ Merge + coalesce complete")
df2.head(3)

✅ Merge + coalesce complete


/tmp/ipython-input-1778259497.py:39: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df2["revenue_domestic"] = df2.get("revenue_domestic", pd.Series([pd.NA]*len(df2))).combine_first(df2["revenue_domestic_omdb"])


,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,language_best,region_best,...,country_omdb_code,averageRating_final,numVotes_final,rated,revenue_domestic,release_date_source,language_source,region_source,rated_source,revenue_domestic_source
0,tt0003419,The Student of Prague,Der Student von Prag,1913.0,83.0,"Drama,Fantasy,Horror",6.4,2616.0,en,US,...,DE,6.4,2616.0,Approved,NaN,omdb,omdb,omdb,omdb,<NA>
1,tt0003643,The Avenging Conscience: or 'Thou Shalt Not Kill',The Avenging Conscience: or 'Thou Shalt Not Kill',1914.0,78.0,"Crime,Drama,Horror",6.4,1549.0,ja,US,...,US,6.4,1549.0,Not Rated,NaN,omdb,omdb,omdb,omdb,<NA>
2,tt0004013,The Ghost Breaker,The Ghost Breaker,1914.0,60.0,"Adventure,Horror",4.7,52.0,NaN,NaN,...,US,4.7,52.0,NaN,NaN,tmdb,<NA>,omdb,<NA>,<NA>


In [ ]:
# ============================
# Cell 5: Save versioned master + optional Drive backup
# ============================

# Save locally
df2.to_csv(OUTPUT_FILE, index=False, compression="gzip")
print(f"✅ Saved master: {OUTPUT_FILE}")

# Optional: back up to Drive
BACKUP_TO_DRIVE = True
if BACKUP_TO_DRIVE:
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        !mkdir -p /content/drive/MyDrive/horror_master
        !cp -n {CACHE_FILE} /content/drive/MyDrive/horror_master/
        !cp -n {OUTPUT_FILE} /content/drive/MyDrive/horror_master/
        print("✅ Backed up cache + master to Drive.")
    except Exception as e:
        print("Drive backup skipped or failed:", e)

✅ Saved master: horror_data_master_v1.csv.gz
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Backed up cache + master to Drive.


In [ ]:
# ============================
# Cell 6: Coverage summary
# ============================

total = len(df2)
def pct(n):
    return f"{(n/total):.1%}" if total else "0.0%"

have_date   = df2["release_date_final"].notna().sum()
have_lang   = (df2.get("language_best_filled", df2.get("language_best"))\
               .combine_first(df2["language_omdb_code"])).notna().sum()
have_region = (df2.get("region_best").combine_first(df2["country_omdb_code"]) if "region_best" in df2.columns else df2["country_omdb_code"]).notna().sum()
have_meta   = df2["metascore_omdb"].notna().sum()
have_rated  = df2["rated"].notna().sum()
have_rev    = df2["revenue_domestic"].notna().sum()

print("📊 Coverage after OMDb enrichment")
print(f"release_date_final: {have_date:,}/{total:,} ({pct(have_date)})")
print(f"language (best or omdb code): {have_lang:,}/{total:,} ({pct(have_lang)})")
print(f"region (best or omdb code): {have_region:,}/{total:,} ({pct(have_region)})")
print(f"metascore_omdb: {have_meta:,}/{total:,} ({pct(have_meta)})")
print(f"rated: {have_rated:,}/{total:,} ({pct(have_rated)})")
print(f"revenue_domestic: {have_rev:,}/{total:,} ({pct(have_rev)})")

df2.filter(items=[
    "tconst","primaryTitle" if "primaryTitle" in df2.columns else "originalTitle",
    "release_date_final","release_date_source",
    "averageRating","averageRating_final","numVotes","numVotes_final",
    "rated","rated_source",
    "language_best","language_omdb_raw","language_omdb_code","language_source",
    "region_best","country_omdb_raw","country_omdb_code","region_source",
    "revenue_domestic","revenue_domestic_source",
    "metascore_omdb"
]).head(10)

📊 Coverage after OMDb enrichment
release_date_final: 30,984/44,741 (69.3%)
language (best or omdb code): 32,811/44,741 (73.3%)
region (best or omdb code): 32,988/44,741 (73.7%)
metascore_omdb: 1,934/44,741 (4.3%)
rated: 10,143/44,741 (22.7%)
revenue_domestic: 1,547/44,741 (3.5%)


,tconst,primaryTitle,release_date_final,release_date_source,averageRating,averageRating_final,numVotes,numVotes_final,rated,rated_source,...,language_omdb_raw,language_omdb_code,language_source,region_best,country_omdb_raw,country_omdb_code,region_source,revenue_domestic,revenue_domestic_source,metascore_omdb
0,tt0003419,The Student of Prague,1913-08-22,omdb,6.4,6.4,2616.0,2616.0,Approved,omdb,...,"German, English",de,omdb,US,Germany,DE,omdb,NaN,<NA>,NaN
1,tt0003643,The Avenging Conscience: or 'Thou Shalt Not Kill',1914-08-02,omdb,6.4,6.4,1549.0,1549.0,Not Rated,omdb,...,English,en,omdb,US,United States,US,omdb,NaN,<NA>,NaN
2,tt0004013,The Ghost Breaker,1914-12-07,tmdb,4.7,4.7,52.0,52.0,NaN,<NA>,...,"None, English",NaN,<NA>,NaN,United States,US,omdb,NaN,<NA>,NaN
3,tt0004026,The Golem,1915-01-15,omdb,6.7,6.7,1305.0,1305.0,NaN,<NA>,...,"None, German",NaN,<NA>,US,Germany,DE,omdb,NaN,<NA>,NaN
4,tt0004121,The Hound of the Baskervilles (1914),1914-06-12,omdb,5.6,5.6,183.0,183.0,NaN,<NA>,...,German,de,omdb,AT,Germany,DE,omdb,NaN,<NA>,NaN
5,tt0005231,The Hound of the Baskervilles,1915-08-01,<NA>,3.3,3.3,50.0,50.0,NaN,<NA>,...,NaN,NaN,<NA>,NaN,Germany,DE,omdb,NaN,<NA>,NaN
6,tt0005513,"Der Hund von Baskerville, 3. Teil - Das unheim...",1915-01-01,omdb,4.8,4.8,44.0,44.0,NaN,<NA>,...,NaN,NaN,<NA>,NaN,Germany,DE,omdb,NaN,<NA>,NaN
7,tt0005514,"Der Hund von Baskerville, 4. Teil",1915-05-01,omdb,4.1,4.1,47.0,47.0,NaN,<NA>,...,NaN,NaN,<NA>,NaN,Germany,DE,omdb,NaN,<NA>,NaN
8,tt0005529,Blind Justice,1916-09-25,tmdb,6.7,6.7,461.0,461.0,NaN,<NA>,...,NaN,NaN,<NA>,US,Denmark,DK,omdb,NaN,<NA>,NaN
9,tt0005615,Life Without Soul,1915-11-01,tmdb,6.4,6.4,67.0,67.0,NaN,<NA>,...,"None, English",NaN,<NA>,NaN,United States,US,omdb,NaN,<NA>,NaN


Merging in box office data from the numbers:

In [ ]:
# ============================
# Cell 7: Load and clean The Numbers dataset
# ============================

import pandas as pd, numpy as np, re

numbers = pd.read_csv("the_numbers_horror_raw.csv", low_memory=False)
print(f"Loaded {len(numbers):,} rows from The Numbers")


# Normalize column names (collapse multiple spaces, keep case)
numbers.columns = numbers.columns.str.replace(r"\s+", " ", regex=True).str.strip()

# Map possible variants to expected names
col_map = {
    "Domestic Box Office": "DomesticBox Office",
    "International Box Office": "InternationalBox Office",
    "Worldwide Box Office": "WorldwideBox Office",
}
for k, v in col_map.items():
    if k in numbers.columns and v not in numbers.columns:
        numbers[v] = numbers[k]

# --- Standardize title + year ---
def clean_title(t):
    return re.sub(r'[^a-z0-9]+', '', str(t).lower())

numbers["title_std"] = numbers["Movie"].apply(clean_title)

# Extract year (best-effort)
numbers["year"] = (
    numbers["Released"]
    .astype(str)
    .str.extract(r'(\d{4})')[0]
    .astype("Int64")
)

# --- Clean revenue columns ---
for col in ["DomesticBox Office","InternationalBox Office","WorldwideBox Office"]:
    if col in numbers.columns:
        numbers[col] = (
            numbers[col]
            .astype(str)
            .str.replace(r"[\$,]", "", regex=True)
            .replace({"nan": np.nan, "": np.nan})
            .astype(float)
        )


# --- Build unified revenue_total_numbers (robust two-column assign) ---
def best_revenue(row):
    if pd.notna(row.get("WorldwideBox Office")):
        return row["WorldwideBox Office"], "numbers_worldwide"
    elif pd.notna(row.get("DomesticBox Office")):
        return row["DomesticBox Office"], "numbers_domestic"
    else:
        return np.nan, np.nan

res = numbers.apply(
    lambda r: pd.Series(best_revenue(r), index=["revenue_total_numbers","revenue_total_source_numbers"]),
    axis=1
)
numbers = pd.concat([numbers, res], axis=1)

print("✅ Cleaned The Numbers revenue columns")
numbers.head(3)


Loaded 2,117 rows from The Numbers
✅ Cleaned The Numbers revenue columns


,Rank,Released,Movie,InternationalBox Office,DomesticBox Office,WorldwideBox Office,MovieURL,title_std,year,revenue_total_numbers,revenue_total_source_numbers
0,1,2017,It,375300000.0,328828874.0,704128874.0,https://www.the-numbers.com/movie/It-(2017)#ta...,it,2017,704128874.0,numbers_worldwide
1,2,2025,The Conjuring: Last Rites,300600000.0,172442000.0,473042000.0,https://www.the-numbers.com/movie/Conjuring-Th...,theconjuringlastrites,2025,473042000.0,numbers_worldwide
2,3,2016,Resident Evil: The Final Chapter,287256498.0,26844692.0,314101190.0,https://www.the-numbers.com/movie/Resident-Evi...,residentevilthefinalchapter,2016,314101190.0,numbers_worldwide


In [ ]:
# ============================
# Cell 8: Merge The Numbers into v1 master and coalesce revenue
# ============================

master = pd.read_csv("horror_data_master_v1.csv", low_memory=False)
print(f"Loaded master v1: {len(master):,} rows")

# --- Build matching keys in master ---
master["title_std"] = master["primaryTitle"].astype(str).str.lower().str.replace(r"[^a-z0-9]+", "", regex=True)
master["year"] = master["startYear"].astype("Int64")

# --- Merge ---
merged = master.merge(
    numbers[["title_std","year","revenue_total_numbers","revenue_total_source_numbers"]],
    on=["title_std","year"],
    how="left"
)

# --- Coalesce revenue_total_usd ---
merged["revenue_total_usd"] = merged["revenue_total_numbers"].combine_first(merged.get("revenue_domestic"))
merged["revenue_total_source"] = np.where(
    merged["revenue_total_numbers"].notna(), "numbers",
    np.where(merged.get("revenue_domestic").notna(), "omdb", pd.NA)
)

print("✅ Merged The Numbers into master")
print(merged[["primaryTitle","year","revenue_total_usd","revenue_total_source"]].head(10))


Loaded master v1: 44,741 rows
✅ Merged The Numbers into master
                                        primaryTitle  year  revenue_total_usd  \
0                              The Student of Prague  1913                NaN   
1  The Avenging Conscience: or 'Thou Shalt Not Kill'  1914                NaN   
2                                  The Ghost Breaker  1914                NaN   
3                                          The Golem  1914                NaN   
4               The Hound of the Baskervilles (1914)  1914                NaN   
5                      The Hound of the Baskervilles  1915                NaN   
6  Der Hund von Baskerville, 3. Teil - Das unheim...  1916                NaN   
7                  Der Hund von Baskerville, 4. Teil  1916                NaN   
8                                      Blind Justice  1916                NaN   
9                                  Life Without Soul  1915                NaN   

  revenue_total_source  
0                 <N

In [ ]:
# ============================
# Cell 9: Save v2 master and summary
# ============================

OUTPUT_FILE_V2 = "horror_data_master_v2.csv.gz"
merged.to_csv(OUTPUT_FILE_V2, index=False, compression="gzip")
print(f"✅ Saved: {OUTPUT_FILE_V2}")

# --- Quick coverage summary ---
total = len(merged)
have_rev = merged["revenue_total_usd"].notna().sum()
pct = have_rev / total * 100
print(f"💰 Movies with total revenue: {have_rev:,} / {total:,} ({pct:.1f}%)")

merged[["primaryTitle","year","revenue_total_usd","revenue_total_source"]].head(10)


✅ Saved: horror_data_master_v2.csv.gz
💰 Movies with total revenue: 2,283 / 44,742 (5.1%)


,primaryTitle,year,revenue_total_usd,revenue_total_source
0,The Student of Prague,1913,NaN,<NA>
1,The Avenging Conscience: or 'Thou Shalt Not Kill',1914,NaN,<NA>
2,The Ghost Breaker,1914,NaN,<NA>
3,The Golem,1914,NaN,<NA>
4,The Hound of the Baskervilles (1914),1914,NaN,<NA>
5,The Hound of the Baskervilles,1915,NaN,<NA>
6,"Der Hund von Baskerville, 3. Teil - Das unheim...",1916,NaN,<NA>
7,"Der Hund von Baskerville, 4. Teil",1916,NaN,<NA>
8,Blind Justice,1916,NaN,<NA>
9,Life Without Soul,1915,NaN,<NA>


Adding in kaggle box office numbers

In [ ]:
# ============================
# Cell 10: Load & clean Kaggle revenue; prep keys
# ============================
import pandas as pd, numpy as np, re, glob

# Load Kaggle (handles the odd filename)
k_candidates = sorted(glob.glob("enhanced_box_office_data*.csv"))
if not k_candidates:
    raise FileNotFoundError("Kaggle CSV not found (enhanced_box_office_data*.csv).")
k_path = k_candidates[-1]
kag = pd.read_csv(k_path, low_memory=False)
print(f"Loaded Kaggle: {len(kag):,} rows from {k_path}")

# Normalize column names just in case
kag.columns = kag.columns.str.strip()

# Money cleaner
def money_to_float(s):
    return (pd.Series(s, dtype="object")
              .astype(str)
              .str.replace(r"[^\d.]", "", regex=True)
              .replace({"": np.nan, "nan": np.nan})
              .astype(float))

# Build standardized title + year
def clean_title(t):
    return re.sub(r'[^a-z0-9]+', '', str(t).lower())

# Try to find the title/year columns in Kaggle
title_col = "Release Group" if "Release Group" in kag.columns else ("Title" if "Title" in kag.columns else None)
year_col  = "Year" if "Year" in kag.columns else None
if title_col is None or year_col is None:
    raise ValueError("Could not find Kaggle title/year columns (expected 'Release Group' and 'Year').")

kag["title_std"] = kag[title_col].apply(clean_title)
kag["year"] = pd.to_numeric(kag[year_col], errors="coerce").astype("Int64")

# Clean money columns
ww_col = "$Worldwide" if "$Worldwide" in kag.columns else None
dom_col = "$Domestic" if "$Domestic" in kag.columns else None
for_col = "$Foreign"   if "$Foreign"   in kag.columns else None

if ww_col:   kag["kaggle_worldwide"]   = money_to_float(kag[ww_col])
if dom_col:  kag["kaggle_domestic"]    = money_to_float(kag[dom_col])
if for_col:  kag["kaggle_foreign"]     = money_to_float(kag[for_col])

# A single "best available" total revenue per Kaggle row
def best_kaggle_rev(row):
    if ww_col and pd.notna(row.get("kaggle_worldwide")):
        return row["kaggle_worldwide"], "kaggle_worldwide"
    # fallback: sum domestic + foreign if both exist
    if dom_col and for_col and pd.notna(row.get("kaggle_domestic")) and pd.notna(row.get("kaggle_foreign")):
        return row["kaggle_domestic"] + row["kaggle_foreign"], "kaggle_domestic_plus_foreign"
    # fallback: domestic only
    if dom_col and pd.notna(row.get("kaggle_domestic")):
        return row["kaggle_domestic"], "kaggle_domestic"
    return np.nan, np.nan

res = kag.apply(lambda r: pd.Series(best_kaggle_rev(r), index=["revenue_total_kaggle","revenue_total_source_kaggle"]), axis=1)
kag = pd.concat([kag, res], axis=1)

# Build a ±1 year tolerance table to improve matches
k0 = kag[["title_std","year","revenue_total_kaggle","revenue_total_source_kaggle"]].dropna(subset=["title_std"])
k_minus = k0.copy(); k_minus["year"] = (k_minus["year"] - 1).astype("Int64")
k_plus  = k0.copy(); k_plus["year"]  = (k_plus["year"]  + 1).astype("Int64")
k_tolerant = pd.concat([k0, k_minus, k_plus], ignore_index=True).drop_duplicates(subset=["title_std","year"])

print("✅ Kaggle cleaned and tolerant key table built")
k_tolerant.head(3)

Loaded Kaggle: 5,000 rows from enhanced_box_office_data(2000-2024)u (1).csv
✅ Kaggle cleaned and tolerant key table built


,title_std,year,revenue_total_kaggle,revenue_total_source_kaggle
0,missionimpossibleii,2000,546388108.0,kaggle_worldwide
1,gladiator,2000,460583960.0,kaggle_worldwide
2,castaway,2000,429632142.0,kaggle_worldwide


In [ ]:
# ============================
# Cell 11 (fixed): Merge Kaggle into master, coalesce revenue_total_usd
# ============================
import glob, re
import pandas as pd
import numpy as np

# Load latest master (v2 if present, else v1)
cands = sorted(glob.glob("horror_data_master_v2.csv.gz")) or sorted(glob.glob("horror_data_master_v1*.csv*"))
if not cands:
    raise FileNotFoundError("Could not find a master file (v1/v2).")
master_path = cands[-1]
comp = "gzip" if master_path.endswith(".gz") else None
m = pd.read_csv(master_path, compression=comp, low_memory=False)
print(f"Loaded master: {master_path}  ({len(m):,} rows)")

# --- Build clean title keys in master (primary + original) ---
def clean_title(t):
    return re.sub(r'[^a-z0-9]+', '', str(t).lower())

m["title_std_primary"]  = m["primaryTitle"].apply(clean_title)  if "primaryTitle"  in m.columns else pd.NA
m["title_std_original"] = m["originalTitle"].apply(clean_title) if "originalTitle" in m.columns else pd.NA
m["year"] = pd.to_numeric(m.get("startYear"), errors="coerce").astype("Int64")

# --- Prepare a minimal Kaggle key table to avoid column collisions ---
k_sel = k_tolerant[["title_std","year","revenue_total_kaggle","revenue_total_source_kaggle"]].drop_duplicates()

# First pass: match on PRIMARY title
k_primary = k_sel.rename(columns={"title_std": "k_title_std_primary"})
m1 = m.merge(
    k_primary,
    left_on=["title_std_primary", "year"],
    right_on=["k_title_std_primary", "year"],
    how="left"
)

# Second pass: rows still missing Kaggle → try ORIGINAL title
need = m1["revenue_total_kaggle"].isna()
k_orig = k_sel.rename(columns={"title_std": "k_title_std_original"})
m2 = m1[need].merge(
    k_orig,
    left_on=["title_std_original", "year"],
    right_on=["k_title_std_original", "year"],
    how="left",
    suffixes=("", "_alt")
)

# Bring alt matches back into m1
for col in ["revenue_total_kaggle", "revenue_total_source_kaggle"]:
    m1.loc[need, col] = m2[col + "_alt"].values

# Coalesce into unified revenue_total_usd:
# Preference: Numbers (already in v2) → Kaggle → OMDb BoxOffice (seed from v1)
if "revenue_total_usd" not in m1.columns:
    # If we’re coming from v1 (no Numbers), seed with OMDb domestic first
    m1["revenue_total_usd"] = m1.get("revenue_domestic")

before = m1["revenue_total_usd"].notna().sum()

m1["revenue_total_usd"] = m1["revenue_total_usd"].combine_first(m1["revenue_total_kaggle"])

# Source updates
if "revenue_total_source" not in m1.columns:
    # If we seeded from OMDb in v1, mark those as 'omdb', otherwise NA
    m1["revenue_total_source"] = np.where(m1.get("revenue_total_usd").notna() & m1.get("revenue_domestic").notna(), "omdb", pd.NA)

# Mark rows newly filled by Kaggle
new_kag = m1["revenue_total_usd"].notna() & m1["revenue_total_kaggle"].notna() & (
    (m1["revenue_total_source"].isna()) | (m1["revenue_total_source"] == "omdb")
)
m1.loc[new_kag, "revenue_total_source"] = m1.loc[new_kag, "revenue_total_source_kaggle"].fillna("kaggle")

after = m1["revenue_total_usd"].notna().sum()
print(f"💰 Filled revenue_total_usd for +{after - before:,} additional movies via Kaggle")

# Tidy temp join columns (keep master’s own title_std if present)
drop_cols = [c for c in ["k_title_std_primary","k_title_std_original","title_std_primary","title_std_original"] if c in m1.columns]
m1.drop(columns=drop_cols, inplace=True)

merged_v3 = m1
merged_v3.head(3)

Loaded master: horror_data_master_v2.csv.gz  (44,742 rows)
💰 Filled revenue_total_usd for +74 additional movies via Kaggle


,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,language_best,region_best,...,language_source,region_source,rated_source,revenue_domestic_source,revenue_total_numbers,revenue_total_source_numbers,revenue_total_usd,revenue_total_source,revenue_total_kaggle,revenue_total_source_kaggle
0,tt0003419,The Student of Prague,Der Student von Prag,1913.0,83.0,"Drama,Fantasy,Horror",6.4,2616.0,en,US,...,omdb,omdb,omdb,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0003643,The Avenging Conscience: or 'Thou Shalt Not Kill',The Avenging Conscience: or 'Thou Shalt Not Kill',1914.0,78.0,"Crime,Drama,Horror",6.4,1549.0,ja,US,...,omdb,omdb,omdb,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0004013,The Ghost Breaker,The Ghost Breaker,1914.0,60.0,"Adventure,Horror",4.7,52.0,NaN,NaN,...,NaN,omdb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# ============================
# Cell 12: (Optional) add TMDb 'revenue' if present, then save v3 + summary
# ============================

import os

# OPTIONAL TMDb revenue pass (if your TMDb CSV is present and has 'revenue' col)
tmdb_paths = sorted(glob.glob("horror_movies_tmdb_enriched*.csv*"))
if tmdb_paths:
    tmdb_path = tmdb_paths[-1]
    comp = "gzip" if tmdb_path.endswith(".gz") else None
    tdf = pd.read_csv(tmdb_path, compression=comp, low_memory=False)
    # Normalize keys
    tdf.columns = tdf.columns.str.strip()
    # Use imdb_id (tt...) if available for precise matches
    id_col = "imdb_id" if "imdb_id" in tdf.columns else None
    if id_col and "tconst" in merged_v3.columns and "revenue" in tdf.columns:
        add = tdf[[id_col, "revenue"]].copy()
        add.rename(columns={id_col: "tconst", "revenue": "tmdb_worldwide"}, inplace=True)
        add["tmdb_worldwide"] = pd.to_numeric(add["tmdb_worldwide"], errors="coerce")
        merged_v3 = merged_v3.merge(add, on="tconst", how="left")
        # Only use tmdb_worldwide when > 0 and no revenue_total yet
        fillable = merged_v3["revenue_total_usd"].isna() & merged_v3["tmdb_worldwide"].notna() & (merged_v3["tmdb_worldwide"] > 0)
        merged_v3.loc[fillable, "revenue_total_usd"] = merged_v3.loc[fillable, "tmdb_worldwide"]
        merged_v3.loc[fillable, "revenue_total_source"] = "tmdb_worldwide"
        print(f"🎬 TMDb added total revenue to {fillable.sum():,} more titles")

# Save v3
OUTPUT_FILE_V3 = "horror_data_master_v3.csv.gz"
merged_v3.to_csv(OUTPUT_FILE_V3, index=False, compression="gzip")
print(f"✅ Saved: {OUTPUT_FILE_V3}")

# Coverage summary
total = len(merged_v3)
have_total = merged_v3["revenue_total_usd"].notna().sum()
print(f"📈 revenue_total_usd coverage: {have_total:,}/{total:,} ({have_total/total:.1%})")

merged_v3[["primaryTitle","year","revenue_total_usd","revenue_total_source"]].head(10)

✅ Saved: horror_data_master_v3.csv.gz
📈 revenue_total_usd coverage: 2,357/44,742 (5.3%)


,primaryTitle,year,revenue_total_usd,revenue_total_source
0,The Student of Prague,1913,NaN,NaN
1,The Avenging Conscience: or 'Thou Shalt Not Kill',1914,NaN,NaN
2,The Ghost Breaker,1914,NaN,NaN
3,The Golem,1914,NaN,NaN
4,The Hound of the Baskervilles (1914),1914,NaN,NaN
5,The Hound of the Baskervilles,1915,NaN,NaN
6,"Der Hund von Baskerville, 3. Teil - Das unheim...",1916,NaN,NaN
7,"Der Hund von Baskerville, 4. Teil",1916,NaN,NaN
8,Blind Justice,1916,NaN,NaN
9,Life Without Soul,1915,NaN,NaN


Using box office api to try to get more box office data

In [ ]:
!pip -q install boxoffice-api

In [ ]:
!pip install -U boxoffice-api

In [ ]:
# ============================
# Cell 1: Install + setup + load master + cache
# ============================

!pip -q install boxoffice-api

import os, glob, time, re, json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from boxoffice import BoxOffice

# -------- CONFIG --------
CACHE_FILE = "bom_cache.csv"
DELAY = 0.20           # ~3-4 req/sec; adjust to 0.2 if stable, 0.5 if errors
BATCH_SAVE = 500       # save cache every N new rows
VERSION = 4
OUTPUT_FILE = f"horror_data_master_v{VERSION}.csv.gz"

# -------- Load latest master (v3 or v1/v2 fallback) --------
cands = sorted(glob.glob("horror_data_master_v*.csv*"))
if not cands:
    raise FileNotFoundError("No master file found (expected horror_data_master_v*.csv[.gz])")
SPINE = cands[-1]  # use most recent
comp = "gzip" if SPINE.endswith(".gz") else None
master = pd.read_csv(SPINE, compression=comp, low_memory=False)
print(f"📂 Using master: {SPINE}  ({len(master):,} rows)")

# -------- Build matching keys on master --------
def title_std(s):
    return re.sub(r'[^a-z0-9]+','', str(s).lower())

master["title_std"] = master.get("primaryTitle", master.get("originalTitle")).astype(str).map(title_std)
if "startYear" in master.columns:
    master["year"] = pd.to_numeric(master["startYear"], errors="coerce").astype("Int64")
else:
    # fallback from any existing final release date
    master["year"] = pd.to_datetime(master.get("release_date_final"), errors="coerce").dt.year.astype("Int64")

# -------- Load / init cache --------
if os.path.exists(CACHE_FILE):
    cache = pd.read_csv(CACHE_FILE, dtype=str)
    # enforce consistent types
    cache_map = {(r["title_std"], (pd.to_numeric(r["year"], errors="coerce") if "year" in r else pd.NA)): r.to_dict()
                 for _, r in cache.iterrows()}
    print(f"🔁 Loaded BoM cache: {len(cache_map):,} entries")
else:
    cache_map = {}
    print("🆕 Starting fresh BoM cache")

# helper to save cache safely
def save_cache_safe(cm, path=CACHE_FILE):
    rows = []
    for (tstd, yr), d in cm.items():
        d = dict(d)
        d["title_std"] = tstd
        d["year"] = yr
        rows.append(d)
    pd.DataFrame.from_records(rows).to_csv(path, index=False)
    print(f"💾 Cache saved: {len(rows):,} rows")

bo = BoxOffice()
print("✅ Setup ready")

ModuleNotFoundError: No module named 'boxoffice'

In [ ]:
# ============================
# Cell 2: Fetch Box Office Mojo data (resume-safe)
# ============================

# limit fetch to titles missing a total revenue (speeds things up)
# if you want ALL, set mask = master["title_std"].notna()
if "revenue_total_usd" in master.columns:
    mask = master["revenue_total_usd"].isna()
else:
    mask = master["title_std"].notna()

todo_df = master.loc[mask, ["title_std","year","primaryTitle","originalTitle"]].drop_duplicates()
print(f"🎬 Titles to query: {len(todo_df):,}")

new_count = 0
for _, row in tqdm(todo_df.iterrows(), total=len(todo_df), desc="Fetching BoM", unit="title"):
    key = (row["title_std"], row["year"])
    if key in cache_map:
        continue

    # Try several query strings to improve hit rate
    queries = []
    if pd.notna(row["primaryTitle"]):
        queries.append(f"{row['primaryTitle']} {'' if pd.isna(row['year']) else int(row['year'])}")
        queries.append(f"{row['primaryTitle']}")
    if pd.notna(row["originalTitle"]) and row["originalTitle"] != row["primaryTitle"]:
        queries.append(f"{row['originalTitle']} {'' if pd.isna(row['year']) else int(row['year'])}")
        queries.append(f"{row['originalTitle']}")

    res = {}
    for q in queries:
        try:
            data = bo.search(q)
            # data can be dict or list depending on wrapper/version; normalize
            if isinstance(data, list) and data:
                cand = data[0]
            else:
                cand = data or {}
            # minimal fields we care about
            res = {
                "bom_query": q,
                "bom_title": cand.get("title"),
                "revenue_domestic_bom": cand.get("domestic"),
                "revenue_international_bom": cand.get("international"),
                "revenue_worldwide_bom": cand.get("worldwide"),
                "release_date_bom": cand.get("release_date"),
            }
            # accept if worldwide or domestic present
            if any(pd.notna(res.get(k)) for k in ["revenue_worldwide_bom","revenue_domestic_bom","revenue_international_bom"]):
                break
        except Exception as e:
            res = {"bom_error": str(e)}
            # keep trying other queries

    # store (even if empty) so we don't keep retrying this key
    cache_map[key] = res
    new_count += 1

    if new_count % BATCH_SAVE == 0:
        save_cache_safe(cache_map, CACHE_FILE)

    time.sleep(DELAY)

# final cache save
save_cache_safe(cache_map, CACHE_FILE)
print("✅ Fetch complete")

In [ ]:
# ============================
# Cell 3: Build BoM frame, clean, merge
# ============================

bom = pd.DataFrame.from_records(
    [dict(v, title_std=k[0], year=k[1]) for k, v in cache_map.items()]
)
print(f"📊 BoM rows: {len(bom):,}")

# Clean numeric grosses
def to_num(x):
    return (pd.Series(x, dtype="object")
              .astype(str)
              .str.replace(r"[^\d.]", "", regex=True)
              .replace({"": np.nan, "nan": np.nan})
              .astype(float))

for col in ["revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom"]:
    if col in bom.columns:
        bom[col] = to_num(bom[col])

# Parse date
if "release_date_bom" in bom.columns:
    bom["release_date_bom"] = pd.to_datetime(bom["release_date_bom"], errors="coerce")

# Merge
merged = master.merge(
    bom[["title_std","year","revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom","release_date_bom"]],
    on=["title_std","year"],
    how="left"
)

print("✅ Merged BoM into master")
merged.head(3)

In [ ]:
# ============================
# Cell 4: Coalesce total revenue + provenance, save v4
# ============================

# Create or update revenue_total_usd:
# Priority: Numbers worldwide -> BoM worldwide -> Numbers domestic -> OMDb domestic (already in your v1/v2)
cands = []
if "revenue_total_usd" in merged.columns:
    cands.append(merged["revenue_total_usd"])
if "revenue_total_numbers" in merged.columns:
    cands.append(merged["revenue_total_numbers"])
if "revenue_worldwide_numbers" in merged.columns:
    cands.append(merged["revenue_worldwide_numbers"])
cands.append(merged.get("revenue_worldwide_bom"))
if "revenue_domestic_numbers" in merged.columns:
    cands.append(merged["revenue_domestic_numbers"])
cands.append(merged.get("revenue_domestic_omdb"))

# Build a combined total
rev_total = pd.Series(np.nan, index=merged.index, dtype="float")
for c in cands:
    if c is not None:
        rev_total = rev_total.combine_first(c)
merged["revenue_total_usd"] = rev_total

# Provenance
src = pd.Series(pd.NA, index=merged.index, dtype="object")
# order aligned with priorities above
if "revenue_total_numbers" in merged.columns:
    src = np.where(merged["revenue_total_usd"].eq(merged["revenue_total_numbers"]), "numbers_worldwide", src)
if "revenue_worldwide_numbers" in merged.columns:
    src = np.where(merged["revenue_total_usd"].eq(merged["revenue_worldwide_numbers"]), "numbers_worldwide", src)
src = np.where(merged["revenue_total_usd"].eq(merged.get("revenue_worldwide_bom")), "bom_worldwide", src)
if "revenue_domestic_numbers" in merged.columns:
    src = np.where(merged["revenue_total_usd"].eq(merged["revenue_domestic_numbers"]), "numbers_domestic", src)
src = np.where(merged["revenue_total_usd"].eq(merged.get("revenue_domestic_omdb")), "omdb_domestic", src)

merged["revenue_total_source"] = src

# Save
merged.to_csv(OUTPUT_FILE, index=False, compression="gzip")
print(f"✅ Saved: {OUTPUT_FILE}")

# Quick coverage
total = len(merged)
have_total = merged["revenue_total_usd"].notna().sum()
print(f"💰 coverage (total revenue): {have_total:,}/{total:,} ({have_total/total:.1%})")

Second Try:

In [ ]:
# ============================
# Cell 1: Setup, load master, cache helpers
# ============================

import os, re, time, glob, math, random
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

# -------- CONFIG --------
CACHE_FILE = "bom_cache (5).csv"     # tconst-keyed cache
DELAY = 0.20                     # seconds between requests (~3 req/s). Safer: 0.3. Faster: 0.2 (moderate risk).
MAX_RETRIES = 3                  # per title
BACKOFF_MULT = 2.0               # exponential backoff on 429/5xx
BATCH_SAVE = 100                 # save cache every N new rows
VERSION = 4
OUTPUT_FILE = f"horror_data_master_v{VERSION}.csv.gz"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/124.0 Safari/537.36"
}

# -------- Load latest master --------
cands = sorted(glob.glob("horror_data_master_v*.csv*"))
if not cands:
    raise FileNotFoundError("No master file found (expected horror_data_master_v*.csv[.gz])")
SPINE = cands[-1]  # most recent
comp = "gzip" if SPINE.endswith(".gz") else None
master = pd.read_csv(SPINE, compression=comp, low_memory=False)
print(f"📂 Using master: {SPINE}  ({len(master):,} rows)")

# Sanity columns
if "tconst" not in master.columns:
    raise ValueError("Expected 'tconst' column (IMDb IDs) in master.")

# -------- Load / init cache (dict of tconst -> dict) --------
if os.path.exists(CACHE_FILE):
    cache_df = pd.read_csv(CACHE_FILE, dtype=str)
    cache_map = {}
    for _, r in cache_df.iterrows():
        d = r.to_dict()
        t = d.pop("tconst", None)
        if not t:
            continue
        cache_map[t] = d
    print(f"🔁 Loaded BoM cache: {len(cache_map):,} entries")
else:
    cache_map = {}
    print("🆕 Starting fresh BoM cache")

def save_cache_safe(cache_map, path=CACHE_FILE):
    rows = []
    for tconst, d in cache_map.items():
        dd = dict(d) if isinstance(d, dict) else {}
        dd["tconst"] = tconst
        rows.append(dd)
    pd.DataFrame.from_records(rows).to_csv(path, index=False)
    print(f"💾 Cache saved: {len(rows):,} rows")

print("✅ Setup ready")

📂 Using master: horror_data_master_v3.csv  (44,742 rows)
🔁 Loaded BoM cache: 29,700 entries
✅ Setup ready


In [ ]:
# ============================
# Cell 2: BoM fetch + parse functions
# ============================

BASE = "https://www.boxofficemojo.com/title/{imdb_id}/"

money_re = re.compile(r"\$[\d,]+(?:\.\d+)?")

def parse_money_to_float(text):
    if text is None:
        return np.nan
    m = money_re.search(str(text))
    if not m:
        return np.nan
    return float(m.group(0).replace("$","").replace(",",""))

def parse_bom_html(html):
    """Return dict with domestic, international, worldwide (floats) and release_date_bom (YYYY-MM-DD if present)."""
    soup = BeautifulSoup(html, "html.parser")
    res = {
        "revenue_domestic_bom": np.nan,
        "revenue_international_bom": np.nan,
        "revenue_worldwide_bom": np.nan,
        "release_date_bom": None
    }

    # 1) Try the performance summary block (most common)
    # labels often appear as text: "Domestic", "International", "Worldwide"
    for label, key in [("Domestic","revenue_domestic_bom"),
                       ("International","revenue_international_bom"),
                       ("Worldwide","revenue_worldwide_bom")]:
        # find any element containing the label
        el = soup.find(string=re.compile(rf"^{label}\b", flags=re.I))
        if el:
            # try the containing row/parent to locate the currency
            parent = el
            for _ in range(4):
                parent = parent.parent
                if parent is None: break
                money = money_re.search(parent.get_text(" ", strip=True))
                if money:
                    res[key] = parse_money_to_float(money.group(0))
                    break

    # 2) Try alternate cards (fallback)
    if math.isnan(res["revenue_worldwide_bom"]):
        # Sometimes the Worldwide is in a div with "mojo-performance-summary-table"
        tbl = soup.find("div", class_=re.compile("mojo-performance-summary-table"))
        if tbl:
            txt = tbl.get_text(" ", strip=True)
            m = re.search(r"Worldwide\s+\$[\d,]+(?:\.\d+)?", txt)
            if m:
                res["revenue_worldwide_bom"] = parse_money_to_float(m.group(0))

    # 3) Try to capture a U.S. release date (not always present)
    # Look for 'Release Date' label
    rd = soup.find(string=re.compile(r"Release Date", flags=re.I))
    if rd:
        # The date often appears nearby; grab the first pattern YYYY-MM-DD or Month DD, YYYY
        parent = rd
        for _ in range(4):
            parent = parent.parent
            if parent is None: break
            txt = parent.get_text(" ", strip=True)
            # Try ISO first
            m_iso = re.search(r"\b\d{4}-\d{2}-\d{2}\b", txt)
            if m_iso:
                res["release_date_bom"] = m_iso.group(0)
                break
            # Try "Month DD, YYYY"
            m_long = re.search(r"\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2}, \d{4}\b", txt, flags=re.I)
            if m_long:
                # leave as string; we can parse to datetime later if we want
                res["release_date_bom"] = m_long.group(0)
                break

    return res

def fetch_bom_by_tconst(imdb_id, delay=DELAY):
    """Fetch one BoM film page by IMDb ID and parse grosses with retries/backoff."""
    url = BASE.format(imdb_id=imdb_id)
    wait = delay
    for attempt in range(1, MAX_RETRIES+1):
        try:
            r = requests.get(url, headers=HEADERS, timeout=15)
            if r.status_code == 404:
                # no film page
                return {"status": 404}
            if r.status_code in (429, 500, 502, 503, 504):
                # backoff and retry
                time.sleep(wait)
                wait *= BACKOFF_MULT
                continue
            if r.status_code != 200:
                return {"status": r.status_code}

            parsed = parse_bom_html(r.text)
            parsed["status"] = 200
            return parsed
        except requests.RequestException:
            time.sleep(wait)
            wait *= BACKOFF_MULT

    return {"status": "error"}

print("✅ Scraper ready")

✅ Scraper ready


In [ ]:
# ============================
# Cell 3: Build TODO and fetch BoM with progress bar
# ============================

# We'll only query titles that still lack a total revenue, if that column exists.
if "revenue_total_usd" in master.columns:
    need_mask = master["revenue_total_usd"].isna()
else:
    need_mask = master["tconst"].notna()

todo = master.loc[need_mask, ["tconst","primaryTitle","startYear"]].dropna(subset=["tconst"]).drop_duplicates()
print(f"🎬 Titles to query (by tconst): {len(todo):,}")

new_since_save = 0
for _, row in tqdm(todo.iterrows(), total=len(todo), desc="Fetching BoM by IMDb ID", unit="title"):
    tc = row["tconst"]
    if tc in cache_map and any(k in cache_map[tc] for k in ["revenue_worldwide_bom","revenue_domestic_bom","revenue_international_bom","status"]):
        continue

    res = fetch_bom_by_tconst(tc, delay=DELAY)
    cache_map[tc] = res
    new_since_save += 1

    if new_since_save >= BATCH_SAVE:
        save_cache_safe(cache_map, CACHE_FILE)
        new_since_save = 0

    # polite jitter around the base delay
    time.sleep(DELAY + random.uniform(0, 0.05))

# Final cache save
save_cache_safe(cache_map, CACHE_FILE)
print("✅ Fetch complete")

🎬 Titles to query (by tconst): 42,385


Fetching BoM by IMDb ID:   0%|          | 0/42385 [00:00<?, ?title/s]

💾 Cache saved: 30,678 rows
💾 Cache saved: 30,778 rows
💾 Cache saved: 30,878 rows
💾 Cache saved: 30,978 rows
💾 Cache saved: 31,078 rows
💾 Cache saved: 31,178 rows
💾 Cache saved: 31,278 rows
💾 Cache saved: 31,378 rows
💾 Cache saved: 31,478 rows
💾 Cache saved: 31,578 rows
💾 Cache saved: 31,678 rows
💾 Cache saved: 31,778 rows
💾 Cache saved: 31,878 rows
💾 Cache saved: 31,978 rows
💾 Cache saved: 32,078 rows
💾 Cache saved: 32,178 rows
💾 Cache saved: 32,278 rows
💾 Cache saved: 32,378 rows
💾 Cache saved: 32,478 rows
💾 Cache saved: 32,578 rows
💾 Cache saved: 32,678 rows
💾 Cache saved: 32,778 rows
💾 Cache saved: 32,878 rows
💾 Cache saved: 32,978 rows
💾 Cache saved: 33,078 rows
💾 Cache saved: 33,178 rows
💾 Cache saved: 33,278 rows
💾 Cache saved: 33,378 rows
💾 Cache saved: 33,478 rows
💾 Cache saved: 33,578 rows
💾 Cache saved: 33,678 rows
💾 Cache saved: 33,778 rows
💾 Cache saved: 33,878 rows
💾 Cache saved: 33,978 rows
💾 Cache saved: 34,078 rows
💾 Cache saved: 34,178 rows
💾 Cache saved: 34,278 rows
💾

In [ ]:
!ls -lh /content | grep bom_cache

-rw-r--r-- 1 root root 593K Oct 18 07:32 bom_cache (5).csv


In [ ]:
# ============================
# Cell 4: Build BoM DF, clean, merge
# ============================

# Build DataFrame from cache
records = []
for tconst, d in cache_map.items():
    dd = dict(d) if isinstance(d, dict) else {}
    dd["tconst"] = tconst
    records.append(dd)
bom = pd.DataFrame.from_records(records)
print(f"📊 BoM cached rows: {len(bom):,}")

# Ensure columns exist
for col in ["revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom"]:
    if col not in bom.columns: bom[col] = np.nan

# Clean numerics (if any strings slipped through)
def to_num(s):
    return (pd.Series(s, dtype="object")
              .astype(str)
              .str.replace(r"[^\d.]", "", regex=True)
              .replace({"": np.nan, "nan": np.nan})
              .astype(float))

for col in ["revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom"]:
    bom[col] = to_num(bom[col])

# Parse release date if present
if "release_date_bom" in bom.columns:
    # Try ISO first; pandas can also parse Month DD, YYYY
    bom["release_date_bom"] = pd.to_datetime(bom["release_date_bom"], errors="coerce")

# Merge on tconst
merged = master.merge(
    bom[["tconst","revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom","release_date_bom"]],
    on="tconst", how="left"
)
print("✅ Merged BoM into master")

merged.head(3)

📊 BoM cached rows: 42,385


/tmp/ipython-input-3918270512.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"": np.nan, "nan": np.nan})
/tmp/ipython-input-3918270512.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"": np.nan, "nan": np.nan})


✅ Merged BoM into master


,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,language_best,region_best,...,revenue_total_numbers,revenue_total_source_numbers,revenue_total_usd,revenue_total_source,revenue_total_kaggle,revenue_total_source_kaggle,revenue_domestic_bom,revenue_international_bom,revenue_worldwide_bom,release_date_bom
0,tt0003419,The Student of Prague,Der Student von Prag,1913.0,83.0,"Drama,Fantasy,Horror",6.4,2616.0,en,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,tt0003643,The Avenging Conscience: or 'Thou Shalt Not Kill',The Avenging Conscience: or 'Thou Shalt Not Kill',1914.0,78.0,"Crime,Drama,Horror",6.4,1549.0,ja,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,tt0004013,The Ghost Breaker,The Ghost Breaker,1914.0,60.0,"Adventure,Horror",4.7,52.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [ ]:
# ============================
# Cell 5: Coalesce revenue_total_usd + provenance, save v4
# ============================

# Build candidate stack (highest priority first)
cands = []
labels = []

# If you already created 'revenue_total_numbers' earlier (The Numbers worldwide)
if "revenue_total_numbers" in merged.columns:
    cands.append(merged["revenue_total_numbers"]); labels.append("numbers_worldwide")

# BoM worldwide (new)
cands.append(merged["revenue_worldwide_bom"]); labels.append("bom_worldwide")

# If only domestic exists from Numbers, consider as total fallback
if "revenue_domestic_numbers" in merged.columns:
    cands.append(merged["revenue_domestic_numbers"]); labels.append("numbers_domestic")

# OMDb domestic as last resort
if "revenue_domestic_omdb" in merged.columns:
    cands.append(merged["revenue_domestic_omdb"]); labels.append("omdb_domestic")

# Start with existing total if present to avoid regressions
if "revenue_total_usd" in merged.columns:
    base_total = merged["revenue_total_usd"].copy()
else:
    base_total = pd.Series(np.nan, index=merged.index, dtype="float")

# Coalesce
rev_total = base_total.copy()
src = pd.Series(pd.NA, index=merged.index, dtype="object")

# If base already had values, mark source as 'existing'
src = np.where(rev_total.notna(), "existing", src)

for s, lab in zip(cands, labels):
    mask = rev_total.isna() & s.notna()
    rev_total[mask] = s[mask]
    src = np.where(mask, lab, src)

merged["revenue_total_usd"] = rev_total
merged["revenue_total_source"] = src

# Optional: use BoM release date if final date missing
if "release_date_final" in merged.columns and "release_date_bom" in merged.columns:
    need_date = merged["release_date_final"].isna() & merged["release_date_bom"].notna()
    merged.loc[need_date, "release_date_final"] = merged.loc[need_date, "release_date_bom"]
    if "release_date_source" not in merged.columns:
        merged["release_date_source"] = pd.NA
    merged.loc[need_date, "release_date_source"] = "bom"

# Save
merged.to_csv(OUTPUT_FILE, index=False, compression="gzip")
print(f"✅ Saved: {OUTPUT_FILE}")

# Coverage summary
total = len(merged)
have_total = merged["revenue_total_usd"].notna().sum()
print(f"💰 Coverage (total revenue): {have_total:,}/{total:,} ({have_total/total:.1%})")

merged[["tconst","primaryTitle","startYear","revenue_worldwide_bom","revenue_total_usd","revenue_total_source"]].head(10)

✅ Saved: horror_data_master_v4.csv.gz
💰 Coverage (total revenue): 4,575/44,742 (10.2%)


,tconst,primaryTitle,startYear,revenue_worldwide_bom,revenue_total_usd,revenue_total_source
0,tt0003419,The Student of Prague,1913.0,NaN,NaN,<NA>
1,tt0003643,The Avenging Conscience: or 'Thou Shalt Not Kill',1914.0,NaN,NaN,<NA>
2,tt0004013,The Ghost Breaker,1914.0,NaN,NaN,<NA>
3,tt0004026,The Golem,1914.0,NaN,NaN,<NA>
4,tt0004121,The Hound of the Baskervilles (1914),1914.0,NaN,NaN,<NA>
5,tt0005231,The Hound of the Baskervilles,1915.0,NaN,NaN,<NA>
6,tt0005513,"Der Hund von Baskerville, 3. Teil - Das unheim...",1916.0,NaN,NaN,<NA>
7,tt0005514,"Der Hund von Baskerville, 4. Teil",1916.0,NaN,NaN,<NA>
8,tt0005529,Blind Justice,1916.0,NaN,NaN,<NA>
9,tt0005615,Life Without Soul,1915.0,NaN,NaN,<NA>


In [ ]:
!ls -lh /content | grep horror_data_master_v4.csv.gz

-rw-r--r-- 1 root root 8.3M Oct 18 09:41 horror_data_master_v4.csv.gz


Sanity check

In [ ]:
import pandas as pd, numpy as np, re, glob

# pick your cache file
cache_path = sorted(glob.glob("bom_cache*.csv"))[-1]
bom = pd.read_csv(cache_path, dtype=str)
for c in ["revenue_worldwide_bom","revenue_domestic_bom","revenue_international_bom"]:
    if c in bom.columns:
        bom[c] = (bom[c].astype(str)
                        .str.replace(r"[^\d.]", "", regex=True)
                        .replace({"": np.nan, "nan": np.nan}).astype(float))

print("Rows in cache:", len(bom))
print(bom["status"].value_counts(dropna=False).sort_index())

has_any = bom[["revenue_worldwide_bom","revenue_domestic_bom","revenue_international_bom"]].notna().any(axis=1).sum()
print("Rows with any gross value:", has_any, f"({has_any/len(bom):.1%})")

print("\nSample of hits:")
print(bom[bom[["revenue_worldwide_bom","revenue_domestic_bom","revenue_international_bom"]].notna().any(axis=1)].head(10))

Rows in cache: 42385
status
200    42375
404       10
Name: count, dtype: int64
Rows with any gross value: 2218 (5.2%)

Sample of hits:
     revenue_domestic_bom  revenue_international_bom  revenue_worldwide_bom  \
75                    NaN                        NaN                48892.0   
103                   NaN                        NaN                 4841.0   
108                   NaN                        NaN                 4347.0   
122                   NaN                        NaN                89078.0   
136                   NaN                        NaN                16615.0   
137                   NaN                        NaN                 4693.0   
150                   NaN                        NaN                   19.0   
177                   NaN                        NaN                12887.0   
253                   NaN                        NaN                  420.0   
322                   NaN                        NaN                35275.

/tmp/ipython-input-384738521.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"": np.nan, "nan": np.nan}).astype(float))
/tmp/ipython-input-384738521.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"": np.nan, "nan": np.nan}).astype(float))


Trying to add tmdb box office data

In [ ]:
# ===========================================
# Merge BoM + TMDb (budget/revenue) + The Numbers into master v4
# Output: horror_data_master_v5.csv.gz
# ===========================================

import pandas as pd, numpy as np, re, glob, os

# ----------- helper cleaners -----------
def to_money_num(s):
    return (pd.Series(s, dtype="object")
             .astype(str)
             .str.replace(r"[^\d.]", "", regex=True)
             .replace({"": np.nan, "nan": np.nan})
             .astype(float))

def title_std(s):
    return re.sub(r"[^a-z0-9]+", "", str(s).lower())

# ----------- files (explicit + safe fallback) -----------
MASTER_FILE = "horror_data_master_v4.csv.gz"

TMDB_FILE_PREF = "horror_movies_tmdb_enriched - horror_movies_tmdb_enriched.csv.csv"
if not os.path.exists(TMDB_FILE_PREF):
    cand = sorted(glob.glob("horror_movies_tmdb_enriched*.csv*"))
    if not cand:
        raise FileNotFoundError("TMDb file not found. Put the enriched TMDb CSV in the workspace.")
    TMDB_FILE_PREF = cand[-1]

NUMBERS_FILE = "the_numbers_horror_raw.csv"

BOM_CACHE_FILE = "bom_cache (5).csv"
if not os.path.exists(BOM_CACHE_FILE):
    cand = sorted(glob.glob("bom_cache*.csv"))
    if not cand:
        raise FileNotFoundError("BoM cache not found. Upload/restore your bom_cache CSV.")
    BOM_CACHE_FILE = cand[-1]

print("Using:")
print("  master :", MASTER_FILE)
print("  tmdb   :", TMDB_FILE_PREF)
print("  numbers:", NUMBERS_FILE)
print("  bom    :", BOM_CACHE_FILE)

# ----------- load master -----------
master = pd.read_csv(MASTER_FILE, compression="gzip", low_memory=False)
print(f"Loaded master: {len(master):,} rows")

# Build standard keys (if not present)
if "year" not in master.columns:
    master["year"] = pd.to_numeric(master.get("startYear"), errors="coerce").astype("Int64")
if "title_std" not in master.columns:
    base_title = master.get("primaryTitle", master.get("originalTitle"))
    master["title_std"] = base_title.astype(str).map(title_std)

# ----------- TMDb: bring budget + revenue -----------
tmdb = pd.read_csv(TMDB_FILE_PREF, low_memory=False)
tmdb_cols = [c.lower() for c in tmdb.columns]
tmdb.columns = tmdb_cols

# normalize imdb_id
if "imdb_id" in tmdb.columns:
    tmdb["imdb_id"] = tmdb["imdb_id"].astype(str).str.strip()
else:
    tmdb["imdb_id"] = np.nan

# numeric budget/revenue
for c in ["budget","revenue"]:
    if c in tmdb.columns:
        tmdb[c] = to_money_num(tmdb[c])
    else:
        tmdb[c] = np.nan

tmdb_fin = tmdb[["imdb_id","budget","revenue"]].drop_duplicates()

# merge by tconst ↔ imdb_id
m1 = master.merge(tmdb_fin, left_on="tconst", right_on="imdb_id", how="left", suffixes=("","_tmdb"))
m1.rename(columns={"budget":"budget_tmdb","revenue":"revenue_tmdb"}, inplace=True)

# ----------- The Numbers: parse totals -----------
numbers = pd.read_csv(NUMBERS_FILE, low_memory=False)
# normalize headers (some exports vary with spaces)
numbers.columns = numbers.columns.str.replace(r"\s+", " ", regex=True).str.strip()

# unify field names
def pick(colnames):
    for k in colnames:
        if k in numbers.columns:
            return k
    return None

col_movie   = pick(["Movie","Title"])
col_released= pick(["Released","Release Date"])
col_dom     = pick(["DomesticBox Office","Domestic Box Office"])
col_intl    = pick(["InternationalBox Office","International Box Office"])
col_ww      = pick(["WorldwideBox Office","Worldwide Box Office"])

# keep subset
keep = [c for c in [col_movie, col_released, col_dom, col_intl, col_ww] if c]
num = numbers[keep].copy()

# create std title + year
num["title_std"] = num[col_movie].astype(str).map(title_std)
num["year"] = pd.to_datetime(num[col_released], errors="coerce").dt.year.astype("Int64")

# numeric monies
for c in [col_dom, col_intl, col_ww]:
    if c:
        num[c] = to_money_num(num[c])

# choose a single total from numbers (prefer worldwide, else domestic, else intl)
num["revenue_total_numbers"] = np.nan
src_num = pd.Series(pd.NA, index=num.index, dtype="object")
if col_ww:
    mask = num[col_ww].notna()
    num.loc[mask, "revenue_total_numbers"] = num.loc[mask, col_ww]
    src_num = np.where(mask, "numbers_worldwide", src_num)
if col_dom:
    mask = num["revenue_total_numbers"].isna() & num[col_dom].notna()
    num.loc[mask, "revenue_total_numbers"] = num.loc[mask, col_dom]
    src_num = np.where(mask, "numbers_domestic", src_num)
if col_intl:
    mask = num["revenue_total_numbers"].isna() & num[col_intl].notna()
    num.loc[mask, "revenue_total_numbers"] = num.loc[mask, col_intl]
    src_num = np.where(mask, "numbers_international", src_num)
num["revenue_total_numbers_source"] = src_num

# merge The Numbers on title_std + year
m2 = m1.merge(
    num[["title_std","year","revenue_total_numbers","revenue_total_numbers_source"]],
    on=["title_std","year"], how="left"
)

# ----------- Box Office Mojo: merge cache ----------
bom = pd.read_csv(BOM_CACHE_FILE, dtype=str)
for c in ["revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom"]:
    if c in bom.columns:
        bom[c] = to_money_num(bom[c])
    else:
        bom[c] = np.nan
# (tconst is key)
bom_fin = bom[["tconst","revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom"]]
m3 = m2.merge(bom_fin, on="tconst", how="left")

# ----------- Coalesce totals + budget + provenance -----------
cands_rev = []
labels_rev = []

# If you previously created a total, preserve it as first priority (no regression)
if "revenue_total_usd" in m3.columns:
    base_total = m3["revenue_total_usd"].copy()
else:
    base_total = pd.Series(np.nan, index=m3.index, dtype="float")

# New candidates (highest priority first)
if "revenue_total_numbers" in m3.columns:
    cands_rev.append(m3["revenue_total_numbers"]); labels_rev.append("numbers_total")
cands_rev.append(m3["revenue_worldwide_bom"]); labels_rev.append("bom_worldwide")
cands_rev.append(m3["revenue_tmdb"]); labels_rev.append("tmdb_revenue")
# fallbacks
if "revenue_domestic_numbers" in m3.columns:
    cands_rev.append(m3["revenue_domestic_numbers"]); labels_rev.append("numbers_domestic")
if "revenue_domestic_omdb" in m3.columns:
    cands_rev.append(m3["revenue_domestic_omdb"]); labels_rev.append("omdb_domestic")

rev_total = base_total.copy()
src = pd.Series(pd.NA, index=m3.index, dtype="object")
# mark existing
src = np.where(rev_total.notna(), "existing", src)

for s, lab in zip(cands_rev, labels_rev):
    mask = rev_total.isna() & s.notna()
    rev_total[mask] = s[mask]
    src = np.where(mask, lab, src)

m3["revenue_total_usd"] = rev_total
m3["revenue_total_source"] = src

# Budget from TMDb
budget = m3.get("budget_tmdb")
m3["budget_usd"] = budget
m3["budget_source"] = np.where(budget.notna(), "tmdb", pd.NA)

# ----------- Save & report -----------
OUT = "horror_data_master_v5.csv.gz"
m3.to_csv(OUT, index=False, compression="gzip")
print(f"✅ Saved: {OUT}")

total = len(m3)
have_rev = m3["revenue_total_usd"].notna().sum()
have_bud = m3["budget_usd"].notna().sum()
print(f"💰 revenue_total_usd coverage: {have_rev:,}/{total:,} ({have_rev/total:.1%})")
print(f"💵 budget_usd coverage:        {have_bud:,}/{total:,} ({have_bud/total:.1%})")

# peek
cols_show = ["primaryTitle","startYear","revenue_total_usd","revenue_total_source",
             "revenue_worldwide_bom","revenue_total_numbers","revenue_tmdb",
             "budget_usd","budget_source"]
print("\nSample:")
print(m3[cols_show].head(10))

Using:
  master : horror_data_master_v4.csv.gz
  tmdb   : horror_movies_tmdb_enriched - horror_movies_tmdb_enriched.csv.csv
  numbers: the_numbers_horror_raw.csv
  bom    : bom_cache (5).csv
Loaded master: 44,742 rows


/tmp/ipython-input-756121233.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"": np.nan, "nan": np.nan})
/tmp/ipython-input-756121233.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"": np.nan, "nan": np.nan})


KeyError: 'revenue_worldwide_bom'

In [ ]:
# --- FIX: Load BoM cache, normalize columns, merge, coalesce, save v5 ---

import pandas as pd, numpy as np, re, glob, os

MASTER_FILE = "horror_data_master_v4.csv.gz"
BOM_CACHE_FILE = "bom_cache (5).csv"
if not os.path.exists(BOM_CACHE_FILE):
    cand = sorted(glob.glob("bom_cache*.csv"))
    if not cand:
        raise FileNotFoundError("BoM cache not found.")
    BOM_CACHE_FILE = cand[-1]

# Load master
master = pd.read_csv(MASTER_FILE, compression="gzip", low_memory=False)
print(f"Loaded master: {len(master):,} rows")

# -------- Load BoM cache and normalize --------
bom = pd.read_csv(BOM_CACHE_FILE, dtype=str)
cols = [c.strip() for c in bom.columns]
bom.columns = cols

# Map any likely variants to our standard names
rename_map_variants = {
    "revenue_worldwide_bom": ["revenue_worldwide_bom","revenue_total_bom","worldwide","revenue_worldwide","total_worldwide"],
    "revenue_domestic_bom":  ["revenue_domestic_bom","domestic","revenue_domestic","gross_domestic"],
    "revenue_international_bom": ["revenue_international_bom","international","revenue_international","gross_international"],
    "release_date_bom": ["release_date_bom","release_date","us_release_date","date_us_release"]
}

for std, alts in rename_map_variants.items():
    for a in alts:
        if a in bom.columns:
            if a != std:
                bom.rename(columns={a: std}, inplace=True)
            break
    if std not in bom.columns:
        bom[std] = np.nan  # create if still missing

# Keep only what we need + tconst
need_cols = ["tconst","revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom","release_date_bom"]
missing_key = "tconst" not in bom.columns
if missing_key:
    raise ValueError("BoM cache must include a 'tconst' column to merge.")

bom = bom[need_cols].copy()

# Clean numbers
def to_money_num(s):
    return (pd.Series(s, dtype="object")
              .astype(str)
              .str.replace(r"[^\d.]", "", regex=True)
              .replace({"": np.nan, "nan": np.nan})
              .astype(float))

for c in ["revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom"]:
    bom[c] = to_money_num(bom[c])

# Parse date (optional)
bom["release_date_bom"] = pd.to_datetime(bom["release_date_bom"], errors="coerce")

# -------- Merge BoM into master --------
m_bom = master.merge(bom, on="tconst", how="left")
print("✅ Merged BoM cache")

# -------- If you already had Numbers/TMDb in v4, coalesce to total --------
rev_total = m_bom.get("revenue_total_usd", pd.Series(np.nan, index=m_bom.index, dtype="float")).copy()
src = pd.Series(pd.NA, index=m_bom.index, dtype="object")
src = np.where(rev_total.notna(), "existing", src)

# Candidate stack (adjust order if you want different priority)
cands = []
labels = []
if "revenue_total_numbers" in m_bom.columns:
    cands.append(m_bom["revenue_total_numbers"]); labels.append("numbers_total")
if "revenue_worldwide_bom" in m_bom.columns:
    cands.append(m_bom["revenue_worldwide_bom"]); labels.append("bom_worldwide")
if "revenue_tmdb" in m_bom.columns:
    cands.append(m_bom["revenue_tmdb"]); labels.append("tmdb_revenue")
if "revenue_domestic_numbers" in m_bom.columns:
    cands.append(m_bom["revenue_domestic_numbers"]); labels.append("numbers_domestic")
if "revenue_domestic_omdb" in m_bom.columns:
    cands.append(m_bom["revenue_domestic_omdb"]); labels.append("omdb_domestic")

for s, lab in zip(cands, labels):
    mask = rev_total.isna() & s.notna()
    rev_total[mask] = s[mask]
    src = np.where(mask, lab, src)

m_bom["revenue_total_usd"] = rev_total
m_bom["revenue_total_source"] = src

# If we brought TMDb budget earlier, expose unified budget
if "budget_tmdb" in m_bom.columns:
    m_bom["budget_usd"] = m_bom["budget_tmdb"]
    m_bom["budget_source"] = np.where(m_bom["budget_tmdb"].notna(), "tmdb", pd.NA)

# Prefer BoM release date when final is missing
if "release_date_bom" in m_bom.columns:
    if "release_date_final" not in m_bom.columns:
        m_bom["release_date_final"] = pd.NaT
    need = m_bom["release_date_final"].isna() & m_bom["release_date_bom"].notna()
    m_bom.loc[need, "release_date_final"] = m_bom.loc[need, "release_date_bom"]
    if "release_date_source" not in m_bom.columns:
        m_bom["release_date_source"] = pd.NA
    m_bom.loc[need, "release_date_source"] = "bom"

# Save v5
OUT = "horror_data_master_v5.csv.gz"
m_bom.to_csv(OUT, index=False, compression="gzip")
print(f"✅ Saved: {OUT}")

have = m_bom["revenue_total_usd"].notna().sum()
print(f"💰 revenue_total_usd coverage: {have:,}/{len(m_bom):,} ({have/len(m_bom):.1%})")

# peek
print(m_bom[["primaryTitle","startYear","revenue_total_usd","revenue_total_source",
             "revenue_worldwide_bom","revenue_total_numbers","revenue_tmdb"]].head(10))

Loaded master: 44,742 rows


/tmp/ipython-input-3260715815.py:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"": np.nan, "nan": np.nan})
/tmp/ipython-input-3260715815.py:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"": np.nan, "nan": np.nan})


✅ Merged BoM cache
✅ Saved: horror_data_master_v5.csv.gz
💰 revenue_total_usd coverage: 4,575/44,742 (10.2%)


KeyError: "['revenue_worldwide_bom', 'revenue_tmdb'] not in index"

In [ ]:
import pandas as pd, numpy as np, re

tmdb = pd.read_csv("horror_movies_tmdb_enriched - horror_movies_tmdb_enriched.csv.csv", low_memory=False)
print("TMDb columns:\n", tmdb.columns.tolist())

# Normalize and inspect
tmdb.columns = [c.lower() for c in tmdb.columns]
print("\nHas columns:", {"imdb_id": "imdb_id" in tmdb.columns,
                        "budget": "budget" in tmdb.columns,
                        "revenue": "revenue" in tmdb.columns})

print("\nNon-null counts:")
for c in ["imdb_id", "budget", "revenue", "title", "release_date"]:
    if c in tmdb.columns:
        print(f"  {c}: {tmdb[c].notna().sum():,}")

# peek at imdb_id formatting
if "imdb_id" in tmdb.columns:
    ex = tmdb["imdb_id"].dropna().astype(str).head(10).tolist()
    print("\nSample imdb_id values:", ex)

TMDb columns:
 ['id', 'title', 'release_date', 'original_language', 'overview', 'genre_ids', 'poster_path', 'budget', 'revenue', 'runtime', 'tagline', 'imdb_id', 'status_detail']

Has columns: {'imdb_id': True, 'budget': True, 'revenue': True}

Non-null counts:
  imdb_id: 5,070
  budget: 11,527
  revenue: 11,530
  title: 59,300
  release_date: 59,295

Sample imdb_id values: ['tt14847302', 'tt17081088', 'tt13942538', 'tt14993352', 'tt30208769', 'tt22029804', 'tt29031126', 'tt10778798', 'tt13316770', 'tt26547864']


V6 trying to mege of title and release date:

In [ ]:
import pandas as pd, numpy as np, re

MASTER_FILE = "horror_data_master_v5.csv.gz"  # use your latest saved master
TMDB_FILE   = "horror_movies_tmdb_enriched - horror_movies_tmdb_enriched.csv.csv"
OUT         = "horror_data_master_v6.csv.gz"

def to_num(s):
    return (pd.Series(s, dtype="object")
              .astype(str)
              .str.replace(r"[^\d.]", "", regex=True)
              .replace({"": np.nan, "nan": np.nan})
              .astype(float))

def title_std(s):
    return re.sub(r"[^a-z0-9]+", "", str(s).lower())

# Load
df   = pd.read_csv(MASTER_FILE, compression="gzip", low_memory=False)
tmdb = pd.read_csv(TMDB_FILE, low_memory=False)

# Normalize TMDb columns
tmdb.columns = [c.lower() for c in tmdb.columns]
if "imdb_id" not in tmdb.columns:
    tmdb["imdb_id"] = np.nan

# Clean money
for c in ("budget","revenue"):
    if c in tmdb.columns:
        tmdb[c] = to_num(tmdb[c])
    else:
        tmdb[c] = np.nan

# --- Pass 1: merge by IMDb ID ---
m = df.merge(
    tmdb[["imdb_id","budget","revenue"]],
    left_on="tconst", right_on="imdb_id", how="left"
)
m.rename(columns={"budget":"budget_tmdb_id","revenue":"revenue_tmdb_id"}, inplace=True)

# --- Build TMDb title_std + year for fallback ---
if "title" in tmdb.columns:
    tmdb["title_std"] = tmdb["title"].astype(str).map(title_std)
else:
    tmdb["title_std"] = np.nan

if "release_date" in tmdb.columns:
    tmdb["year"] = pd.to_datetime(tmdb["release_date"], errors="coerce").dt.year.astype("Int64")
else:
    tmdb["year"] = pd.NA

# Ensure master has keys
if "title_std" not in m.columns:
    base_title = m.get("primaryTitle", m.get("originalTitle"))
    m["title_std"] = base_title.astype(str).map(title_std)
if "year" not in m.columns:
    m["year"] = pd.to_numeric(m.get("startYear"), errors="coerce").astype("Int64")

# --- Pass 2: merge by title_std + year (only for rows still missing from ID pass) ---
tmdb_tit = tmdb[["title_std","year","budget","revenue"]].dropna(subset=["title_std","year"], how="any").copy()
tmdb_tit.rename(columns={"budget":"budget_tmdb_ttl","revenue":"revenue_tmdb_ttl"}, inplace=True)

m2 = m.merge(tmdb_tit, on=["title_std","year"], how="left")

# --- Coalesce TMDb budget/revenue from (ID pass) then (title pass) ---
m2["budget_tmdb"]  = m2["budget_tmdb_id"].combine_first(m2["budget_tmdb_ttl"])
m2["revenue_tmdb"] = m2["revenue_tmdb_id"].combine_first(m2["revenue_tmdb_ttl"])

# --- Unify budget_usd (+provenance) ---
if "budget_usd" not in m2.columns:
    m2["budget_usd"] = np.nan
if "budget_source" not in m2.columns:
    m2["budget_source"] = pd.NA

mask_bud = m2["budget_usd"].isna() & m2["budget_tmdb"].notna()
m2.loc[mask_bud, "budget_usd"]    = m2.loc[mask_bud, "budget_tmdb"]
m2.loc[mask_bud, "budget_source"] = "tmdb"

# --- Use TMDb revenue as a fallback for total (+provenance) ---
if "revenue_total_usd" not in m2.columns:
    m2["revenue_total_usd"] = np.nan
if "revenue_total_source" not in m2.columns:
    m2["revenue_total_source"] = pd.NA

mask_rev = m2["revenue_total_usd"].isna() & m2["revenue_tmdb"].notna()
m2.loc[mask_rev, "revenue_total_usd"]    = m2.loc[mask_rev, "revenue_tmdb"]
m2.loc[mask_rev, "revenue_total_source"] = "tmdb_revenue"

# Save
m2.to_csv(OUT, index=False, compression="gzip")
have_rev = m2["revenue_total_usd"].notna().sum()
have_bud = m2["budget_usd"].notna().sum()
print(f"✅ Saved: {OUT}")
print(f"💰 total revenue coverage: {have_rev:,}/{len(m2):,} ({have_rev/len(m2):.1%})")
print(f"💵 budget coverage:        {have_bud:,}/{len(m2):,} ({have_bud/len(m2):.1%})")

# sanity peek
print(m2[["primaryTitle","startYear","revenue_total_usd","revenue_total_source","revenue_tmdb","budget_usd","budget_source"]].head(10))

✅ Saved: horror_data_master_v6.csv.gz
💰 total revenue coverage: 6,992/45,056 (15.5%)
💵 budget coverage:        2,945/45,056 (6.5%)
                                        primaryTitle  startYear  \
0                              The Student of Prague     1913.0   
1  The Avenging Conscience: or 'Thou Shalt Not Kill'     1914.0   
2                                  The Ghost Breaker     1914.0   
3                                          The Golem     1914.0   
4               The Hound of the Baskervilles (1914)     1914.0   
5                      The Hound of the Baskervilles     1915.0   
6  Der Hund von Baskerville, 3. Teil - Das unheim...     1916.0   
7                  Der Hund von Baskerville, 4. Teil     1916.0   
8                                      Blind Justice     1916.0   
9                                  Life Without Soul     1915.0   

   revenue_total_usd revenue_total_source  revenue_tmdb  budget_usd  \
0                NaN                  NaN           NaN     

In [ ]:
import pandas as pd
v6 = pd.read_csv("horror_data_master_v6.csv.gz", compression="gzip")
print("Revenue coverage:", (v6["revenue_total_usd"].notna().mean()*100).round(1), "%")
print("Budget coverage:",  (v6["budget_usd"].notna().mean()*100).round(1), "%")
v6[["primaryTitle","startYear","revenue_tmdb","budget_tmdb","revenue_total_usd","revenue_total_source","budget_usd","budget_source"]].head(10)

/tmp/ipython-input-2358552272.py:2: DtypeWarning: Columns (14,23,63) have mixed types. Specify dtype option on import or set low_memory=False.
  v6 = pd.read_csv("horror_data_master_v6.csv.gz", compression="gzip")


Revenue coverage: 15.5 %
Budget coverage: 6.5 %


,primaryTitle,startYear,revenue_tmdb,budget_tmdb,revenue_total_usd,revenue_total_source,budget_usd,budget_source
0,The Student of Prague,1913.0,NaN,NaN,NaN,NaN,NaN,NaN
1,The Avenging Conscience: or 'Thou Shalt Not Kill',1914.0,NaN,NaN,NaN,NaN,NaN,NaN
2,The Ghost Breaker,1914.0,NaN,NaN,NaN,NaN,NaN,NaN
3,The Golem,1914.0,NaN,NaN,NaN,NaN,NaN,NaN
4,The Hound of the Baskervilles (1914),1914.0,NaN,NaN,NaN,NaN,NaN,NaN
5,The Hound of the Baskervilles,1915.0,NaN,NaN,NaN,NaN,NaN,NaN
6,"Der Hund von Baskerville, 3. Teil - Das unheim...",1916.0,NaN,NaN,NaN,NaN,NaN,NaN
7,"Der Hund von Baskerville, 4. Teil",1916.0,NaN,NaN,NaN,NaN,NaN,NaN
8,Blind Justice,1916.0,NaN,NaN,NaN,NaN,NaN,NaN
9,Life Without Soul,1915.0,NaN,NaN,NaN,NaN,NaN,NaN


Seeing how much data I have gathered:


In [ ]:
import glob, pandas as pd, numpy as np

# Auto-pick the newest master (or hardcode a filename if you prefer)
cands = sorted(glob.glob("horror_data_master_v*.csv*"))
if not cands:
    raise FileNotFoundError("No master file found (expected horror_data_master_v*.csv[.gz])")
MASTER = cands[-1]
comp = "gzip" if MASTER.endswith(".gz") else None

df = pd.read_csv(MASTER, compression=comp, low_memory=False)
print(f"📂 Loaded: {MASTER}  ({len(df):,} rows, {len(df.columns)} cols)")

def coverage(series):
    n = len(series)
    filled = series.notna().sum()
    pct = (filled / n) * 100 if n else 0.0
    return filled, n, pct

📂 Loaded: horror_data_master_v6.csv.gz  (45,056 rows, 73 cols)


In [ ]:
key_cols = [
    # spine / metadata
    "tconst","primaryTitle","startYear","runtimeMinutes","genres",
    "averageRating","numVotes","language_best","region_best",
    "DirectorsNames","WritersNames","TopCast",

    # dates
    "release_date_tmdb","release_date_omdb","release_date_final",

    # OMDb enrichments
    "imdb_rating_omdb","imdb_votes_omdb","metascore_omdb","rated_omdb",

    # financials by source
    "revenue_worldwide_bom","revenue_total_numbers","revenue_tmdb",
    "revenue_domestic_omdb","budget_tmdb",

    # unified fields + provenance
    "revenue_total_usd","revenue_total_source","budget_usd","budget_source"
]

present = [c for c in key_cols if c in df.columns]
missing = [c for c in key_cols if c not in df.columns]
if missing:
    print("⚠️ Missing in this master (skipped):", missing)

rows = []
for c in present:
    filled, n, pct = coverage(df[c])
    rows.append({"column": c, "filled": filled, "total": n, "pct": round(pct, 1)})

cov_key = pd.DataFrame(rows).sort_values(["pct","column"], ascending=[False, True]).reset_index(drop=True)
display(cov_key)

⚠️ Missing in this master (skipped): ['revenue_worldwide_bom']


,column,filled,total,pct
0,genres,45056,45056,100.0
1,primaryTitle,45055,45056,100.0
2,tconst,45056,45056,100.0
3,WritersNames,40737,45056,90.4
4,DirectorsNames,40057,45056,88.9
5,TopCast,36731,45056,81.5
6,startYear,32601,45056,72.4
7,release_date_final,31307,45056,69.5
8,runtimeMinutes,28536,45056,63.3
9,release_date_omdb,28143,45056,62.5


In [ ]:
rows_all = []
for c in df.columns:
    filled, n, pct = coverage(df[c])
    rows_all.append({"column": c, "filled": filled, "total": n, "pct": round(pct, 1)})

cov_all = pd.DataFrame(rows_all).sort_values(["pct","column"], ascending=[False, True]).reset_index(drop=True)
display(cov_all.head(40))  # top 40 most-complete columns

,column,filled,total,pct
0,genres,45056,45056,100.0
1,originalTitle,45055,45056,100.0
2,primaryTitle,45055,45056,100.0
3,tconst,45056,45056,100.0
4,title_std,45054,45056,100.0
5,WritersNames,40737,45056,90.4
6,DirectorsNames,40057,45056,88.9
7,TopCast,36731,45056,81.5
8,Title,34974,45056,77.6
9,Year,34975,45056,77.6


# idk bruh

In [ ]:
# ==== BoM Resume: Setup & Build TODO (targeted only) ====
import os, re, time, glob, math, random, requests
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

# -------- files (explicit) --------
MASTER_FILE = "horror_data_master_v6.csv.gz"     # your latest saved master
CACHE_FILE  = "bom_cache (5).csv"                # your existing BoM cache; change if needed
OUT_MASTER  = "horror_data_master_v7.csv.gz"     # will be created at the end

# -------- helpers --------
def to_money_num(x):
    return (pd.Series(x, dtype="object")
              .astype(str)
              .str.replace(r"[^\d.]", "", regex=True)
              .replace({"": np.nan, "nan": np.nan})
              .astype(float))

def any_gross_row(row):
    for c in ("revenue_worldwide_bom", "revenue_domestic_bom", "revenue_international_bom"):
        if c in row and pd.notna(row[c]) and str(row[c]) not in ("", "nan"):
            return True
    return False

def save_cache_safe(df_cache, path):
    df_cache.to_csv(path, index=False)
    print(f"💾 Cache saved: {len(df_cache):,} rows")
    # also mirror to Drive if mounted
    try:
        if os.path.exists("/content/drive"):
            drive_path = "/content/drive/MyDrive/horror_master/bom_cache.csv"
            df_cache.to_csv(drive_path, index=False)
            print(f"☁️  Also saved to Drive: {drive_path}")
    except Exception:
        pass

# -------- load master --------
comp = "gzip" if MASTER_FILE.endswith(".gz") else None
master = pd.read_csv(MASTER_FILE, compression=comp, low_memory=False)
print(f"📂 Loaded master: {MASTER_FILE}  ({len(master):,} rows)")

# ensure keys
if "tconst" not in master.columns:
    raise ValueError("Master must have 'tconst' IMDb IDs.")

# -------- load cache (or init) --------
if os.path.exists(CACHE_FILE):
    cache = pd.read_csv(CACHE_FILE, dtype=str)
    # normalize numeric columns
    for c in ("revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom"):
        if c in cache.columns:
            cache[c] = to_money_num(cache[c])
    print(f"🔁 Loaded BoM cache: {len(cache):,} rows")
else:
    cache = pd.DataFrame(columns=["tconst","status","revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom","release_date_bom"])
    print("🆕 Starting fresh BoM cache")

# baseline coverage (from cache only)
if len(cache):
    has_any = cache.apply(any_gross_row, axis=1).sum()
    print(f"📊 Baseline BoM-any coverage in cache: {has_any:,}/{len(cache):,} = {has_any/len(cache):.1%}")

# -------- build TODO (targeted) --------
# Only rows missing unified revenue in master:
missing_rev_master = master.loc[ master["revenue_total_usd"].isna(), ["tconst"] ].dropna().drop_duplicates()

# Join with cache to skip ones that already have a good 200 + some gross
cache_min = cache[["tconst","status","revenue_worldwide_bom","revenue_domestic_bom","revenue_international_bom"]].copy()
cache_min["has_any_gross"] = cache_min.apply(any_gross_row, axis=1)

todo = missing_rev_master.merge(cache_min, on="tconst", how="left")

# ensure boolean dtype even after merge (NaN -> False)
cache_min["has_any_gross"] = cache_min.apply(any_gross_row, axis=1).astype(bool)

todo = missing_rev_master.merge(cache_min, on="tconst", how="left")

# normalize status
if "status" in cache_min.columns:
    cache_min["status"] = cache_min["status"].astype(str)
if "status" in todo.columns:
    todo["status"] = todo["status"].astype(str)

status = todo["status"]
has_any = todo["has_any_gross"].fillna(False).astype(bool)

mask = status.isna() | (status != "200") | ((status == "200") & (~has_any))
todo_ids = todo.loc[mask, "tconst"].dropna().unique().tolist()

print(f"🎯 Targets to (re)query: {len(todo_ids):,} (out of {len(missing_rev_master):,} missing in master)")
print("First 10 targets:", todo_ids[:10])
print(f"🎯 Targets to (re)query: {len(todo_ids):,} (out of {len(missing_rev_master):,} missing in master)")

# quick peek
pd.Series(todo_ids[:10], name="first_10_targets")

📂 Loaded master: horror_data_master_v6.csv.gz  (45,056 rows)


/tmp/ipython-input-42897842.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"": np.nan, "nan": np.nan})
/tmp/ipython-input-42897842.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"": np.nan, "nan": np.nan})


🔁 Loaded BoM cache: 29,700 rows
📊 Baseline BoM-any coverage in cache: 1,618/29,700 = 5.4%
🎯 Targets to (re)query: 37,906 (out of 37,906 missing in master)
First 10 targets: ['tt0003419', 'tt0003643', 'tt0004013', 'tt0004026', 'tt0004121', 'tt0005231', 'tt0005513', 'tt0005514', 'tt0005529', 'tt0005615']
🎯 Targets to (re)query: 37,906 (out of 37,906 missing in master)


/tmp/ipython-input-42897842.py:87: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  has_any = todo["has_any_gross"].fillna(False).astype(bool)


,first_10_targets
0,tt0003419
1,tt0003643
2,tt0004013
3,tt0004026
4,tt0004121
5,tt0005231
6,tt0005513
7,tt0005514
8,tt0005529
9,tt0005615


In [ ]:
# ==== BoM Resume: Smoke test (first few only) ====

SMOKE_N = 5   # change to 10 if you want

def parse_bom_html(html):
    # Extract numbers by label anywhere on the page text (robust fallback)
    text = BeautifulSoup(html, "html.parser").get_text(" ", strip=True)
    # Try common labels
    def grab(label):
        m = re.search(label + r".{0,40}\$([0-9,]+)", text, flags=re.I)
        return m.group(1) if m else None

    dom = grab(r"Domestic")
    intl = grab(r"International")
    ww  = grab(r"Worldwide")
    # Fallback: sometimes "Grosses" block prints "Worldwide:" only
    return {
        "revenue_domestic_bom": dom,
        "revenue_international_bom": intl,
        "revenue_worldwide_bom": ww
    }

def fetch_one_bom(tconst, sleep_s=0.25):
    url = f"https://www.boxofficemojo.com/title/{tconst}/"
    r = requests.get(url, timeout=20)
    status = r.status_code
    out = {"tconst": tconst, "status": str(status)}
    if status == 200:
        vals = parse_bom_html(r.text)
        out.update(vals)
    time.sleep(sleep_s)
    return out

print("🔎 Smoke testing on first few targets...")
samples = []
for imdb_id in todo_ids[:SMOKE_N]:
    try:
        row = fetch_one_bom(imdb_id, sleep_s=0.4 + random.random()*0.2)
        samples.append(row)
        print(row)
    except Exception as e:
        print(f"{imdb_id} -> ERROR {e}")

pd.DataFrame(samples)

🔎 Smoke testing on first few targets...
{'tconst': 'tt0003419', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0003643', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0004013', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0004026', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0004121', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}


,tconst,status,revenue_domestic_bom,revenue_international_bom,revenue_worldwide_bom
0,tt0003419,200,None,None,None
1,tt0003643,200,None,None,None
2,tt0004013,200,None,None,None
3,tt0004026,200,None,None,None
4,tt0004121,200,None,None,None


Smoke test v2

In [ ]:
import pandas as pd
import numpy as np

# Build lookups from master
year_map  = dict(zip(master["tconst"], master["startYear"]))
votes_map = dict(zip(master["tconst"], master["numVotes"]))

# Sort targets: newest first, then most-voted
todo_sorted = sorted(
    todo_ids,
    key=lambda t: (pd.to_numeric(year_map.get(t, np.nan), errors="coerce"),
                   pd.to_numeric(votes_map.get(t, np.nan), errors="coerce")),
    reverse=True
)

# Try the FIRST 5 of this sorted list
SMOKE_LIST = todo_sorted[:5]
print("Smoke candidates:", SMOKE_LIST)

samples = []
for imdb_id in SMOKE_LIST:
    try:
        row = fetch_one_bom(imdb_id, sleep_s=0.35)
        samples.append(row)
        print(row)
    except Exception as e:
        print(f"{imdb_id} -> ERROR {e}")

pd.DataFrame(samples)

Smoke candidates: ['tt0191010', 'tt0260293', 'tt0093119', 'tt0339736', 'tt0464054']
{'tconst': 'tt0191010', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0260293', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0093119', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0339736', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0464054', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}


,tconst,status,revenue_domestic_bom,revenue_international_bom,revenue_worldwide_bom
0,tt0191010,200,None,None,None
1,tt0260293,200,None,None,None
2,tt0093119,200,None,None,None
3,tt0339736,200,None,None,None
4,tt0464054,200,None,None,None


Smoke test v3

In [ ]:
# Filter todo_ids to year>=1980 & numVotes>=1000
year_map  = dict(zip(master["tconst"], master["startYear"]))
votes_map = dict(zip(master["tconst"], master["numVotes"]))

todo_ids_modern = [
    t for t in todo_ids
    if pd.to_numeric(year_map.get(t, np.nan), errors="coerce") >= 1980
    and pd.to_numeric(votes_map.get(t, np.nan), errors="coerce") >= 1000
]

print("Filtered targets (year>=1980 & votes>=1000):", len(todo_ids_modern))

# Smoke test on this filtered set
SMOKE_LIST = todo_ids_modern[:5]
samples = []
for imdb_id in SMOKE_LIST:
    row = fetch_one_bom(imdb_id, sleep_s=0.35)
    samples.append(row)
    print(row)
pd.DataFrame(samples)

Filtered targets (year>=1980 & votes>=1000): 2375
{'tconst': 'tt0078749', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0078751', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0078935', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0078936', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0079642', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}


,tconst,status,revenue_domestic_bom,revenue_international_bom,revenue_worldwide_bom
0,tt0078749,200,None,None,None
1,tt0078751,200,None,None,None
2,tt0078935,200,None,None,None
3,tt0078936,200,None,None,None
4,tt0079642,200,None,None,None


Smoke test v4

In [ ]:
import random
random.seed(42)  # reproducible

SMOKE_LIST = random.sample(todo_ids, k=min(5, len(todo_ids)))
print("Random smoke candidates:", SMOKE_LIST)

samples = []
for imdb_id in SMOKE_LIST:
    row = fetch_one_bom(imdb_id, sleep_s=0.35)
    samples.append(row)
    print(row)
pd.DataFrame(samples)

Random smoke candidates: ['tt10432954', 'tt0074806', 'tt2269544', 'tt19844394', 'tt17275688']
{'tconst': 'tt10432954', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt0074806', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt2269544', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt19844394', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}
{'tconst': 'tt17275688', 'status': '200', 'revenue_domestic_bom': None, 'revenue_international_bom': None, 'revenue_worldwide_bom': None}


,tconst,status,revenue_domestic_bom,revenue_international_bom,revenue_worldwide_bom
0,tt10432954,200,None,None,None
1,tt0074806,200,None,None,None
2,tt2269544,200,None,None,None
3,tt19844394,200,None,None,None
4,tt17275688,200,None,None,None


In [ ]:
# ==== BoM Resume: Refill loop with frequent saves + coverage reports ====

# Tunables — make it noisy so you can SEE progress
BATCH_SAVE    = 200        # save every N fetches
REPORT_EVERY  = 400        # print coverage update every N fetches
MAX_REQUESTS  = None       # set to e.g. 2000 if you want to cap for a shorter run
BASE_DELAY    = 0.30       # base delay between requests
JITTER        = 0.20       # random 0..JITTER added to delay

# Make a mutable working copy of the cache
cache_work = cache.copy()

def upsert_cache_row(cache_df, d):
    # ensure columns
    for c in ["tconst","status","revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom","release_date_bom"]:
        if c not in cache_df.columns:
            cache_df[c] = np.nan
    # insert/replace
    t = d.get("tconst")
    if t is None:
        return cache_df
    idx = cache_df.index[cache_df["tconst"] == t]
    row = {
        "tconst": t,
        "status": str(d.get("status", "")),
        "revenue_domestic_bom": d.get("revenue_domestic_bom"),
        "revenue_international_bom": d.get("revenue_international_bom"),
        "revenue_worldwide_bom": d.get("revenue_worldwide_bom"),
        "release_date_bom": d.get("release_date_bom"),
    }
    if len(idx):
        cache_df.loc[idx[0], list(row.keys())] = list(row.values())
    else:
        cache_df = pd.concat([cache_df, pd.DataFrame([row])], ignore_index=True)
    return cache_df

fetched = 0
errors  = 0
t0 = time.time()

for i, imdb_id in enumerate(tqdm(todo_ids, desc="Refilling BoM (targeted)")):
    if MAX_REQUESTS and fetched >= MAX_REQUESTS:
        print("⏹️ Reached MAX_REQUESTS; stopping early.")
        break
    try:
        d = fetch_one_bom(imdb_id, sleep_s=BASE_DELAY + random.random()*JITTER)
        cache_work = upsert_cache_row(cache_work, d)
        fetched += 1
    except Exception as e:
        errors += 1

    if fetched % BATCH_SAVE == 0:
        save_cache_safe(cache_work, CACHE_FILE)

    if fetched % REPORT_EVERY == 0:
        # quick coverage inside cache
        tmp = cache_work.copy()
        for c in ("revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom"):
            if c in tmp.columns:
                tmp[c] = to_money_num(tmp[c])
        cov = tmp.apply(any_gross_row, axis=1).sum()
        print(f"⏱️ Fetched {fetched:,} | cache rows {len(tmp):,} | BoM-any coverage in cache: {cov:,} ({cov/len(tmp):.1%})")

# Final save
save_cache_safe(cache_work, CACHE_FILE)

print(f"✅ Done. fetched={fetched:,}, errors={errors:,}, elapsed ~{(time.time()-t0)/60:.1f} min")

# Show a few of the newest rows that now have any gross
tmp = cache_work.copy()
for c in ("revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom"):
    if c in tmp.columns:
        tmp[c] = to_money_num(tmp[c])
hits = tmp[tmp.apply(any_gross_row, axis=1)]
print("🆕 Recent hits (tail 5):")
display(hits.tail(5))

In [ ]:
# ==== Merge refreshed BoM cache into master and save v7 ====
# (Lightweight — runs quickly)

# reload cache (normalized)
bom = pd.read_csv(CACHE_FILE, dtype=str)
for c in ("revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom"):
    if c in bom.columns:
        bom[c] = to_money_num(bom[c])

# merge
df = master.merge(
    bom[["tconst","revenue_domestic_bom","revenue_international_bom","revenue_worldwide_bom"]],
    on="tconst", how="left"
)

# coalesce unified total without overwriting existing
if "revenue_total_usd" not in df.columns:
    df["revenue_total_usd"] = np.nan
if "revenue_total_source" not in df.columns:
    df["revenue_total_source"] = pd.NA

# prefer existing; fill with BoM worldwide if still missing
mask = df["revenue_total_usd"].isna() & df["revenue_worldwide_bom"].notna()
df.loc[mask, "revenue_total_usd"]    = df.loc[mask, "revenue_worldwide_bom"]
df.loc[mask, "revenue_total_source"] = "bom_worldwide_refill"

# save v7
df.to_csv(OUT_MASTER, index=False, compression="gzip")
have = df["revenue_total_usd"].notna().sum()
print(f"✅ Saved: {OUT_MASTER}")
print(f"💰 total revenue coverage: {have:,}/{len(df):,} ({have/len(df):.1%})")

# peek at the freshly filled
print("🧾 Newly filled (sample):")
display(df.loc[mask, ["primaryTitle","startYear","revenue_total_usd","revenue_total_source"]].head(10))

Adding the numbers data in

In [ ]:
# ===========================================
# Merge "The Numbers" into master v6 (two-pass title match)
# Output: horror_data_master_v7.csv.gz
# ===========================================

import pandas as pd, numpy as np, re, glob, os

# ---------- CONFIG ----------
MASTER_CANDIDATES = ["horror_data_master_v6.csv.gz", "horror_data_master_v6.csv"]
NUMBERS_FILE = "the_numbers_horror_raw.csv"
OUT = "horror_data_master_v7.csv.gz"

# ---------- LOAD MASTER ----------
master_path = None
for p in MASTER_CANDIDATES:
    if os.path.exists(p):
        master_path = p
        break
if master_path is None:
    # fall back to "latest v*" if needed
    cand = sorted(glob.glob("horror_data_master_v*.csv*"))
    if not cand:
        raise FileNotFoundError("No master file found. Expected horror_data_master_v6.csv[.gz].")
    master_path = cand[-1]

comp = "gzip" if master_path.endswith(".gz") else None
df = pd.read_csv(master_path, compression=comp, low_memory=False)
print(f"📂 Loaded master: {master_path}  ({len(df):,} rows)")

# ---------- HELPERS ----------
def to_money_num(s):
    return (pd.Series(s, dtype="object")
              .astype(str)
              .str.replace(r"[^\d.]", "", regex=True)
              .replace({"": np.nan, "nan": np.nan})
              .astype(float))

def norm_title(s):
    # lower, strip accents-like punctuation, collapse spaces, remove punctuation
    s = str(s).lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)  # keep letters/numbers/underscore and spaces
    s = s.replace("_", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ensure year in master
df["year"] = pd.to_numeric(df.get("startYear"), errors="coerce").astype("Int64")
df["primary_std"]  = df.get("primaryTitle", "").map(norm_title)
df["original_std"] = df.get("originalTitle", "").map(norm_title)

# ---------- LOAD & PREP THE NUMBERS ----------
num = pd.read_csv(NUMBERS_FILE, low_memory=False)
# normalize headers (some scrape variants include spaces)
num.columns = num.columns.str.replace(r"\s+", " ", regex=True).str.strip()

# expected columns from your inspection
col_title = "Movie"
col_year  = "Released"
col_dom   = "DomesticBox Office"
col_intl  = "InternationalBox Office"
col_ww    = "WorldwideBoxOffice" if "WorldwideBoxOffice" in num.columns else "WorldwideBox Office"

if col_title not in num.columns or col_year not in num.columns:
    raise ValueError(f"Numbers file must have title/year. Found columns: {num.columns.tolist()}")

keep = [col_title, col_year] + [c for c in [col_dom, col_intl, col_ww] if c in num.columns]
num = num[keep].copy()

# rename to _numbers
rename_map = {
    col_title: "title_numbers",
    col_year: "year_numbers",
}
if col_dom in num.columns:
    rename_map[col_dom] = "revenue_domestic_numbers"
if col_intl in num.columns:
    rename_map[col_intl] = "revenue_international_numbers"
if col_ww in num.columns:
    rename_map[col_ww] = "revenue_worldwide_numbers"

num.rename(columns=rename_map, inplace=True)

# clean values
num["title_numbers_std"] = num["title_numbers"].map(norm_title)
num["year_numbers"] = pd.to_numeric(num["year_numbers"], errors="coerce").astype("Int64")

for c in ["revenue_domestic_numbers", "revenue_international_numbers", "revenue_worldwide_numbers"]:
    if c in num.columns:
        num[c] = to_money_num(num[c])

# create a numbers "total" preferring worldwide; else sum dom+intl; else whichever exists
num["revenue_total_numbers"] = np.nan
src_num = pd.Series(pd.NA, index=num.index, dtype="object")

if "revenue_worldwide_numbers" in num.columns:
    m = num["revenue_worldwide_numbers"].notna()
    num.loc[m, "revenue_total_numbers"] = num.loc[m, "revenue_worldwide_numbers"]
    src_num = np.where(m, "numbers_worldwide", src_num)

if "revenue_domestic_numbers" in num.columns and "revenue_international_numbers" in num.columns:
    m = num["revenue_total_numbers"].isna() & num["revenue_domestic_numbers"].notna() & num["revenue_international_numbers"].notna()
    num.loc[m, "revenue_total_numbers"] = num.loc[m, "revenue_domestic_numbers"] + num.loc[m, "revenue_international_numbers"]
    src_num = np.where(m, "numbers_domestic+international", src_num)

# fallback: take whichever exists
for col, tag in [
    ("revenue_domestic_numbers", "numbers_domestic"),
    ("revenue_international_numbers", "numbers_international"),
]:
    if col in num.columns:
        m = num["revenue_total_numbers"].isna() & num[col].notna()
        num.loc[m, "revenue_total_numbers"] = num.loc[m, col]
        src_num = np.where(m, tag, src_num)

num["revenue_total_numbers_source"] = src_num

print(f"🧾 The Numbers rows: {len(num):,}  | with total: {num['revenue_total_numbers'].notna().sum():,}")

# ---------- MERGE PASS 1: primaryTitle + year ----------
m1 = df.merge(
    num[
        ["title_numbers_std","year_numbers",
         "revenue_domestic_numbers","revenue_international_numbers","revenue_worldwide_numbers",
         "revenue_total_numbers","revenue_total_numbers_source"]
    ],
    left_on=["primary_std","year"],
    right_on=["title_numbers_std","year_numbers"],
    how="left",
    suffixes=("","_num1")
)

# ---------- MERGE PASS 2: originalTitle + year (for rows still not matched) ----------
m2 = m1.merge(
    num[
        ["title_numbers_std","year_numbers",
         "revenue_domestic_numbers","revenue_international_numbers","revenue_worldwide_numbers",
         "revenue_total_numbers","revenue_total_numbers_source"]
    ].rename(columns={
        "revenue_domestic_numbers":"revenue_domestic_numbers_alt",
        "revenue_international_numbers":"revenue_international_numbers_alt",
        "revenue_worldwide_numbers":"revenue_worldwide_numbers_alt",
        "revenue_total_numbers":"revenue_total_numbers_alt",
        "revenue_total_numbers_source":"revenue_total_numbers_source_alt"
    }),
    left_on=["original_std","year"],
    right_on=["title_numbers_std","year_numbers"],
    how="left"
)

# ---------- COALESCE NUMBERS COLUMNS (ID pass first, then title alt) ----------
def coalesce(a, b):
    return a.combine_first(b) if a is not None else b

for pair in [
    ("revenue_domestic_numbers", "revenue_domestic_numbers_alt"),
    ("revenue_international_numbers", "revenue_international_numbers_alt"),
    ("revenue_worldwide_numbers", "revenue_worldwide_numbers_alt"),
    ("revenue_total_numbers", "revenue_total_numbers_alt"),
    ("revenue_total_numbers_source", "revenue_total_numbers_source_alt"),
]:
    a, b = pair
    if a in m2.columns and b in m2.columns:
        m2[a] = coalesce(m2[a], m2[b])

# drop helper alt cols
m2.drop(columns=[c for c in m2.columns if c.endswith("_alt")], inplace=True, errors="ignore")

# ---------- UPDATE UNIFIED TOTAL (only where missing) ----------
before = m2["revenue_total_usd"].notna().sum() if "revenue_total_usd" in m2.columns else 0
if "revenue_total_usd" not in m2.columns:
    m2["revenue_total_usd"] = np.nan
if "revenue_total_source" not in m2.columns:
    m2["revenue_total_source"] = pd.NA

mask_fill = m2["revenue_total_usd"].isna() & m2["revenue_total_numbers"].notna()
m2.loc[mask_fill, "revenue_total_usd"] = m2.loc[mask_fill, "revenue_total_numbers"]
m2.loc[mask_fill, "revenue_total_source"] = m2.loc[mask_fill, "revenue_total_numbers_source"]

after = m2["revenue_total_usd"].notna().sum()
print(f"💰 unified total filled from Numbers: +{after - before:,} rows (now {after:,}/{len(m2):,} = {after/len(m2):.1%})")

# ---------- SAVE ----------
m2.to_csv(OUT, index=False, compression="gzip")
print(f"✅ Saved: {OUT}")

# ---------- QUICK SAMPLE ----------
cols_show = [
    "primaryTitle","originalTitle","year",
    "revenue_worldwide_numbers","revenue_domestic_numbers","revenue_international_numbers",
    "revenue_total_numbers","revenue_total_numbers_source",
    "revenue_total_usd","revenue_total_source"
]
cols_show = [c for c in cols_show if c in m2.columns]
print("\n🧪 Sample:")
print(m2[cols_show].head(10))

📂 Loaded master: horror_data_master_v6.csv  (45,056 rows)
🧾 The Numbers rows: 2,117  | with total: 2,117
💰 unified total filled from Numbers: +18 rows (now 7,012/45,058 = 15.6%)
✅ Saved: horror_data_master_v7.csv.gz

🧪 Sample:
                                        primaryTitle  \
0                              The Student of Prague   
1  The Avenging Conscience: or 'Thou Shalt Not Kill'   
2                                  The Ghost Breaker   
3                                          The Golem   
4               The Hound of the Baskervilles (1914)   
5                      The Hound of the Baskervilles   
6  Der Hund von Baskerville, 3. Teil - Das unheim...   
7                  Der Hund von Baskerville, 4. Teil   
8                                      Blind Justice   
9                                  Life Without Soul   

                                       originalTitle  year  \
0                               Der Student von Prag  1913   
1  The Avenging Conscience: or '

In [ ]:
import pandas as pd, numpy as np, re, os, glob

MASTER = "horror_data_master_v6.csv.gz"
NUMBERS = "the_numbers_horror_raw.csv"

comp = "gzip" if MASTER.endswith(".gz") else None
df = pd.read_csv(MASTER, compression=comp, low_memory=False)

num = pd.read_csv(NUMBERS, low_memory=False)
num.columns = num.columns.str.replace(r"\s+", " ", regex=True).str.strip()

def strip_trailing_parens(s: str) -> str:
    # remove ONE trailing parenthetical group, e.g. "Title (2014)" or "Title (aka Something)"
    return re.sub(r"\s*\([^)]*\)\s*$", "", str(s)).strip()

def norm_title(s: str) -> str:
    s = strip_trailing_parens(s)
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)  # remove punctuation
    s = s.replace("_", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

# master keys
df["year"] = pd.to_numeric(df.get("startYear"), errors="coerce").astype("Int64")
df["primary_std2"]  = df.get("primaryTitle","").map(norm_title)
df["original_std2"] = df.get("originalTitle","").map(norm_title)

# numbers keys
col_title = "Movie"
col_year  = "Released"
num["title_numbers_std2"] = num[col_title].map(norm_title)
num["year_numbers"] = pd.to_numeric(num[col_year], errors="coerce").astype("Int64")

left_key_primary  = set(zip(df["primary_std2"],  df["year"]))
left_key_original = set(zip(df["original_std2"], df["year"]))
right_key         = set(zip(num["title_numbers_std2"], num["year_numbers"]))

overlap_primary = len(left_key_primary & right_key)
overlap_any     = len((left_key_primary | left_key_original) & right_key)

print(f"🔎 Overlap (primary+year): {overlap_primary:,}")
print(f"🔎 Overlap (primary/original + year): {overlap_any:,}")

print("Examples of master titles that likely failed before but should match now:")
print(df.loc[df["primaryTitle"].str.contains(r"\(\d{4}\)", na=False), ["primaryTitle","startYear"]].head(10))

🔎 Overlap (primary+year): 1,193
🔎 Overlap (primary/original + year): 1,352
Examples of master titles that likely failed before but should match now:
                               primaryTitle  startYear
4      The Hound of the Baskervilles (1914)     1914.0
16079                 Happiness Costs(2023)        NaN
26741                        Siharan (2023)        NaN
41284                       Pit Stop (2020)     2019.0


Error cell:

In [ ]:
# ===========================================
# Merge "The Numbers" into master v6 (fixed normalization)
# Output: horror_data_master_v7.csv.gz
# ===========================================

import pandas as pd, numpy as np, re, os, glob

MASTER_CANDIDATES = ["horror_data_master_v6.csv.gz", "horror_data_master_v6.csv"]
NUMBERS_FILE = "the_numbers_horror_raw.csv"
OUT = "horror_data_master_v7.csv.gz"

# ---- load master ----
master_path = None
for p in MASTER_CANDIDATES:
    if os.path.exists(p):
        master_path = p; break
if master_path is None:
    cand = sorted(glob.glob("horror_data_master_v*.csv*"))
    if not cand:
        raise FileNotFoundError("No master file found.")
    master_path = cand[-1]

comp = "gzip" if master_path.endswith(".gz") else None
df = pd.read_csv(master_path, compression=comp, low_memory=False)
print(f"📂 Loaded master: {master_path}  ({len(df):,} rows)")

# ---- helpers ----
def strip_trailing_parens(s: str) -> str:
    return re.sub(r"\s*\([^)]*\)\s*$", "", str(s)).strip()

def norm_title(s: str) -> str:
    s = strip_trailing_parens(s)
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)
    s = s.replace("_", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def to_money_num(s):
    return (pd.Series(s, dtype="object")
              .astype(str)
              .str.replace(r"[^\d.]", "", regex=True)
              .replace({"": np.nan, "nan": np.nan})
              .astype(float))

# keys on master
df["year"] = pd.to_numeric(df.get("startYear"), errors="coerce").astype("Int64")
df["primary_std2"]  = df.get("primaryTitle","").map(norm_title)
df["original_std2"] = df.get("originalTitle","").map(norm_title)

# ---- load + prep The Numbers ----
num = pd.read_csv(NUMBERS_FILE, low_memory=False)
num.columns = num.columns.str.replace(r"\s+", " ", regex=True).str.strip()

col_title = "Movie"
col_year  = "Released"
col_dom   = "DomesticBox Office"
col_intl  = "InternationalBox Office"
col_ww    = "WorldwideBoxOffice" if "WorldwideBoxOffice" in num.columns else "WorldwideBox Office"

need = [col_title, col_year]
for c in need:
    if c not in num.columns:
        raise ValueError(f"Numbers file missing column: {c}")

keep = [c for c in [col_title, col_year, col_dom, col_intl, col_ww] if c in num.columns]
num = num[keep].copy()

num.rename(columns={
    col_title: "title_numbers",
    col_year:  "year_numbers",
    col_dom:   "revenue_domestic_numbers",
    col_intl:  "revenue_international_numbers",
    col_ww:    "revenue_worldwide_numbers"
}, inplace=True)

num["title_numbers_std2"] = num["title_numbers"].map(norm_title)
num["year_numbers"] = pd.to_numeric(num["year_numbers"], errors="coerce").astype("Int64")

for c in ["revenue_domestic_numbers","revenue_international_numbers","revenue_worldwide_numbers"]:
    if c in num.columns:
        num[c] = to_money_num(num[c])

# compute a robust total
num["revenue_total_numbers"] = np.nan
src = pd.Series(pd.NA, index=num.index, dtype="object")

if "revenue_worldwide_numbers" in num.columns:
    m = num["revenue_worldwide_numbers"].notna()
    num.loc[m, "revenue_total_numbers"] = num.loc[m, "revenue_worldwide_numbers"]
    src = np.where(m, "numbers_worldwide", src)

if "revenue_domestic_numbers" in num.columns and "revenue_international_numbers" in num.columns:
    m = num["revenue_total_numbers"].isna() & num["revenue_domestic_numbers"].notna() & num["revenue_international_numbers"].notna()
    num.loc[m, "revenue_total_numbers"] = num.loc[m, "revenue_domestic_numbers"] + num.loc[m, "revenue_international_numbers"]
    src = np.where(m, "numbers_domestic+international", src)

for col, tag in [("revenue_domestic_numbers","numbers_domestic"),
                 ("revenue_international_numbers","numbers_international")]:
    if col in num.columns:
        m = num["revenue_total_numbers"].isna() & num[col].notna()
        num.loc[m, "revenue_total_numbers"] = num.loc[m, col]
        src = np.where(m, tag, src)

num["revenue_total_numbers_source"] = src

print(f"🧾 Numbers rows: {len(num):,} | with any total: {num['revenue_total_numbers'].notna().sum():,}")

# ---- MERGE PASS 1: (primary_std2, year) ----
m1 = df.merge(
    num[["title_numbers_std2","year_numbers",
         "revenue_domestic_numbers","revenue_international_numbers","revenue_worldwide_numbers",
         "revenue_total_numbers","revenue_total_numbers_source"]],
    left_on=["primary_std2","year"],
    right_on=["title_numbers_std2","year_numbers"],
    how="left"
)

# ---- MERGE PASS 2: (original_std2, year) for gaps ----
m2 = m1.merge(
    num[["title_numbers_std2","year_numbers",
         "revenue_domestic_numbers","revenue_international_numbers","revenue_worldwide_numbers",
         "revenue_total_numbers","revenue_total_numbers_source"]]
    .rename(columns={
        "revenue_domestic_numbers":"revenue_domestic_numbers_alt",
        "revenue_international_numbers":"revenue_international_numbers_alt",
        "revenue_worldwide_numbers":"revenue_worldwide_numbers_alt",
        "revenue_total_numbers":"revenue_total_numbers_alt",
        "revenue_total_numbers_source":"revenue_total_numbers_source_alt"
    }),
    left_on=["original_std2","year"],
    right_on=["title_numbers_std2","year_numbers"],
    how="left"
)

# coalesce Numbers columns
for a, b in [
    ("revenue_domestic_numbers","revenue_domestic_numbers_alt"),
    ("revenue_international_numbers","revenue_international_numbers_alt"),
    ("revenue_worldwide_numbers","revenue_worldwide_numbers_alt"),
    ("revenue_total_numbers","revenue_total_numbers_alt"),
    ("revenue_total_numbers_source","revenue_total_numbers_source_alt"),
]:
    if a in m2.columns and b in m2.columns:
        m2[a] = m2[a].combine_first(m2[b])

m2.drop(columns=[c for c in m2.columns if c.endswith("_alt")], inplace=True, errors="ignore")

# ---- fill unified total only where missing ----
if "revenue_total_usd" not in m2.columns:
    m2["revenue_total_usd"] = np.nan
if "revenue_total_source" not in m2.columns:
    m2["revenue_total_source"] = pd.NA

before = m2["revenue_total_usd"].notna().sum()
mask_fill = m2["revenue_total_usd"].isna() & m2["revenue_total_numbers"].notna()
m2.loc[mask_fill, "revenue_total_usd"]    = m2.loc[mask_fill, "revenue_total_numbers"]
m2.loc[mask_fill, "revenue_total_source"] = m2.loc[mask_fill, "revenue_total_numbers_source"]
after = m2["revenue_total_usd"].notna().sum()

print(f"💰 Unified total filled from Numbers: +{after - before:,} rows (now {after:,}/{len(m2):,} = {after/len(m2):.1%})")

# ---- save ----
m2.to_csv(OUT, index=False, compression="gzip")
print(f"✅ Saved: {OUT}")

# peek
cols = ["primaryTitle","originalTitle","year",
        "revenue_worldwide_numbers","revenue_domestic_numbers","revenue_international_numbers",
        "revenue_total_numbers","revenue_total_numbers_source",
        "revenue_total_usd","revenue_total_source"]
print(m2[[c for c in cols if c in m2.columns]].head(12))

📂 Loaded master: horror_data_master_v6.csv.gz  (45,056 rows)
🧾 Numbers rows: 2,117 | with any total: 2,117


KeyError: 'revenue_total_numbers'

In [ ]:
x

In [ ]:
# ===========================================
# Merge "The Numbers" into master v6 (fixed + suffix-safe)
# Output: horror_data_master_v7.csv.gz
# ===========================================

import pandas as pd, numpy as np, re, os, glob

MASTER_CANDIDATES = ["horror_data_master_v6.csv.gz", "horror_data_master_v6.csv"]
NUMBERS_FILE = "the_numbers_horror_raw.csv"
OUT = "horror_data_master_v7.csv.gz"

# ---- load master ----
master_path = None
for p in MASTER_CANDIDATES:
    if os.path.exists(p):
        master_path = p; break
if master_path is None:
    cand = sorted(glob.glob("horror_data_master_v*.csv*"))
    if not cand:
        raise FileNotFoundError("No master file found.")
    master_path = cand[-1]

comp = "gzip" if master_path.endswith(".gz") else None
df = pd.read_csv(master_path, compression=comp, low_memory=False)
print(f"📂 Loaded master: {master_path}  ({len(df):,} rows)")

# ---- helpers ----
def strip_trailing_parens(s: str) -> str:
    return re.sub(r"\s*\([^)]*\)\s*$", "", str(s)).strip()

def norm_title(s: str) -> str:
    s = strip_trailing_parens(s)
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)
    s = s.replace("_", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def to_money_num(s):
    return (pd.Series(s, dtype="object").astype(str)
              .str.replace(r"[^\d.]", "", regex=True)
              .replace({"": np.nan, "nan": np.nan})
              .astype(float))

# keys on master
df["year"] = pd.to_numeric(df.get("startYear"), errors="coerce").astype("Int64")
df["primary_std2"]  = df.get("primaryTitle","").map(norm_title)
df["original_std2"] = df.get("originalTitle","").map(norm_title)

# ---- load + prep The Numbers ----
num = pd.read_csv(NUMBERS_FILE, low_memory=False)
num.columns = num.columns.str.replace(r"\s+", " ", regex=True).str.strip()

col_title = "Movie"
col_year  = "Released"
col_dom   = "DomesticBox Office"
col_intl  = "InternationalBox Office"
col_ww    = "WorldwideBoxOffice" if "WorldwideBoxOffice" in num.columns else "WorldwideBox Office"

need = [col_title, col_year]
for c in need:
    if c not in num.columns:
        raise ValueError(f"Numbers file missing column: {c}")

keep = [c for c in [col_title, col_year, col_dom, col_intl, col_ww] if c in num.columns]
num = num[keep].copy()

num.rename(columns={
    col_title: "title_numbers",
    col_year:  "year_numbers",
    col_dom:   "revenue_domestic_numbers",
    col_intl:  "revenue_international_numbers",
    col_ww:    "revenue_worldwide_numbers"
}, inplace=True)

num["title_numbers_std2"] = num["title_numbers"].map(norm_title)
num["year_numbers"] = pd.to_numeric(num["year_numbers"], errors="coerce").astype("Int64")

for c in ["revenue_domestic_numbers","revenue_international_numbers","revenue_worldwide_numbers"]:
    if c in num.columns:
        num[c] = to_money_num(num[c])

# compute a robust total in Numbers
num["revenue_total_numbers"] = np.nan
src = pd.Series(pd.NA, index=num.index, dtype="object")

if "revenue_worldwide_numbers" in num.columns:
    m = num["revenue_worldwide_numbers"].notna()
    num.loc[m, "revenue_total_numbers"] = num.loc[m, "revenue_worldwide_numbers"]
    src = np.where(m, "numbers_worldwide", src)

if "revenue_domestic_numbers" in num.columns and "revenue_international_numbers" in num.columns:
    m = num["revenue_total_numbers"].isna() & num["revenue_domestic_numbers"].notna() & num["revenue_international_numbers"].notna()
    num.loc[m, "revenue_total_numbers"] = num.loc[m, "revenue_domestic_numbers"] + num.loc[m, "revenue_international_numbers"]
    src = np.where(m, "numbers_domestic+international", src)

for col, tag in [("revenue_domestic_numbers","numbers_domestic"),
                 ("revenue_international_numbers","numbers_international")]:
    if col in num.columns:
        m = num["revenue_total_numbers"].isna() & num[col].notna()
        num.loc[m, "revenue_total_numbers"] = num.loc[m, col]
        src = np.where(m, tag, src)

num["revenue_total_numbers_source"] = src

print(f"🧾 Numbers rows: {len(num):,} | with any total: {num['revenue_total_numbers'].notna().sum():,}")

# ---- MERGE PASS 1: (primary_std2, year) -> suffix _num1 on Numbers cols ----
num_cols = ["revenue_domestic_numbers","revenue_international_numbers","revenue_worldwide_numbers",
            "revenue_total_numbers","revenue_total_numbers_source"]

m1 = df.merge(
    num[["title_numbers_std2","year_numbers"] + num_cols],
    left_on=["primary_std2","year"],
    right_on=["title_numbers_std2","year_numbers"],
    how="left",
    suffixes=("","_num1")
)

# After merge, Numbers cols got suffix _num1 (on the RIGHT side).
num1_cols = [c+"_num1" for c in num_cols]

# ---- MERGE PASS 2: (original_std2, year) -> rename to _alt then merge ----
num_alt = num[["title_numbers_std2","year_numbers"] + num_cols].rename(
    columns={c: c+"_alt" for c in num_cols}
)
m2 = m1.merge(
    num_alt,
    left_on=["original_std2","year"],
    right_on=["title_numbers_std2","year_numbers"],
    how="left"
)

# ---- Coalesce to canonical Numbers columns (no suffix) ----
def get_col(df, name):
    # prefer canonical if it already exists, else try _num1 then _alt
    for c in [name, name+"_num1", name+"_alt"]:
        if c in df.columns:
            return c
    return None

canonical = {}
for base in num_cols:
    c1 = base+"_num1"
    c2 = base+"_alt"
    # Start with an empty column
    m2[base] = np.nan
    if c1 in m2.columns:
        m2[base] = m2[base].combine_first(m2[c1])
    if c2 in m2.columns:
        m2[base] = m2[base].combine_first(m2[c2])

# Clean up suffix columns to keep dataframe tidy
drop_these = [c for c in m2.columns if c.endswith("_num1") or c.endswith("_alt")]
m2.drop(columns=drop_these, inplace=True, errors="ignore")

# ---- fill unified total only where missing ----
if "revenue_total_usd" not in m2.columns:
    m2["revenue_total_usd"] = np.nan
if "revenue_total_source" not in m2.columns:
    m2["revenue_total_source"] = pd.NA

before = m2["revenue_total_usd"].notna().sum()
mask_fill = m2["revenue_total_usd"].isna() & m2["revenue_total_numbers"].notna()
m2.loc[mask_fill, "revenue_total_usd"]    = m2.loc[mask_fill, "revenue_total_numbers"]
m2.loc[mask_fill, "revenue_total_source"] = m2.loc[mask_fill, "revenue_total_numbers_source"]
after = m2["revenue_total_usd"].notna().sum()

print(f"💰 Unified total filled from Numbers: +{after - before:,} rows (now {after:,}/{len(m2):,} = {after/len(m2):.1%})")

# ---- save ----
m2.to_csv(OUT, index=False, compression="gzip")
print(f"✅ Saved: {OUT}")

# peek a few matches
cols = ["primaryTitle","originalTitle","year",
        "revenue_worldwide_numbers","revenue_domestic_numbers","revenue_international_numbers",
        "revenue_total_numbers","revenue_total_numbers_source",
        "revenue_total_usd","revenue_total_source"]
print("\n🧪 Sample:")
print(m2[[c for c in cols if c in m2.columns]].head(12))

📂 Loaded master: horror_data_master_v6.csv.gz  (45,056 rows)
🧾 Numbers rows: 2,117 | with any total: 2,117
💰 Unified total filled from Numbers: +20 rows (now 7,014/45,058 = 15.6%)
✅ Saved: horror_data_master_v7.csv.gz

🧪 Sample:
                                         primaryTitle  \
0                               The Student of Prague   
1   The Avenging Conscience: or 'Thou Shalt Not Kill'   
2                                   The Ghost Breaker   
3                                           The Golem   
4                The Hound of the Baskervilles (1914)   
5                       The Hound of the Baskervilles   
6   Der Hund von Baskerville, 3. Teil - Das unheim...   
7                   Der Hund von Baskerville, 4. Teil   
8                                       Blind Justice   
9                                   Life Without Soul   
10                                           Mortmain   
11                                        After Death   

                             

In [ ]:
# how many matched at all vs actually filled
matched = m2["revenue_total_numbers"].notna().sum()
filled  = (m2["revenue_total_usd"].isna() & m2["revenue_total_numbers"].notna()).sum()
print(f"🎯 Titles matched in Numbers: {matched:,}")
print(f"💰 Titles newly filled (previously blank): {filled:,}")

🎯 Titles matched in Numbers: 1,423
💰 Titles newly filled (previously blank): 0


Try merging in more kaggle Data

In [ ]:
import pandas as pd
import numpy as np

# -------- CONFIG --------
MASTER_FILE = "horror_data_master_v6.csv.gz"  # or .csv; auto-detects compression
KAGGLE_FILE = "horror_movies 3.csv"

# -------- LOAD --------
comp = "gzip" if MASTER_FILE.endswith(".gz") else None
master = pd.read_csv(MASTER_FILE, compression=comp, low_memory=False)
kaggle = pd.read_csv(KAGGLE_FILE, low_memory=False)

print(f"📂 Loaded master: {len(master):,} rows")
print(f"📂 Loaded kaggle: {len(kaggle):,} rows")

# -------- CLEAN: years --------
master["startYear"] = pd.to_numeric(master["startYear"], errors="coerce").astype("Int64")
kaggle["kaggle_year"] = pd.to_datetime(kaggle["release_date"], errors="coerce").dt.year.astype("Int64")

# -------- CLEAN: numbers (treat 0 as missing) --------
for col in ["budget", "revenue"]:
    if col in kaggle.columns:
        # already numeric per your schema; still normalize zeros to NaN
        kaggle[col] = pd.to_numeric(kaggle[col], errors="coerce")
        kaggle.loc[kaggle[col] == 0, col] = np.nan

# -------- NORMALIZE TITLES --------
def norm_title(s: str) -> str:
    s = str(s).lower().strip()
    # light punctuation/spacing normalization
    s = (s.replace("’","'")
           .replace("–","-")
           .replace("—","-")
           .replace(":", "")
           .replace("-", " "))
    s = " ".join(s.split())
    return s

master["primary_norm"]  = master["primaryTitle"].map(norm_title)
master["original_norm"] = master.get("originalTitle", master["primaryTitle"]).map(norm_title)

kaggle["title_norm"]    = kaggle["title"].map(norm_title) if "title" in kaggle.columns else pd.Series("", index=kaggle.index)
kaggle["original_norm"] = kaggle["original_title"].map(norm_title) if "original_title" in kaggle.columns else pd.Series("", index=kaggle.index)

# -------- MATCH PASS 1: primaryTitle + year --------
m1 = master.merge(
    kaggle[["title_norm","kaggle_year","budget","revenue"]],
    left_on=["primary_norm","startYear"],
    right_on=["title_norm","kaggle_year"],
    how="inner",
    suffixes=("", "_k1")
)

# -------- MATCH PASS 2: originalTitle + year --------
m2 = master.merge(
    kaggle[["original_norm","kaggle_year","budget","revenue"]],
    left_on=["original_norm","startYear"],
    right_on=["original_norm","kaggle_year"],
    how="inner",
    suffixes=("", "_k2")
)

# Combine unique matches by tconst to avoid double counting
merged_all = pd.concat([m1, m2], ignore_index=True)
merged_all = merged_all.sort_values("tconst").drop_duplicates(subset=["tconst"])

# -------- COVERAGE / POTENTIAL FILLS --------
# Current missing flags in master
has_budget_col  = "budget_usd"         in master.columns
has_revenue_col = "revenue_total_usd"  in master.columns

budget_missing_mask  = (master["budget_usd"].isna()        if has_budget_col  else pd.Series(True, index=master.index))
revenue_missing_mask = (master["revenue_total_usd"].isna() if has_revenue_col else pd.Series(True, index=master.index))

miss_budget_by_t  = dict(zip(master["tconst"], budget_missing_mask))
miss_revenue_by_t = dict(zip(master["tconst"], revenue_missing_mask))

merged_all["could_fill_budget"]  = merged_all["tconst"].map(miss_budget_by_t)
merged_all["could_fill_revenue"] = merged_all["tconst"].map(miss_revenue_by_t)

overlap_total = len(merged_all)
kaggle_budget_present  = merged_all["budget"].notna().sum()  if "budget"  in merged_all.columns else 0
kaggle_revenue_present = merged_all["revenue"].notna().sum() if "revenue" in merged_all.columns else 0

pot_budget_fills  = merged_all.query("budget.notna() and could_fill_budget == True").shape[0]
pot_revenue_fills = merged_all.query("revenue.notna() and could_fill_revenue == True").shape[0]

print("\n🎯 Overlap & Contribution")
print(f"🔹 Titles matched (any title+year):        {overlap_total:,}")
print(f"💵 Kaggle rows with budget present:        {kaggle_budget_present:,}")
print(f"💰 Kaggle rows with revenue present:       {kaggle_revenue_present:,}")
print(f"➡️  Potential NEW budget fills for master:  {pot_budget_fills:,}")
print(f"➡️  Potential NEW revenue fills for master: {pot_revenue_fills:,}")

# -------- SAMPLES OF POTENTIAL FILLS --------
sample_budget  = merged_all.query("budget.notna() and could_fill_budget == True")[["tconst","primaryTitle","startYear","budget"]].head(10)
sample_revenue = merged_all.query("revenue.notna() and could_fill_revenue == True")[["tconst","primaryTitle","startYear","revenue"]].head(10)

print("\n🔍 Sample of potential budget fills:")
print(sample_budget if len(sample_budget) else "(none)")

print("\n🔍 Sample of potential revenue fills:")
print(sample_revenue if len(sample_revenue) else "(none)")

📂 Loaded master: 45,056 rows
📂 Loaded kaggle: 32,540 rows

🎯 Overlap & Contribution
🔹 Titles matched (any title+year):        13,836
💵 Kaggle rows with budget present:        2,619
💰 Kaggle rows with revenue present:       1,164
➡️  Potential NEW budget fills for master:  2,615
➡️  Potential NEW revenue fills for master: 128

🔍 Sample of potential budget fills:
          tconst                                      primaryTitle  startYear  \
13345  tt0043778                             The Man from Planet X       1951   
6      tt0044344                                           Alraune       1952   
13355  tt0045283                                The White Reindeer       1952   
12     tt0045469  Abbott and Costello Meet Dr. Jekyll and Mr. Hyde       1953   
13     tt0045546                     The Beast from 20,000 Fathoms       1953   
13360  tt0045888                                      House of Wax       1953   
17     tt0045917                                Invaders from Mars   

In [ ]:
# =========================================================
# Merge Kaggle financials into master (non-destructive)
# Output: horror_data_master_v8.csv.gz
# =========================================================

import pandas as pd, numpy as np, os, glob

# ---- CONFIG ----
KAGGLE_FILE = "horror_movies 3.csv"
MASTER_CANDIDATES = [
    "horror_data_master_v7.csv.gz",  # prefer if you created a v7 from The Numbers
    "horror_data_master_v6.csv.gz",
    "horror_data_master_v7.csv",
    "horror_data_master_v6.csv",
]
OUT = "horror_data_master_v8.csv.gz"

# ---- LOAD MASTER (pick first available) ----
master_path = next((p for p in MASTER_CANDIDATES if os.path.exists(p)),
                   sorted(glob.glob("horror_data_master_v*.csv*"))[-1]
                   if glob.glob("horror_data_master_v*.csv*") else None)
if not master_path:
    raise FileNotFoundError("No master file found (expected v6/v7).")

comp = "gzip" if master_path.endswith(".gz") else None
df = pd.read_csv(master_path, compression=comp, low_memory=False)
print(f"📂 Loaded master: {master_path}  ({len(df):,} rows)")

# Ensure expected columns exist
if "tconst" not in df.columns or "primaryTitle" not in df.columns:
    raise ValueError("Master must include at least 'tconst' and 'primaryTitle'.")

# ---- LOAD KAGGLE ----
kg = pd.read_csv(KAGGLE_FILE, low_memory=False)
print(f"📥 Loaded Kaggle: {len(kg):,} rows")

# ---- CLEAN KAGGLE: year + numeric ----
kg["kaggle_year"] = pd.to_datetime(kg.get("release_date"), errors="coerce").dt.year.astype("Int64")

for col in ["budget", "revenue"]:
    if col in kg.columns:
        kg[col] = pd.to_numeric(kg[col], errors="coerce")
        kg.loc[kg[col] == 0, col] = np.nan  # treat 0 as missing

# ---- NORMALIZE TITLES (light but robust) ----
def norm_title(s: str) -> str:
    s = str(s).lower().strip()
    s = (s
         .replace("’","'")
         .replace("–","-")
         .replace("—","-")
         .replace(":", "")
         .replace("-", " "))
    s = " ".join(s.split())
    return s

df["startYear"] = pd.to_numeric(df.get("startYear"), errors="coerce").astype("Int64")
df["primary_norm"]  = df["primaryTitle"].map(norm_title)
df["original_norm"] = df.get("originalTitle", df["primaryTitle"]).map(norm_title)

kg["title_norm"]    = kg.get("title", "").map(norm_title) if "title" in kg.columns else pd.Series("", index=kg.index)
kg["original_norm"] = kg.get("original_title", "").map(norm_title) if "original_title" in kg.columns else pd.Series("", index=kg.index)

# ---- PASS 1: (primaryTitle, startYear) -> (title, kaggle_year) ----
m1 = df.merge(
    kg[["title_norm", "kaggle_year", "budget", "revenue"]],
    left_on=["primary_norm", "startYear"],
    right_on=["title_norm",   "kaggle_year"],
    how="left",
    suffixes=("", "_k1")
)
m1.rename(columns={"budget": "budget_kaggle_p1", "revenue": "revenue_kaggle_p1"}, inplace=True)

# ---- PASS 2: (originalTitle, startYear) -> (original_title, kaggle_year) ----
m2 = m1.merge(
    kg[["original_norm", "kaggle_year", "budget", "revenue"]].rename(
        columns={"budget":"budget_kaggle_p2", "revenue":"revenue_kaggle_p2"}
    ),
    left_on=["original_norm", "startYear"],
    right_on=["original_norm", "kaggle_year"],
    how="left"
)

# ---- COALESCE to single Kaggle columns ----
m2["budget_kaggle"]  = m2["budget_kaggle_p1"].combine_first(m2["budget_kaggle_p2"])
m2["revenue_kaggle"] = m2["revenue_kaggle_p1"].combine_first(m2["revenue_kaggle_p2"])

# cleanup helper cols
m2.drop(columns=[c for c in m2.columns if c.endswith("_k1") or c.endswith("_k2") or c in ["title_norm","kaggle_year"]],
        inplace=True, errors="ignore")

# ---- BASELINE COVERAGE BEFORE FILL ----
if "budget_usd" not in m2.columns:       m2["budget_usd"] = np.nan
if "revenue_total_usd" not in m2.columns: m2["revenue_total_usd"] = np.nan
if "budget_source" not in m2.columns:    m2["budget_source"] = pd.NA
if "revenue_total_source" not in m2.columns: m2["revenue_total_source"] = pd.NA

before_budget  = m2["budget_usd"].notna().sum()
before_revenue = m2["revenue_total_usd"].notna().sum()

# ---- FILL UNIFIED COLUMNS ONLY WHERE MISSING ----
mask_budget_fill  = m2["budget_usd"].isna()         & m2["budget_kaggle"].notna()
mask_revenue_fill = m2["revenue_total_usd"].isna()  & m2["revenue_kaggle"].notna()

m2.loc[mask_budget_fill,  "budget_usd"]          = m2.loc[mask_budget_fill,  "budget_kaggle"]
m2.loc[mask_budget_fill,  "budget_source"]       = "kaggle"

m2.loc[mask_revenue_fill, "revenue_total_usd"]   = m2.loc[mask_revenue_fill, "revenue_kaggle"]
m2.loc[mask_revenue_fill, "revenue_total_source"]= "kaggle"

after_budget  = m2["budget_usd"].notna().sum()
after_revenue = m2["revenue_total_usd"].notna().sum()

print(f"\n✅ Kaggle merge complete")
print(f"   💵 Budget newly filled:   +{after_budget - before_budget:,} (now {after_budget:,}/{len(m2):,} = {after_budget/len(m2):.1%})")
print(f"   💰 Revenue newly filled:  +{after_revenue - before_revenue:,} (now {after_revenue:,}/{len(m2):,} = {after_revenue/len(m2):.1%})")

# ---- SAVE ----
m2.to_csv(OUT, index=False, compression="gzip")
print(f"\n💾 Saved: {OUT}")

# ---- QUICK PEEKS ----
print("\n🔍 Sample of newly filled budgets:")
print(m2.loc[mask_budget_fill, ["tconst","primaryTitle","startYear","budget_kaggle","budget_usd","budget_source"]].head(10))

print("\n🔍 Sample of newly filled revenues:")
print(m2.loc[mask_revenue_fill, ["tconst","primaryTitle","startYear","revenue_kaggle","revenue_total_usd","revenue_total_source"]].head(10))

📂 Loaded master: horror_data_master_v7.csv.gz  (45,058 rows)
📥 Loaded Kaggle: 32,540 rows

✅ Kaggle merge complete
   💵 Budget newly filled:   +2,832 (now 5,780/45,810 = 12.6%)
   💰 Revenue newly filled:  +139 (now 7,326/45,810 = 16.0%)

💾 Saved: horror_data_master_v8.csv.gz

🔍 Sample of newly filled budgets:
        tconst                                      primaryTitle  startYear  \
409  tt0043778                             The Man from Planet X       1951   
413  tt0044344                                           Alraune       1952   
419  tt0045283                                The White Reindeer       1952   
420  tt0045469  Abbott and Costello Meet Dr. Jekyll and Mr. Hyde       1953   
421  tt0045546                     The Beast from 20,000 Fathoms       1953   
425  tt0045888                                      House of Wax       1953   
426  tt0045917                                Invaders from Mars       1953   
427  tt0045920                          It Came from Oute

Sanity check with dashboard

In [ ]:
import pandas as pd
import numpy as np

MASTER = "horror_data_master_v8.csv.gz"
comp = "gzip" if MASTER.endswith(".gz") else None
df = pd.read_csv(MASTER, compression=comp, low_memory=False)

def cov(col):
    n = df[col].notna().sum() if col in df.columns else 0
    return n, len(df), round(100*n/len(df), 1)

cols = [
    "budget_usd", "budget_source",
    "revenue_total_usd", "revenue_total_source",
    "revenue_worldwide_numbers", "revenue_domestic_numbers", "revenue_international_numbers",
    "revenue_tmdb", "budget_tmdb",
    "release_date_final", "release_date_tmdb", "release_date_omdb",
    "averageRating", "imdb_rating_omdb", "numVotes", "imdb_votes_omdb",
]

summary = []
for c in cols:
    if c in df.columns:
        filled, total, pct = cov(c)
        summary.append({"column": c, "filled": filled, "total": total, "pct": pct})

cov_df = pd.DataFrame(summary).sort_values("pct", ascending=False)
print(cov_df.to_string(index=False))

                       column  filled  total  pct
           release_date_final   32061  45810 70.0
            release_date_omdb   28872  45810 63.0
                averageRating   26646  45810 58.2
                     numVotes   26646  45810 58.2
              imdb_votes_omdb   24475  45810 53.4
             imdb_rating_omdb   22386  45810 48.9
            release_date_tmdb   19245  45810 42.0
         revenue_total_source    7326  45810 16.0
            revenue_total_usd    7326  45810 16.0
                budget_source    5780  45810 12.6
                   budget_usd    5780  45810 12.6
                 revenue_tmdb    2948  45810  6.4
                  budget_tmdb    2948  45810  6.4
    revenue_worldwide_numbers    1459  45810  3.2
revenue_international_numbers    1459  45810  3.2
     revenue_domestic_numbers     535  45810  1.2


In [ ]:
import numpy as np

# suspiciously low budgets (possible data-entry artifacts)
low_budgets = df.loc[(df["budget_usd"].notna()) & (df["budget_usd"] > 0) & (df["budget_usd"] < 200),
                     ["tconst","primaryTitle","startYear","budget_usd","budget_source"]].head(20)
print("\n⚠️ Very low budgets (<$200):")
print(low_budgets if not low_budgets.empty else "(none)")

# where we just filled from Kaggle (spot-check)
kaggle_fills = df.loc[df["budget_source"].eq("kaggle"),
                      ["tconst","primaryTitle","startYear","budget_usd"]].head(20)
print("\n🔍 Sample Kaggle-filled budgets:")
print(kaggle_fills if not kaggle_fills.empty else "(none)")

# check for places with both TMDb & Numbers totals available (possible reconciliation later)
has_tmdb  = df.get("revenue_tmdb").notna() if "revenue_tmdb" in df.columns else pd.Series(False, index=df.index)
has_nums  = df.get("revenue_worldwide_numbers").notna() if "revenue_worldwide_numbers" in df.columns else pd.Series(False, index=df.index)
both = df.loc[has_tmdb & has_nums, ["tconst","primaryTitle","startYear","revenue_tmdb","revenue_worldwide_numbers"]].head(20)
print("\n🔁 Titles with both TMDb and Numbers totals (for QA):")
print(both if not both.empty else "(none)")


⚠️ Very low budgets (<$200):
           tconst                             primaryTitle  startYear  \
413     tt0044344                                  Alraune     1952.0   
1550    tt0070795                     A Thief in the Night     1972.0   
2095    tt0081186                             Mother's Day     1980.0   
2536    tt0090093            Thou Shalt Not Kill... Except     1985.0   
3857    tt0126714                  Sorority House Vampires     1998.0   
4121    tt0156413                    The Clown at Midnight     1998.0   
4326    tt0181466                                Demagogue     1998.0   
5386    tt0287072                       Under Surveillance     2006.0   
5835    tt0339869                         Voices of Desire     1972.0   
6028    tt0361953             The Nest of the Cuckoo Birds     1965.0   
6525    tt0419098            Trees 2: The Root of All Evil     2004.0   
7119    tt0477430                              White Panic     2005.0   
7197    tt0484873    

Checking to see if weird values are errors or not:

In [ ]:
# ===========================================
# DRY-RUN: Budget QC diagnostics (no edits / no save)
# Checks tiny Kaggle budgets and proposes safe fixes
# ===========================================

import pandas as pd, numpy as np, os

MASTER = "horror_data_master_v8.csv.gz"
comp = "gzip" if MASTER.endswith(".gz") else None
df = pd.read_csv(MASTER, compression=comp, low_memory=False)
print(f"📂 Loaded: {MASTER} ({len(df):,} rows)")

# Ensure columns exist
for col in ["budget_usd","budget_source","budget_tmdb","budget_kaggle","revenue_total_usd","runtimeMinutes","startYear"]:
    if col not in df.columns:
        df[col] = np.nan

# -------- Define “tiny budget” and heuristics (same as before, but DRY-RUN) --------
tiny_mask = (df["budget_usd"].notna()) & (df["budget_usd"] > 0) & (df["budget_usd"] < 250) & (df["budget_source"].eq("kaggle"))

prefer_tmdb_mask = tiny_mask & df["budget_tmdb"].notna() & (df["budget_tmdb"] >= 1_000)

feature_like = ((pd.to_numeric(df["runtimeMinutes"], errors="coerce") >= 60) |
                (pd.to_numeric(df["startYear"], errors="coerce") >= 1940))
made_money   = (pd.to_numeric(df["revenue_total_usd"], errors="coerce") >= 100_000)

scale_mask = tiny_mask & ~prefer_tmdb_mask & feature_like & made_money & df["budget_kaggle"].notna()

remaining_tiny = tiny_mask & ~prefer_tmdb_mask & ~scale_mask

# -------- Build a proposed changes table (no edits applied) --------
proposed = []

# Proposal A: replace with TMDb budget
if prefer_tmdb_mask.any():
    tmp = df.loc[prefer_tmdb_mask, ["tconst","primaryTitle","startYear","budget_usd","budget_tmdb","budget_source","revenue_total_usd","runtimeMinutes"]].copy()
    tmp["action"] = "replace_with_tmdb"
    tmp["proposed_budget_usd"] = tmp["budget_tmdb"]
    tmp["reason"] = "tiny_kaggle_budget; tmdb>=1000"
    proposed.append(tmp)

# Proposal B: scale Kaggle by 1000
if scale_mask.any():
    tmp = df.loc[scale_mask, ["tconst","primaryTitle","startYear","budget_usd","budget_kaggle","budget_source","revenue_total_usd","runtimeMinutes"]].copy()
    tmp["action"] = "scale_kaggle_x1000"
    tmp["proposed_budget_usd"] = tmp["budget_kaggle"] * 1000
    tmp["reason"] = "tiny_kaggle_budget; feature_like & revenue>=100k"
    proposed.append(tmp)

proposed_df = pd.concat(proposed, ignore_index=True) if proposed else pd.DataFrame(columns=[
    "tconst","primaryTitle","startYear","budget_usd","budget_tmdb","budget_kaggle","budget_source",
    "revenue_total_usd","runtimeMinutes","action","proposed_budget_usd","reason"
])

# -------- Reports --------
n_tiny        = int(tiny_mask.sum())
n_tmdb_rep    = int(prefer_tmdb_mask.sum())
n_scale       = int(scale_mask.sum())
n_flag_only   = int(remaining_tiny.sum())

print("\n🧪 DRY-RUN SUMMARY (no changes made)")
print(f"   • Tiny Kaggle budgets detected (<$250):   {n_tiny:,}")
print(f"   • Would replace with TMDb (>=$1k):        {n_tmdb_rep:,}")
print(f"   • Would scale Kaggle x1000 (heuristic):    {n_scale:,}")
print(f"   • Would leave & flag as weird:             {n_flag_only:,}")

# Show samples
print("\n🔍 Sample — would replace with TMDb:")
cols_a = ["tconst","primaryTitle","startYear","budget_usd","budget_tmdb","revenue_total_usd","runtimeMinutes","reason"]
print(proposed_df.loc[proposed_df["action"]=="replace_with_tmdb", cols_a].head(10) if n_tmdb_rep else "(none)")

print("\n🔍 Sample — would scale Kaggle x1000:")
cols_b = ["tconst","primaryTitle","startYear","budget_usd","budget_kaggle","proposed_budget_usd","revenue_total_usd","runtimeMinutes","reason"]
print(proposed_df.loc[proposed_df["action"]=="scale_kaggle_x1000", cols_b].head(10) if n_scale else "(none)")

print("\n🔍 Sample — remaining tiny (left as-is, would flag):")
cols_c = ["tconst","primaryTitle","startYear","budget_usd","budget_source","revenue_total_usd","runtimeMinutes"]
print(df.loc[remaining_tiny, cols_c].head(10) if n_flag_only else "(none)")

# Optional: inspect overall budget distribution (quick quantiles)
if df["budget_usd"].notna().any():
    print("\n📈 Budget distribution (USD) — quick quantiles:")
    print(df["budget_usd"].describe(percentiles=[.01,.05,.1,.25,.5,.75,.9,.95,.99]))

📂 Loaded: horror_data_master_v8.csv.gz (45,810 rows)

🧪 DRY-RUN SUMMARY (no changes made)
   • Tiny Kaggle budgets detected (<$250):   131
   • Would replace with TMDb (>=$1k):        0
   • Would scale Kaggle x1000 (heuristic):    5
   • Would leave & flag as weird:             126

🔍 Sample — would replace with TMDb:
(none)

🔍 Sample — would scale Kaggle x1000:
       tconst                primaryTitle  startYear  budget_usd  \
0   tt1090634              The 13th Alley     2008.0       150.0   
1   tt1093369                        Hush     2008.0         1.0   
2  tt22042742  Christmas Bloody Christmas     2022.0         2.0   
3   tt6793180                    Deccal 2     2017.0         2.0   
4   tt7235038                        Boo!     2018.0       240.0   

   budget_kaggle  proposed_budget_usd  revenue_total_usd  runtimeMinutes  \
0          150.0             150000.0           250992.0            81.0   
1            1.0               1000.0           288667.0            91.0 

In [ ]:
import pandas as pd

MASTER = "horror_data_master_v8.csv.gz"
df = pd.read_csv(MASTER, compression="gzip", low_memory=False)

# sanity: columns present?
expected = ["budget_kaggle","revenue_kaggle","budget_usd","revenue_total_usd","budget_source","revenue_total_source"]
print("✅ Columns present:", [c for c in expected if c in df.columns])

# coverage counts
kaggle_budget = (df.get("budget_source") == "kaggle").sum()
kaggle_rev    = (df.get("revenue_total_source") == "kaggle").sum()
print(f"📊 budget_source=='kaggle': {kaggle_budget:,}")
print(f"📊 revenue_total_source=='kaggle': {kaggle_rev:,}")

# quick peek of kaggle-filled rows
print("\n🔍 Sample Kaggle budgets:")
print(df.loc[df.get("budget_source").eq("kaggle"), ["tconst","primaryTitle","startYear","budget_kaggle","budget_usd","budget_source"]].head(10))

print("\n🔍 Sample Kaggle revenues:")
print(df.loc[df.get("revenue_total_source").eq("kaggle"), ["tconst","primaryTitle","startYear","revenue_kaggle","revenue_total_usd","revenue_total_source"]].head(10))

✅ Columns present: ['budget_kaggle', 'revenue_kaggle', 'budget_usd', 'revenue_total_usd', 'budget_source', 'revenue_total_source']
📊 budget_source=='kaggle': 2,832
📊 revenue_total_source=='kaggle': 139

🔍 Sample Kaggle budgets:
        tconst                                      primaryTitle  startYear  \
409  tt0043778                             The Man from Planet X     1951.0   
413  tt0044344                                           Alraune     1952.0   
419  tt0045283                                The White Reindeer     1952.0   
420  tt0045469  Abbott and Costello Meet Dr. Jekyll and Mr. Hyde     1953.0   
421  tt0045546                     The Beast from 20,000 Fathoms     1953.0   
425  tt0045888                                      House of Wax     1953.0   
426  tt0045917                                Invaders from Mars     1953.0   
427  tt0045920                          It Came from Outer Space     1953.0   
434  tt0046248                                     Robot Mons

Starting to get final columns:

In [ ]:
# ===========================================
# SAFE COLLAPSE: build final columns without overwriting or saving
# Input : horror_data_master_v8.csv.gz
# Output: in-memory df with new *_final and *_final_source columns
# ===========================================

import pandas as pd, numpy as np, os

MASTER = "horror_data_master_v8.csv.gz"
SAVE   = False  # set to True to write out a v9 after you review
OUT    = "horror_data_master_v9_collapsed.csv.gz"

comp = "gzip" if MASTER.endswith(".gz") else None
df = pd.read_csv(MASTER, compression=comp, low_memory=False)
n = len(df)
print(f"📂 Loaded {MASTER} ({n:,} rows)")

# ---------- helpers ----------
def cov(series):
    s = series.notna().sum()
    return s, n, round(100*s/n, 1)

def first_nonnull(*cols):
    """Vectorized first non-null across multiple Series of equal length."""
    out = None
    for c in cols:
        if c is None:
            continue
        if out is None:
            out = c
        else:
            out = out.combine_first(c)
    return out

def year_of(s):
    dt = pd.to_datetime(s, errors="coerce")
    return dt.dt.year.astype("Int64")

# ---------- BEFORE snapshot (key columns we’ll collapse) ----------
pre_cols = [
    "startYear","release_date_final","release_date_tmdb","release_date_omdb",
    "runtimeMinutes","runtime_omdb",
    "language_best","language_omdb_code","original_language",
    "region_best","country_omdb_code",
    "budget_usd","budget_tmdb","budget_kaggle","budget_source",
    "revenue_total_usd","revenue_total_source",
    "revenue_worldwide_numbers","revenue_domestic_numbers","revenue_international_numbers",
    "revenue_domestic_omdb",
    "rated","rated_omdb",
]
print("\n🔎 Availability before collapse:")
for c in pre_cols:
    if c in df.columns:
        filled, total, pct = cov(df[c])
        print(f"{c:>28}  {filled:6}/{total:<6} ({pct:>4}%)")

# ---------- FINAL: Title ----------
# Keep title as-is; add a convenience alias
df["title_final"] = df.get("primaryTitle")

# ---------- FINAL: Year (Int64) + source ----------
# Order: explicit startYear -> year(release_date_final) -> year(TMDb) -> year(OMDb)
df["startYear"] = pd.to_numeric(df.get("startYear"), errors="coerce").astype("Int64")
y1 = df["startYear"]
y2 = year_of(df.get("release_date_final"))
y3 = year_of(df.get("release_date_tmdb"))
y4 = year_of(df.get("release_date_omdb"))
df["year_final"] = first_nonnull(y1, y2, y3, y4)

# source
src_year = pd.Series(pd.NA, index=df.index, dtype="object")
src_year = np.where(y1.notna(), "startYear", src_year)
src_year = np.where((pd.isna(src_year)) & y2.notna(), "release_date_final", src_year)
src_year = np.where((pd.isna(src_year)) & y3.notna(), "release_date_tmdb", src_year)
src_year = np.where((pd.isna(src_year)) & y4.notna(), "release_date_omdb", src_year)
df["year_final_source"] = src_year

# ---------- FINAL: Release date + source ----------
# Keep your existing release_date_final if present, else fallback TMDb, then OMDb
d1 = pd.to_datetime(df.get("release_date_final"), errors="coerce")
d2 = pd.to_datetime(df.get("release_date_tmdb"), errors="coerce")
d3 = pd.to_datetime(df.get("release_date_omdb"), errors="coerce")
df["release_date_final2"] = first_nonnull(d1, d2, d3)

src_date = pd.Series(pd.NA, index=df.index, dtype="object")
src_date = np.where(d1.notna(), "final_existing", src_date)
src_date = np.where((pd.isna(src_date)) & d2.notna(), "tmdb", src_date)
src_date = np.where((pd.isna(src_date)) & d3.notna(), "omdb", src_date)
df["release_date_final_source"] = src_date

# ---------- FINAL: Runtime + source ----------
rt1 = pd.to_numeric(df.get("runtimeMinutes"), errors="coerce")
rt2 = pd.to_numeric(df.get("runtime_omdb"), errors="coerce")
df["runtime_final"] = first_nonnull(rt1, rt2)

src_rt = pd.Series(pd.NA, index=df.index, dtype="object")
src_rt = np.where(rt1.notna(), "imdb_runtimeMinutes", src_rt)
src_rt = np.where((pd.isna(src_rt)) & rt2.notna(), "omdb", src_rt)
df["runtime_final_source"] = src_rt

# ---------- FINAL: Language (2-letter codes) + source ----------
# We’ll prefer two-letter codes from: language_best -> language_omdb_code -> original_language (if looks like 2-letter)
lang1 = df.get("language_best")
lang2 = df.get("language_omdb_code")
lang3 = df.get("original_language")  # often from TMDb/Kaggle

def iso2(series):
    s = series.astype("string")
    s = s.str.strip().str.lower()
    # be conservative: accept only 2-letter a-z
    s = s.where(s.str.fullmatch(r"[a-z]{2}"), pd.NA)
    return s

l1 = iso2(lang1) if lang1 is not None else pd.Series(pd.NA, index=df.index, dtype="string")
l2 = iso2(lang2) if lang2 is not None else pd.Series(pd.NA, index=df.index, dtype="string")
l3 = iso2(lang3) if lang3 is not None else pd.Series(pd.NA, index=df.index, dtype="string")

df["language_final"] = first_nonnull(l1, l2, l3)

src_lang = pd.Series(pd.NA, index=df.index, dtype="object")
src_lang = np.where(l1.notna(), "imdb_language_best", src_lang)
src_lang = np.where((pd.isna(src_lang)) & l2.notna(), "omdb_language_code", src_lang)
src_lang = np.where((pd.isna(src_lang)) & l3.notna(), "tmdb_original_language", src_lang)
df["language_final_source"] = src_lang

# ---------- FINAL: Region/Country (2-letter) + source ----------
reg1 = df.get("region_best")
reg2 = df.get("country_omdb_code")

def iso2_region(series):
    s = series.astype("string")
    s = s.str.strip().str.upper()
    s = s.where(s.str.fullmatch(r"[A-Z]{2}"), pd.NA)
    return s

r1 = iso2_region(reg1) if reg1 is not None else pd.Series(pd.NA, index=df.index, dtype="string")
r2 = iso2_region(reg2) if reg2 is not None else pd.Series(pd.NA, index=df.index, dtype="string")

df["region_final"] = first_nonnull(r1, r2)

src_reg = pd.Series(pd.NA, index=df.index, dtype="object")
src_reg = np.where(r1.notna(), "imdb_region_best", src_reg)
src_reg = np.where((pd.isna(src_reg)) & r2.notna(), "omdb_country_code", src_reg)
df["region_final_source"] = src_reg

# ---------- FINAL: Ratings + votes (IMDb) ----------
# averageRating/numVotes (IMDb) fallback to OMDb copies if IMDb missing
ar1 = pd.to_numeric(df.get("averageRating"), errors="coerce")
ar2 = pd.to_numeric(df.get("imdb_rating_omdb"), errors="coerce")
nv1 = pd.to_numeric(df.get("numVotes"), errors="coerce")
nv2 = pd.to_numeric(df.get("imdb_votes_omdb"), errors="coerce")

df["rating_imdb_final"] = first_nonnull(ar1, ar2)
df["votes_imdb_final"]  = first_nonnull(nv1, nv2)

src_rating = pd.Series(pd.NA, index=df.index, dtype="object")
src_rating = np.where(ar1.notna(), "imdb_title_ratings", src_rating)
src_rating = np.where((pd.isna(src_rating)) & ar2.notna(), "omdb_imdb_rating", src_rating)
df["rating_imdb_final_source"] = src_rating

src_votes = pd.Series(pd.NA, index=df.index, dtype="object")
src_votes = np.where(nv1.notna(), "imdb_title_ratings", src_votes)
src_votes = np.where((pd.isna(src_votes)) & nv2.notna(), "omdb_imdb_votes", src_votes)
df["votes_imdb_final_source"] = src_votes

# ---------- FINAL: Rated (MPAA) ----------
rated1 = df.get("rated")
rated2 = df.get("rated_omdb")
r1 = rated1.astype("string") if rated1 is not None else pd.Series(pd.NA, index=df.index, dtype="string")
r2 = rated2.astype("string") if rated2 is not None else pd.Series(pd.NA, index=df.index, dtype="string")
df["rated_final"] = first_nonnull(r1, r2)

src_rated = pd.Series(pd.NA, index=df.index, dtype="object")
src_rated = np.where(r1.notna(), "imdb_rated", src_rated)
src_rated = np.where((pd.isna(src_rated)) & r2.notna(), "omdb_rated", src_rated)
df["rated_final_source"] = src_rated

# ---------- FINAL: Budget & Revenue (use your unified columns) ----------
# You already unified to budget_usd / revenue_total_usd with sources; surface them as *_final.
df["budget_final"]         = df.get("budget_usd")
df["budget_final_source"]  = df.get("budget_source")
df["revenue_total_final"]  = df.get("revenue_total_usd")
df["revenue_total_final_source"] = df.get("revenue_total_source")

# Also surface best available splits if present (Numbers preferred)
df["revenue_domestic_final"]      = df.get("revenue_domestic_numbers").combine_first(df.get("revenue_domestic_omdb"))
df["revenue_international_final"] = df.get("revenue_international_numbers")
df["revenue_worldwide_final"]     = df.get("revenue_worldwide_numbers").combine_first(df.get("revenue_total_usd"))

# ---------- AFTER snapshot ----------
final_cols = [
    "title_final",
    "year_final","year_final_source",
    "release_date_final2","release_date_final_source",
    "runtime_final","runtime_final_source",
    "language_final","language_final_source",
    "region_final","region_final_source",
    "rated_final","rated_final_source",
    "budget_final","budget_final_source",
    "revenue_domestic_final","revenue_international_final","revenue_worldwide_final",
    "revenue_total_final","revenue_total_final_source",
    "rating_imdb_final","rating_imdb_final_source",
    "votes_imdb_final","votes_imdb_final_source",
]
print("\n✅ Coverage after collapse (new *_final columns):")
for c in final_cols:
    if c in df.columns and not c.endswith("_source"):
        filled, total, pct = cov(df[c])
        print(f"{c:>28}  {filled:6}/{total:<6} ({pct:>4}%)")

# peek a few rows
print("\n🧪 Quick peek:")
show = [c for c in final_cols if c in df.columns]
print(df[show].head(8))

# ---------- SAVE (optional) ----------
if SAVE:
    df.to_csv(OUT, index=False, compression="gzip")
    print(f"\n💾 Saved collapsed view to: {OUT}")
else:
    print("\n💡 Not saved (SAVE=False). If this looks good, set SAVE=True to write v9.")

📂 Loaded horror_data_master_v8.csv.gz (45,810 rows)

🔎 Availability before collapse:
                   startYear   33355/45810  (72.8%)
          release_date_final   32061/45810  (70.0%)
           release_date_tmdb   19245/45810  (42.0%)
           release_date_omdb   28872/45810  (63.0%)
              runtimeMinutes   29270/45810  (63.9%)
                runtime_omdb   27514/45810  (60.1%)
               language_best   18312/45810  (40.0%)
          language_omdb_code   31430/45810  (68.6%)
                 region_best   18312/45810  (40.0%)
           country_omdb_code   32079/45810  (70.0%)
                  budget_usd    5780/45810  (12.6%)
                 budget_tmdb    2948/45810  ( 6.4%)
               budget_kaggle    2836/45810  ( 6.2%)
               budget_source    5780/45810  (12.6%)
           revenue_total_usd    7326/45810  (16.0%)
        revenue_total_source    7326/45810  (16.0%)
   revenue_worldwide_numbers    1459/45810  ( 3.2%)
    revenue_domestic_numbers   

In [ ]:
# Use the in-memory df created by the SAFE COLLAPSE cell
if "df" not in globals():
    raise RuntimeError("No in-memory DataFrame named `df`. Re-run the SAFE COLLAPSE cell first.")

final_cols = [
    "title_final","year_final","release_date_final2","runtime_final",
    "language_final","region_final","rated_final",
    "budget_final","revenue_domestic_final","revenue_international_final",
    "revenue_worldwide_final","revenue_total_final",
    "rating_imdb_final","votes_imdb_final"
]

def coverage(series):
    total = len(series)
    filled = series.notna().sum()
    return f"{filled}/{total} ({filled/total*100:.1f}%)"

print("📊 Final Column Coverage Summary:\n")
for col in final_cols:
    if col in df.columns:
        print(f"{col:>28}: {coverage(df[col])}")
    else:
        print(f"{col:>28}: (missing in df)")

📊 Final Column Coverage Summary:

                 title_final: (missing in df)
                  year_final: (missing in df)
         release_date_final2: (missing in df)
               runtime_final: (missing in df)
              language_final: (missing in df)
                region_final: (missing in df)
                 rated_final: (missing in df)
                budget_final: (missing in df)
      revenue_domestic_final: (missing in df)
 revenue_international_final: (missing in df)
     revenue_worldwide_final: (missing in df)
         revenue_total_final: (missing in df)
           rating_imdb_final: (missing in df)
            votes_imdb_final: (missing in df)


In [ ]:
import pandas as pd

# Load your in-memory df if it's still there, or reload from the v8 file if you restarted
#df = pd.read_csv("horror_data_master_v8.csv.gz", compression="gzip", low_memory=False)

# These are the unified columns from the collapse cell
final_cols = [
    "title_final",
    "year_final",
    "release_date_final2",
    "runtime_final",
    "language_final",
    "region_final",
    "rated_final",
    "budget_final",
    "revenue_domestic_final",
    "revenue_international_final",
    "revenue_worldwide_final",
    "revenue_total_final",
    "rating_imdb_final",
    "votes_imdb_final"
]

def coverage(series):
    total = len(series)
    filled = series.notna().sum()
    pct = 100 * filled / total
    return f"{filled}/{total} ({pct:.1f}%)"

print("📊 Final Column Coverage Summary:\n")
for col in final_cols:
    if col in df.columns:
        print(f"{col:>28}: {coverage(df[col])}")
    else:
        print(f"{col:>28}: (missing)")

📊 Final Column Coverage Summary:

                 title_final: (missing)
                  year_final: (missing)
         release_date_final2: (missing)
               runtime_final: (missing)
              language_final: (missing)
                region_final: (missing)
                 rated_final: (missing)
                budget_final: (missing)
      revenue_domestic_final: (missing)
 revenue_international_final: (missing)
     revenue_worldwide_final: (missing)
         revenue_total_final: (missing)
           rating_imdb_final: (missing)
            votes_imdb_final: (missing)


In [ ]:
# ===========================================
# UNIVERSAL SAFE COLLAPSE (all categories)
# - Non-destructive: only creates *_final + *_final_source
# - Prints before → after coverage (with deltas)
# - Does NOT save unless SAVE=True
# ===========================================

import pandas as pd, numpy as np

MASTER = "horror_data_master_v8.csv.gz"  # change if you want the .csv instead
SAVE   = True                           # set True to write out after review
OUT    = "horror_data_master_v9_collapsed.csv.gz"

# ---------- load ----------
comp = "gzip" if MASTER.endswith(".gz") else None
df = pd.read_csv(MASTER, compression=comp, low_memory=False)
n = len(df)
print(f"📂 Loaded {MASTER} ({n:,} rows)")

# ---------- helpers ----------
def first_nonnull(*cols):
    """Vectorized 'coalesce' across equal-length Series (None allowed)."""
    out = None
    for s in cols:
        if s is None:
            continue
        if out is None:
            out = s
        else:
            out = out.combine_first(s)
    return out

def year_of(s):
    dt = pd.to_datetime(s, errors="coerce")
    return dt.dt.year.astype("Int64")

def iso2_lang(s):
    if s is None:
        return pd.Series(pd.NA, index=df.index, dtype="string")
    s = s.astype("string").str.strip().str.lower()
    return s.where(s.str.fullmatch(r"[a-z]{2}"), pd.NA)

def iso2_region(s):
    if s is None:
        return pd.Series(pd.NA, index=df.index, dtype="string")
    s = s.astype("string").str.strip().str.upper()
    return s.where(s.str.fullmatch(r"[A-Z]{2}"), pd.NA)

def num(s):
    return pd.to_numeric(s, errors="coerce")

def coverage(series):  # returns (filled_count, pct)
    filled = series.notna().sum()
    pct = round(100*filled/len(series), 1)
    return filled, pct

def report_delta(name, before_series, after_series):
    b_cnt, b_pct = coverage(before_series)
    a_cnt, a_pct = coverage(after_series)
    delta = a_cnt - b_cnt
    print(f"{name:>28}: {b_cnt:6}/{n:<6} ({b_pct:>4}%) → {a_cnt:6}/{n:<6} ({a_pct:>4}%)   Δ +{delta:,}")

def choose_with_source(target_name, candidates, postproc=None):
    """
    Build final column + source from ordered candidates.
    candidates: list of tuples (series, label, transform_fn or None)
    postproc : optional function to apply to the final series at the end
    """
    # normalize series
    vals = []
    labs = []
    for ser, lab, tf in candidates:
        if ser is None:
            ser_n = pd.Series(pd.NA, index=df.index)
        else:
            ser_n = tf(ser) if callable(tf) else ser
        vals.append(ser_n)
        labs.append(lab)

    final = None
    src   = pd.Series(pd.NA, index=df.index, dtype="object")
    for ser, lab in zip(vals, labs):
        if final is None:
            take = ser.notna()
            final = ser.copy()
            src   = np.where(take, lab, src)
        else:
            take = final.isna() & ser.notna()
            final = final.where(~take, ser)
            src   = np.where(take, lab, src)

    if postproc:
        final = postproc(final)

    df[target_name] = final
    df[f"{target_name}_source"] = src
    return final, src

# ---------- build finals (priority orders are conservative & safe) ----------

# Title (simple alias to keep consistent)
df["title_final"] = df.get("primaryTitle")

# Year: startYear → year(release_date_final) → year(TMDb) → year(OMDb)
df["startYear"] = num(df.get("startYear")).astype("Int64")
y1 = df["startYear"]
y2 = year_of(df.get("release_date_final"))
y3 = year_of(df.get("release_date_tmdb"))
y4 = year_of(df.get("release_date_omdb"))
year_before = df.get("year_final", pd.Series(pd.NA, index=df.index))
year_final, _ = choose_with_source(
    "year_final",
    [(y1, "startYear", None),
     (y2, "release_date_final", None),
     (y3, "release_date_tmdb", None),
     (y4, "release_date_omdb", None)]
)

# Release date: existing final → TMDb → OMDb
d1 = pd.to_datetime(df.get("release_date_final"), errors="coerce")
d2 = pd.to_datetime(df.get("release_date_tmdb"),  errors="coerce")
d3 = pd.to_datetime(df.get("release_date_omdb"),  errors="coerce")
date_before = df.get("release_date_final2", pd.Series(pd.NA, index=df.index))
date_final, _ = choose_with_source(
    "release_date_final2",
    [(d1, "final_existing", None),
     (d2, "tmdb", None),
     (d3, "omdb", None)]
)

# Runtime: IMDb → OMDb
rt1 = num(df.get("runtimeMinutes"))
rt2 = num(df.get("runtime_omdb"))
rt_before = df.get("runtime_final", pd.Series(pd.NA, index=df.index))
rt_final, _ = choose_with_source(
    "runtime_final",
    [(rt1, "imdb_runtimeMinutes", None),
     (rt2, "omdb", None)]
)

# Language (ISO-2): language_best → omdb_code → original_language (if present)
l1 = iso2_lang(df.get("language_best"))
l2 = iso2_lang(df.get("language_omdb_code"))
l3 = iso2_lang(df.get("original_language")) if "original_language" in df.columns else pd.Series(pd.NA, index=df.index, dtype="string")
lang_before = df.get("language_final", pd.Series(pd.NA, index=df.index, dtype="string"))
lang_final, _ = choose_with_source(
    "language_final",
    [(l1, "imdb_language_best", None),
     (l2, "omdb_language_code", None),
     (l3, "tmdb_original_language", None)]
)

# Region/Country (ISO-2): region_best → omdb_country_code
r1 = iso2_region(df.get("region_best"))
r2 = iso2_region(df.get("country_omdb_code"))
reg_before = df.get("region_final", pd.Series(pd.NA, index=df.index, dtype="string"))
reg_final, _ = choose_with_source(
    "region_final",
    [(r1, "imdb_region_best", None),
     (r2, "omdb_country_code", None)]
)

# Rated (MPAA): IMDb → OMDb
rated1 = (df.get("rated").astype("string") if "rated" in df.columns else pd.Series(pd.NA, index=df.index, dtype="string"))
rated2 = (df.get("rated_omdb").astype("string") if "rated_omdb" in df.columns else pd.Series(pd.NA, index=df.index, dtype="string"))
rated_before = df.get("rated_final", pd.Series(pd.NA, index=df.index, dtype="string"))
rated_final, _ = choose_with_source(
    "rated_final",
    [(rated1, "imdb_rated", None),
     (rated2, "omdb_rated", None)]
)

# IMDb Ratings & Votes: IMDb → OMDb
ar1 = num(df.get("averageRating"))
ar2 = num(df.get("imdb_rating_omdb"))
nv1 = num(df.get("numVotes"))
nv2 = num(df.get("imdb_votes_omdb"))
rating_before = df.get("rating_imdb_final", pd.Series(pd.NA, index=df.index))
votes_before  = df.get("votes_imdb_final",  pd.Series(pd.NA, index=df.index))
choose_with_source("rating_imdb_final", [(ar1,"imdb_title_ratings",None),(ar2,"omdb_imdb_rating",None)])
choose_with_source("votes_imdb_final",  [(nv1,"imdb_title_ratings",None),(nv2,"omdb_imdb_votes",None)])

# Budget: unified budget_usd → tmdb → kaggle  (non-destructive)
for col in ["budget_usd","budget_tmdb","budget_kaggle"]:
    if col not in df.columns:
        df[col] = np.nan
budget_before = df.get("budget_final", pd.Series(pd.NA, index=df.index))
choose_with_source("budget_final",
    [(num(df["budget_usd"]),   "unified_budget_usd", None),
     (num(df["budget_tmdb"]),  "tmdb_budget",        None),
     (num(df["budget_kaggle"]), "kaggle_budget",     None)]
)

# Revenue totals: unified total → Numbers worldwide → TMDb
for col in ["revenue_total_usd","revenue_worldwide_numbers","revenue_tmdb"]:
    if col not in df.columns:
        df[col] = np.nan
rev_total_before = df.get("revenue_total_final", pd.Series(pd.NA, index=df.index))
choose_with_source("revenue_total_final",
    [(num(df["revenue_total_usd"]),        "unified_total_usd", None),
     (num(df["revenue_worldwide_numbers"]), "numbers_worldwide", None),
     (num(df["revenue_tmdb"]),             "tmdb_revenue",      None)]
)

# Revenue splits (best-effort surfaces; keep source-specific cols intact)
for col in ["revenue_domestic_numbers","revenue_domestic_omdb","revenue_international_numbers","revenue_worldwide_numbers"]:
    if col not in df.columns:
        df[col] = np.nan

dom_before = df.get("revenue_domestic_final", pd.Series(pd.NA, index=df.index))
intl_before= df.get("revenue_international_final", pd.Series(pd.NA, index=df.index))
ww_before  = df.get("revenue_worldwide_final", pd.Series(pd.NA, index=df.index))

df["revenue_domestic_final"]      = num(df["revenue_domestic_numbers"]).combine_first(num(df["revenue_domestic_omdb"]))
df["revenue_international_final"] = num(df["revenue_international_numbers"])
df["revenue_worldwide_final"]     = num(df["revenue_worldwide_numbers"]).combine_first(num(df["revenue_total_usd"]))

# ---------- coverage report (before → after with deltas) ----------
print("\n📊 Coverage improvements (before → after):")
report_delta("title_final",              df.get("title_final"),              df.get("title_final"))
report_delta("year_final",               year_before,                        df.get("year_final"))
report_delta("release_date_final2",      date_before,                        df.get("release_date_final2"))
report_delta("runtime_final",            rt_before,                          df.get("runtime_final"))
report_delta("language_final",           lang_before,                        df.get("language_final"))
report_delta("region_final",             reg_before,                         df.get("region_final"))
report_delta("rated_final",              rated_before,                       df.get("rated_final"))
report_delta("budget_final",             budget_before,                      df.get("budget_final"))
report_delta("revenue_total_final",      rev_total_before,                   df.get("revenue_total_final"))
report_delta("revenue_domestic_final",   dom_before,                         df.get("revenue_domestic_final"))
report_delta("revenue_international_final", intl_before,                     df.get("revenue_international_final"))
report_delta("revenue_worldwide_final",  ww_before,                          df.get("revenue_worldwide_final"))
report_delta("rating_imdb_final",        rating_before,                      df.get("rating_imdb_final"))
report_delta("votes_imdb_final",         votes_before,                       df.get("votes_imdb_final"))

# ---------- optional save ----------
if SAVE:
    df.to_csv(OUT, index=False, compression="gzip")
    print(f"\n💾 Saved collapsed file: {OUT}")
else:
    print("\n💡 Not saved (SAVE=False). If this looks good, set SAVE=True and re-run to write v9.")

📂 Loaded horror_data_master_v8.csv.gz (45,810 rows)

📊 Coverage improvements (before → after):
                 title_final:  45809/45810  (100.0%) →  45809/45810  (100.0%)   Δ +0
                  year_final:      0/45810  ( 0.0%) →  34152/45810  (74.6%)   Δ +34,152
         release_date_final2:      0/45810  ( 0.0%) →  32053/45810  (70.0%)   Δ +32,053
               runtime_final:      0/45810  ( 0.0%) →  29270/45810  (63.9%)   Δ +29,270
              language_final:      0/45810  ( 0.0%) →  33167/45810  (72.4%)   Δ +33,167
                region_final:      0/45810  ( 0.0%) →  33982/45810  (74.2%)   Δ +33,982
                 rated_final:      0/45810  ( 0.0%) →  10726/45810  (23.4%)   Δ +10,726
                budget_final:      0/45810  ( 0.0%) →   5780/45810  (12.6%)   Δ +5,780
         revenue_total_final:      0/45810  ( 0.0%) →   7326/45810  (16.0%)   Δ +7,326
      revenue_domestic_final:      0/45810  ( 0.0%) →   1649/45810  ( 3.6%)   Δ +1,649
 revenue_international_final:  

In [ ]:
# ===========================================
# Build v10: "final-only" columns
# Input : horror_data_master_v9_collapsed.csv.gz
# Output: horror_data_master_v10_final_only.csv.gz + .parquet
# ===========================================

import pandas as pd
from datetime import datetime

IN  = "horror_data_master_v9_collapsed.csv.gz"   # change if needed
CSV_OUT = "horror_data_master_v10_final_only.csv.gz"
PARQ_OUT = "horror_data_master_v10_final_only.parquet"

df = pd.read_csv(IN, compression="gzip", low_memory=False)
n = len(df)
print(f"📂 Loaded {IN} ({n:,} rows)")

# ——— final signals + provenance we want to keep ———
wanted = [
    # identity
    "tconst", "title_final", "year_final",

    # release & runtime
    "release_date_final2", "runtime_final",

    # language & region
    "language_final", "region_final",

    # ratings & votes
    "rating_imdb_final", "votes_imdb_final",

    # MPAA
    "rated_final",

    # money + provenance
    "budget_final", "budget_final_source",
    "revenue_domestic_final", "revenue_international_final", "revenue_worldwide_final",
    "revenue_total_final", "revenue_total_final_source",
]

# Keep only those that exist (robust to minor column drift)
final_cols = [c for c in wanted if c in df.columns]
missing = [c for c in wanted if c not in df.columns]
if missing:
    print("⚠️ Missing expected columns (skipped):", missing)

v10 = df[final_cols].copy()

# Simple coverage recap
def cov(s):
    return f"{s.notna().sum():,}/{len(s):,} ({100*s.notna().mean():.1f}%)"

print("\n📊 Coverage in v10 (selected):")
for c in ["year_final","release_date_final2","runtime_final",
          "language_final","region_final","rated_final",
          "budget_final","revenue_total_final",
          "revenue_domestic_final","revenue_international_final","revenue_worldwide_final",
          "rating_imdb_final","votes_imdb_final"]:
    if c in v10.columns:
        print(f"{c:>28}: {cov(v10[c])}")

# Save
v10.to_csv(CSV_OUT, index=False, compression="gzip")
v10.to_parquet(PARQ_OUT, index=False)

ts = datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"\n✅ Saved:\n  • {CSV_OUT}\n  • {PARQ_OUT}\n🕒 {ts}")

📂 Loaded horror_data_master_v9_collapsed.csv.gz (45,810 rows)

📊 Coverage in v10 (selected):
                  year_final: 34,152/45,810 (74.6%)
         release_date_final2: 32,053/45,810 (70.0%)
               runtime_final: 29,270/45,810 (63.9%)
              language_final: 33,167/45,810 (72.4%)
                region_final: 33,982/45,810 (74.2%)
                 rated_final: 10,726/45,810 (23.4%)
                budget_final: 5,780/45,810 (12.6%)
         revenue_total_final: 7,326/45,810 (16.0%)
      revenue_domestic_final: 1,649/45,810 (3.6%)
 revenue_international_final: 1,459/45,810 (3.2%)
     revenue_worldwide_final: 7,326/45,810 (16.0%)
           rating_imdb_final: 26,761/45,810 (58.4%)
            votes_imdb_final: 26,795/45,810 (58.5%)

✅ Saved:
  • horror_data_master_v10_final_only.csv.gz
  • horror_data_master_v10_final_only.parquet
🕒 2025-10-18 23:52
